![ModInterv](https://user-images.githubusercontent.com/66036012/85095333-72272e00-b1c7-11ea-8c85-8f251fea910a.jpg)



In [18]:
#nbi:hide_in

import pandas
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.style.use('ggplot')
!pip install -q lmfit &> /dev/null
import lmfit
!pip install -q numdifftools &> /dev/null
import numdifftools
from timeit import default_timer as timer
from lmfit import Parameters, Minimizer, report_fit
import ipywidgets as widgets
from ipywidgets import interact, Dropdown
from scipy.optimize import curve_fit, minimize_scalar, fsolve
from scipy.special import hyp2f1
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy.integrate import solve_ivp
from IPython.display import display
from IPython.display import FileLink, FileLinks
from contextlib import suppress
from IPython.display import HTML
import base64
#!pip install -q fpdf
#from fpdf import FPDF
import time
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timezone, timedelta, date
!pip install -q sigfig &> /dev/null
from sigfig import round
!pip install -q tsmoothie &> /dev/null
from tsmoothie.smoother import LowessSmoother
from scipy.signal import find_peaks
from scipy.integrate import odeint
from scipy import interpolate
!pip install -q dataframe_image &> /dev/null
import dataframe_image as dfi
from pandas.plotting import table
import dask.dataframe as dd

global R2_generalc
global stage
stage = ' '
global lang
lang = 'Eng'
global index
index = 0
global data_segunda_onda
data_segunda_onda = '0'
global date_first
date_first = '0'

button_pt = widgets.Button(description='Português')
button_en = widgets.Button(description='English')
display(widgets.HBox([button_pt, button_en]))
html_display_widget_intro = widgets.HTML(
    value='<hr><p>Este aplicativo faz ajustes matemáticos das curvas epidêmicas de países, bem como de estados e cidades do Brasil e dos EUA.</p> <p> Esta versão implementa um modelo com múltiplas ondas. Os ajustes numéricos podem demorar um tempo maior, devido ao maior número de parâmetros necessários para descrever múltiplas ondas epidêmicas. </p> <hr> <p>O aplicativo foi desenvolvido pela Rede de Pesquisa Modinterv-Covid19 formada por pesquisadores das Universidades Federais do Paraná, Pernambuco e Sergipe.</p> <hr>')
display(html_display_widget_intro)
html_widget_lang = widgets.HTML(value='Eng')

data_e_hora_atuais = datetime.now()
diferenca = timedelta(hours=-3)
fuso_horario = timezone(diferenca)
data_e_hora_sao_paulo = data_e_hora_atuais.astimezone(fuso_horario)
data_e_hora_sao_paulo_em_texto = data_e_hora_sao_paulo.strftime('%d/%m/%Y %H:%M')
data_e_hora_sao_paulo_em_texto_en = data_e_hora_sao_paulo.strftime('%m/%d/%Y %H:%M')
data_e_hora_papers = data_e_hora_sao_paulo.strftime('%b %e, %Y %H:%M')

time_dic = {'Eng':data_e_hora_sao_paulo_em_texto_en,'Por':data_e_hora_sao_paulo_em_texto}
data_dic = {'Eng':'Data up to ','Por':'Dados até '}

fase_en = ['increasing acceleration', 'decreasing acceleration', 'increasing deceleration', 'decreasing deceleration']
fase_br = ['aceleração crescente', 'aceleração decrescente','desaceleração crescente', 'desaceleração decrescente'] 
fase = {'Eng':fase_en,'Por':fase_br}
wave_disc = {'Eng':' (2nd wave)','Por':r' ($2^{{\rm a}}$ onda)'}
colordict = ['red', 'darkorange', 'yellow', 'green', 'royalblue']

estagio = {'Eng':'Epidemic stage:','Por':'Estágio da epidemia:'}

label_x = {'Eng':{'Deaths':'Days since first death','Cases':'Days since first case'},'Por':{'Deaths':'Dias desde o primeiro óbito','Cases':'Dias desde o primeiro caso'}}
label_y = {'Eng':{'Deaths':'Accumulated deaths','Cases':'Accumulated cases'},'Por':{'Deaths':'Óbitos acumulados','Cases':'Casos acumulados'}}
label_y_daily = {'Eng':{'Deaths':'Daily deaths','Cases':'Daily cases'},'Por':{'Deaths':'Óbitos diários','Cases':'Casos diários'}}

subs = {'Eng':{'Deaths':'Deaths','Cases':'Cases'},'Por':{'Deaths':'Mortes','Cases':'Casos'}}
methods_name = {'Eng':['q-exponential fit (1 wave)','RM fit (1 wave)','GRM fit (1 wave)','BLM fit (1 wave)','BLM fit (2 waves)'],'Por':['ajuste q-exponencial (1 onda)','ajuste MR (1 onda)','ajuste MRG (1 onda)','ajuste MLB (1 onda)','ajuste MLB (2 ondas)']}
methods_name1 = {'Eng':['q-exponential fit (1 wave)','RM fit (1 wave)','GRM fit (1 wave)','BLM fit (1 wave)','BLM fit (2 waves)'],'Por':['Ajuste q-exponencial (1 onda)','Ajuste MR (1 onda)','Ajuste MRG (1 onda)','Ajuste MLB (1 onda)','Ajuste MLB (2 ondas)']}
R2_message = {'Eng':'The methods did not provide reliable fits for this location, possibly because the curve has unusual behavior. The least-worst fit can be seen by clicking the checkbox below.','Por':'Os métodos não produziram ajustes confiáveis para esta localidade, possivelmente porque a curva tem um comportamento anômalo. Não obstante, o melhor ajuste dentre os métodos pode ser visto clicando na caixa de seleção abaixo.'}
dic = {'Linear':'linear', 'Logarithmic':'log'}
dic1 = {'Linear':'linear', 'Logarithmic':'log'}
wave_date = {'Eng':'Start 2nd wave: ', 'Por':r'Início $2^{{\rm a}}$ onda: '}
first_death = {'Eng':{'Deaths':'1st death: ','Cases':'1st case: '}, 'Por':{'Deaths':r'$1^{{\rm o}}$ óbito: ','Cases':r'$1^{{\rm o}}$ caso: '}}
#error_message = {'Eng':'Warning: fit not entirely reliable because of large errors in some of the parameters.','Por':'Aviso: ajuste não totalmente confiável por causa de erros grandes em algum dos parâmetros.'}
first_plateau = {'Eng':r'Plateau 1st wave: $K_1$= ','Por':r'Platô $1^{{\rm a}}$ onda: $K_1$= '}
second_plateau = {'Eng':r'Plateau 2nd wave: $K_2$= ','Por':r'Platô $2^{{\rm a}}$ onda: $K_2$= '}
html_loading_message = {'Eng':'<html><head><style>body {background-color: #ffffff;}h1 {background: url("http://i.stack.imgur.com/tQTRW.gif");background-repeat:no-repeat;background-size:120px 30px;font-size:14pt}</style></head><body><h1>Computing...</h1></body></html>','Por':'<html><head><style>body {background-color: #ffffff;}h1 {background: url("http://i.stack.imgur.com/tQTRW.gif");background-repeat:no-repeat;background-size:120px 30px;font-size:14pt}</style></head><body><h1>Calculando...</h1></body></html>'}
html_loading_data = {'Eng':'<html><head><style>body {background-color: #ffffff;}h1 {background: url("http://i.stack.imgur.com/tQTRW.gif");background-repeat:no-repeat;background-size:120px 30px;font-size:14pt}</style></head><body><h1>Loading...</h1></body></html>','Por':'<html><head><style>body {background-color: #ffffff;}h1 {background: url("http://i.stack.imgur.com/tQTRW.gif");background-repeat:no-repeat;background-size:120px 30px;font-size:14pt}</style></head><body><h1>Carregando...</h1></body></html>'}
first_peak = {'Eng':'Peak 1st wave: ','Por':r'Pico $1^{{\rm a}}$ onda: '}
second_peak = {'Eng':'Peak 2nd wave: ','Por':r'Pico $2^{{\rm a}}$ onda: '}
date_last_message = {'Eng':'Date of last point: ', 'Por':'Data do último ponto: '}
forecast_message = {'Eng':{'7':'Forecast 7d: ','14':'Forecast 14d: ','21':'Forecast 21d: ','28':'Forecast 28d: '}, 'Por':{'7':'Previsão 7d: ','14':'Previsão 14d: ','21':'Previsão 21d: ','28':'Previsão 28d: '}}
error_message = {'Eng':'<FONT COLOR=red><b>ERROR:</b> The numerical fitting for this location did not converge for the selected date. Please choose a new location or select an earlier date by sliding back the slider "Fit time".', 'Por':'<FONT COLOR=red><b>ERRO:</b> O ajuste numérico para esta localidade não convergiu para a data selecionada. Por favor escolha uma nova localidade ou selecione uma data anterior deslizando o controle "Tempo fit".'}
error_message1 = {'Eng':'<FONT COLOR=red><b>ERROR:</b> The chosen location does not have sufficient data in order to perform a numerical fitting. Please choose a new location.', 'Por':'<FONT COLOR=red><b>ERRO:</b> A localidade selecionada não possui dados suficientes para fazer um ajuste numérico. Por favor escolha uma nova localidade.'}
copyright_msg = '$\copyright$'+str(date.today().year)+' ModInterv'


HTML(value='<hr><p>Este aplicativo faz ajustes matemáticos das curvas epidêmicas de países, bem como de estado…

In [19]:
# updated version
def wave_detection(data):
  Tsep = 36
  Tcen = 36
  Pabs = 0.3
  maxima0, _ = find_peaks(data)
  minima0, _ = find_peaks(-data)
  maxima1 = []
  minima1 = []
  vec = []
  # label and unify all critical points in one array, then sort in ascending order
  for i in range (0, len(maxima0)):
    maxima1.append((maxima0[i], 'max'))
  for i in range (0, len(minima0)):
    minima1.append((minima0[i], 'min'))
  vec = maxima1 + minima1
  vec.sort()
  if vec[0][1]== 'min':
    vec.pop(0)
  # check if neighboring maxima or minima occur closer than Tsep
  i = 0
  while i>=0:
    try:
      if vec[i+2][0]-vec[i][0]<Tsep:
        if vec[i][1] == 'max':
          if data[vec[i+2][0]]>data[vec[i][0]]:
            vec.pop(i)
            vec.pop(i)
          else:
            vec.pop(i+1)
            vec.pop(i+1)
        if vec[i][1] == 'min':
          if data[vec[i+2][0]]<data[vec[i][0]]:
            vec.pop(i)
            vec.pop(i)
          else:
            vec.pop(i+1)
            vec.pop(i+1)
      else:
        i += 1
    except IndexError:
      i = -1
  # check if consecutive critical points occur in less than Tcen/2
  i = 0
  while i>=0:
    try:
      if vec[i+1][0]-vec[i][0]<Tcen/2:
        if (vec[i][1] == 'max') and (data[vec[i][0]-1+Tcen]>data[vec[i][0]-1]):
          vec.pop(i)
          vec.pop(i)
        if (vec[i][1] == 'min') and (data[vec[i][0]-1+Tcen]<data[vec[i][0]-1]):
          vec.pop(i)
          vec.pop(i)
        else:
          i += 1 
      else:
        i += 1
    except IndexError:
      i = -1
  # check the relative height of the peaks
  i = 0
  while i>=0:
    try:
      if i==0:
        if vec[i][1] == 'max':
          if abs(data[vec[i][0]]-data[vec[i+1][0]])/data[vec[i][0]]<Pabs:
            vec.pop(i)
            vec.pop(i)
          else:
            i += 1
        else:
          i += 1
      if i>=1:
        if vec[i][1] == 'max':
          if abs(data[vec[i][0]]-data[vec[i-1][0]])<Pabs*data[vec[i][0]]:
            vec.pop(i-1)
            vec.pop(i-1)
          if abs(data[vec[i][0]]-data[vec[i+1][0]])<Pabs*data[vec[i][0]]:
            vec.pop(i)
            vec.pop(i)
          else:
            i += 1
        else:
          i += 1
    except IndexError:
      i = -1
  vec_maxima = []
  vec_minima = []
  for i in range(0, len(vec)):
    if vec[i][1] == 'max':
      vec_maxima.append(vec[i][0])
    if vec[i][1] == 'min':
      vec_minima.append(vec[i][0])

  return vec_maxima, vec_minima

In [20]:
#nbi:hide_in

def RMPlot(Country,Type,name2,time_fit):
  name = Country
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
  df_recovered = pandas.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
  df_deaths = pandas.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
  df_confirmed = pandas.read_csv(url, error_bad_lines=False)
 
  df_deaths = df_deaths.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  df_confirmed = df_confirmed.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  deaths = df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
  recovered = df_recovered[(df_recovered['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
  confirmed = df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float')[0][np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_confirmed.columns.values)]
 
 
  if Type == 'Deaths':
    kind1 = deaths
  if Type == 'Cases':
    kind1 = confirmed

  kind1 = kind1[0:time_fit]
 
  t_forecast = 1 + 0/100
 
  t = np.linspace(0, int(t_forecast*len(kind1)), 2000)
  t0 = np.array(range(len(kind1)))
 
  def expfit(x,C0,r):
      return C0*np.exp(r*x)
 
  t_exp = np.array(range(len(kind1[0:20])))
  ydata = kind1[0:20]
  popt, pcov = curve_fit(expfit, t_exp, ydata)
  C0 = popt[0]
 
  def fcn2min(params, t, data):
      """Generalized Richard's Model"""
      K = params['K']
      alpha = params['alpha']
      r = params['r']
      tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
      model = K/(1+alpha*np.exp(-alpha*r*(t-tc)))**(1/alpha)
      return model - data
 
  params = Parameters()
  params.add('K', value=np.amax(kind1), min=0)
  params.add('alpha', value=.1, min=0, max=1)
  params.add('r', value=0.2, min=0, max=1)
 
  start =timer()
 
 
  minner = Minimizer(fcn2min, params, fcn_args=(t0,kind1))
 
 
  out1 = minner.minimize(method= 'least_squares')
 
  end = timer()
 
 
  start = timer()
 
  out2 = minner.minimize(method='leastsq', params=out1.params)
 
 
  end = timer()
 

  R2 = 1 - out2.residual.var() / np.var(kind1)
  global R2_generalc
  R2_generalc = R2
 
  params = out2.params
  K = params['K']
  alpha = params['alpha']
  r = params['r']

  K = np.float(params['K'].value)
  alpha = np.float(params['alpha'].value)
  r = np.float(params['r'].value)

  tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
  tc = np.float(tc)
  tjp = tc - np.log(1.5 + 0.5*alpha - 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
  tjm = tc - np.log(1.5 + 0.5*alpha + 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
  #ts = np.real(( alpha )**( -1 ) * ( r )**( -1 ) * np.log( ( 1/3 * ( 7 + 4 * alpha ) * ( np.e )**( alpha * r * tc ) + ( -1/3 * ( 2 )**( 1/3 ) * ( -31 * ( np.e )**( 2 * alpha * r * tc ) + ( -44 * alpha * ( np.e )**( 2 * alpha * r * tc ) + -13 * ( alpha )**( 2 ) * ( np.e )**( 2 * alpha * r * tc ) ) ) * ( ( 335 * ( np.e )**( 3 * alpha * r * tc ) + ( 708 * alpha * ( np.e )**( 3 * alpha * r * tc ) + ( 465 * ( alpha )**( 2 ) * ( np.e )**( 3 * alpha * r * tc ) + ( 92 * ( alpha )**( 3 ) * ( np.e )**( 3 * alpha * r * tc ) + 3 * ( 3 )**( 1/2 ) * ( ( -257 * ( np.e )**( 6 * alpha * r * tc ) + ( -1224 * alpha * ( np.e )**( 6 * alpha * r * tc ) + ( -2122 * ( alpha )**( 2 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -1712 * ( alpha )**( 3 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -681 * ( alpha )**( 4 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -136 * ( alpha )**( 5 ) * ( np.e )**( 6 * alpha * r * tc ) + -12 * ( alpha )**( 6 ) * ( np.e )**( 6 * alpha * r * tc ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) )**( -1/3 ) + 1/3 * ( 2 )**( -1/3 ) * ( ( 335 * ( np.e )**( 3 * alpha * r * tc ) + ( 708 * alpha * ( np.e )**( 3 * alpha * r * tc ) + ( 465 * ( alpha )**( 2 ) * ( np.e )**( 3 * alpha * r * tc ) + ( 92 * ( alpha )**( 3 ) * ( np.e )**( 3 * alpha * r * tc ) + 3 * ( 3 )**( 1/2 ) * ( ( -257 * ( np.e )**( 6 * alpha * r * tc ) + ( -1224 * alpha * ( np.e )**( 6 * alpha * r * tc ) + ( -2122 * ( alpha )**( 2 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -1712 * ( alpha )**( 3 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -681 * ( alpha )**( 4 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -136 * ( alpha )**( 5 ) * ( np.e )**( 6 * alpha * r * tc ) + -12 * ( alpha )**( 6 ) * ( np.e )**( 6 * alpha * r * tc ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) )**( 1/3 ) ) ) ))
  
  K = params['K']
  alpha = params['alpha']
  r = params['r']

  global stage
  if t[-1]>tjm and t[-1]<=tc:
    stage = 'early intermediate'
  if t[-1]>tc and t[-1]<=tjp:
    stage = 'late intermediate'
  if t[-1]>tjp:
    stage = 'early saturation'
  #if t[-1]>ts:
    #stage = 'late saturation'
 
  def func(t,r,alpha,K):
      tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
      return K/(1+alpha*np.exp(-alpha*r*(t-tc)))**(1/alpha)

  return Country, Type, name2, t0, kind1, K, r, alpha, C0

def qexpPlot(Country,Type,name2,time_fit):
  name = Country
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
  df_recovered = pandas.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
  df_deaths = pandas.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
  df_confirmed = pandas.read_csv(url, error_bad_lines=False)
 
  df_deaths = df_deaths.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  df_confirmed = df_confirmed.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  deaths = df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
  recovered = df_recovered[(df_recovered['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
  confirmed = df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float')[0][np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_confirmed.columns.values)]
 
 
  if Type == 'Deaths':
    kind1 = deaths
  if Type == 'Cases':
    kind1 = confirmed

  kind1 = kind1[0:time_fit]
  
  t_forecast = 1 + 0/100
 
  t = np.linspace(0, int(t_forecast*len(kind1)), 2000)
  t0 = np.array(range(len(kind1)))
 
 
  def fcn2min(params, t, data):
      """Generalized Richard's Model"""
      r = params['r']
      q = params['q']
      A = data[0]**(1-q)
      model = (A+(1-q)*r*t)**(1/(1-q))
      return model - data 
  
  params = Parameters()
  params.add('r', value=0.1, min=0)
  params.add('q', value=0.2, min=0, max=1) 
 
  minner = Minimizer(fcn2min, params, fcn_args=(t0,kind1))
  
  out1 = minner.minimize(method= 'nelder')
 
  out2 = minner.minimize(method='leastsq', params=out1.params)

  R2 = 1 - out2.residual.var() / np.var(kind1)
  global R2_generalc
  R2_generalc = R2
 
  params = out2.params
  r = params['r']
  q = params['q']
 
  def func(t,q,r):
      A = kind1[0]**(1-q)
      return (A+(1-q)*r*t)**(1/(1-q))
 
  A = kind1[0]**(1-q)
 
  td = (-A + (2*kind1[len(kind1)-1])**(1-q))/((1-q)*r) - len(kind1)-1

  return Country, Type, name2, params, t0, kind1, q, r

def qexpPlotPlot(Country,Type,name2, params,t0, kind1, q, r, Forecast, togg2, x_axis, y_axis, toggdetails1):
  name = Country
  t_forecast = 1 + Forecast/100
 
  t = np.linspace(0, len(kind1)-1+Forecast, 2000)

  def func(t,q,r):
      A = kind1[0]**(1-q)
      return (A+(1-q)*r*t)**(1/(1-q))

  def funcprime(t,q,r):
      A = kind1[0]**(1-q)
      return r*(A+(1-q)*r*t)**(q/(1-q))
 
  A = kind1[0]**(1-q)
 
  td = (-A + (2*kind1[len(kind1)-1])**(1-q))/((1-q)*r) - len(kind1)-1
  t_7 = func(max(t0)+7,q,r)

  if Type=='Deaths':
    tipo = 'Mortes'
    tipo1 = 'morte'
    tipo2 = 'a primeira morte'
  if Type=='Cases':
    tipo = 'Casos'
    tipo1 = 'caso'
    tipo2 = 'o primeiro caso'

  lang = html_widget_lang.value

  date_last = date_first + timedelta(days=len(kind1))

  if lang == 'Por':
    date_first1 = date_first.strftime('%d/%m/%y')
    date_last = date_last.strftime('%d/%m/%y')
  if lang == 'Eng':
    date_first1 = date_first.strftime('%m/%d/%y')
    date_last = date_last.strftime('%m/%d/%y')

  fig, ax = plt.subplots(figsize=(6,4))
  ax.plot(t0,kind1,'ro', alpha=0.5, label=str(subs[lang][Type])+' '+str(name))
  ax.plot(t,func(t,q,r),'-k', label=methods_name1[lang][0])
  ax.plot(t[0],kind1[0], 'wo', alpha=0, label=r'$r= $'+str('%.3f' % r.value))
  ax.plot(t[0],kind1[0], 'wo', alpha=0, label=r'$q= $'+str('%.3f' % q.value))
  ax.plot(t[0],kind1[0], 'wo', alpha=0, label=r'$t_d= $'+str('%.1f' % td))
  ax.plot(t[0],kind1[0], 'wo', alpha=0, label=first_death[lang][Type]+str(date_first1))
  ax.plot(t[0],kind1[0], 'ko', alpha=0, label=estagio[lang])
  ax.plot(t[0],kind1[0], 'o', alpha=0, label=fase[lang][0], color='red')
  ax.plot(t[0],kind1[0], 'o', alpha=0, label=copyright_msg)
  ax.set_xlabel(label_x[lang][Type])
  ax.set_ylabel(label_y[lang][Type])
  if x_axis=='Linear' and y_axis=='Linear':
    legend = ax.legend(loc='best', fontsize='small')
    line = legend.get_lines()
    text = legend.get_texts()
    text[-1].set_color(line[-1].get_color())
  if x_axis=='Logarithmic':
    ax.set_xlim(left=0.8)
  if y_axis=='Logarithmic':
    ax.set_ylim(bottom=0.8)
  plt.xscale(dic[x_axis])
  plt.yscale(dic[y_axis])
  plt.title(name, y=1.04)
  plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
  plt.tight_layout()
  plt.savefig('fig.png', format='png', dpi=200)
  ax.set_rasterized(True)
  plt.savefig('fig.eps', format='eps')
  plt.show()

  if toggdetails1 == True:
    variables = []
    for key, value in params.valuesdict().items():
      variables.append(key)

    ti_names = []
    ti_values = []

    ti_names.append('td')
    ti_values.append(td)
      
    for i in range(len(ti_names), 2):
      ti_names.append('')
      ti_values.append('')


    values = []
    stderr = []

    for i in range(2):
      values.append(params[variables[i]].value)
      stderr.append(params[variables[i]].stderr)

    if lang == 'Eng':
      d = {'Parameter':variables, 'Value':values, 'StdErr':stderr, 'ti':ti_names, 'ti_values':ti_values}
    else:
      d = {'Parâmetro':variables, 'Valor':values, 'Erro':stderr, 'ti':ti_names, 'valor_ti':ti_values}

    df_table = pd.DataFrame({ key:pd.Series(value) for key, value in d.items() })
    df_table.fillna('')
    pd.set_option('display.colheader_justify', 'center')
    print(df_table)
    fig, ax = plt.subplots(figsize=(8,4))
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    ax.table(cellText=df_table.values, colLabels=df_table.columns, loc='center', cellLoc='center', fontsize=12)
    fig.tight_layout()
    plt.savefig('fig2.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig2.eps', format='eps')
    plt.close(fig)

  if togg2==True:
    daily=[]
    for i in range(0,len(kind1)-1):
      daily.append(kind1[i+1]-kind1[i])
    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(t0[0:len(t0)-1],daily,'ro-', markersize=4, alpha=0.5, label=label_y_daily[lang][Type]+' '+str(name))
    ax.plot(t,funcprime(t,q,r), '-k', label=methods_name[lang][0])
    if x_axis=='Linear' and y_axis=='Linear':
      legend = ax.legend(loc='best', fontsize='small')
      pts = ax.dataLim.get_points()
      pts[0, 1] = 0 
      ax.dataLim.set_points(pts)
      ax.autoscale_view()
    if x_axis=='Logarithmic':
      ax.set_xlim(left=0.8)
    if y_axis=='Logarithmic':
      ax.set_ylim(bottom=0.8)
    ax.set_xlabel(label_x[lang][Type])
    ax.set_ylabel(label_y_daily[lang][Type])
    plt.xscale(dic[x_axis])
    plt.yscale(dic[y_axis])
    plt.title(name, y=1.04)
    plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
    plt.tight_layout()
    plt.savefig('fig1.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig1.eps', format='eps')
    plt.show()
  
def BLM_fit(Country,Type,name2,time_fit):
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    name = Country
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
    df_recovered = pandas.read_csv(url, error_bad_lines=False)
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
    df_deaths = pandas.read_csv(url, error_bad_lines=False)
    url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    df_confirmed = pandas.read_csv(url, error_bad_lines=False)
  
    df_deaths = df_deaths.groupby(['Country/Region']).aggregate(np.sum).reset_index()
    df_confirmed = df_confirmed.groupby(['Country/Region']).aggregate(np.sum).reset_index()
    deaths = df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
    recovered = df_recovered[(df_recovered['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
    confirmed = df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float')[0][np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_confirmed.columns.values)]

    global date_first
    if Type == 'Deaths':
      date_first = df_deaths.columns[3+np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]]
      date_first = datetime.strptime(date_first, '%m/%d/%y')
    if Type == 'Cases':
      date_first = df_confirmed.columns[3+np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]]
      date_first = datetime.strptime(date_first, '%m/%d/%y')

    if Type == 'Deaths':
      kind1 = deaths
    if Type == 'Cases':
      kind1 = confirmed 

    kind1 = kind1[0:time_fit]

    x = np.arange(0,len(kind1),1)

    daily = []

    daily.append(kind1[0])

    for j in range(0,len(kind1)-1):
        delta = kind1[j+1] - kind1[j]
        daily.append(delta)

    def f(y,beta,q,alpha):
        return hyp2f1(beta,(1-q)/alpha,1+(1-q)/alpha,y)


    def f_imp(C,C0,beta,q,alpha,r,K):
        return (C**(1-q)*f((C/K)**alpha,beta,q,alpha) - (C0)**(1-q)*f((C0/K)**alpha,beta,q,alpha))/(r*(1-q))



    def fcn2min(params, x, data):
      """hyper-generalized logistic Model (HGLM)."""
      K = params['K1']
      alpha = params['a1']
      q = params['q1']
      r = params['r1']
      C0=kind1[0]
      beta=params['p1']
      model = f_imp(x,C0,beta,q,alpha,r,K)
      return model - data

    params = Parameters()
    params.add('K1', value=1.1*max(kind1),min=max(kind1))
    params.add('a1', value=0.6, min=0, vary = True)
    params.add('q1', value=0.9, min=0.,max=.99999, vary=True)
    params.add('r1', value=0.5, min=0.,vary=True)

    try:
        params.add('p1', value=1, min=1.,vary=True)
        minner = Minimizer(fcn2min, params,
                      fcn_args=(kind1, x))

        
        result = minner.minimize(method= 'least_squares')

        result = minner.minimize(method= 'leastsq', params = result.params)


        R2 = 1 - result.residual.var() / np.var(kind1)
        global R2_generalc
        R2_generalc = R2

        params = result.params
        alpha2=params['a1']
        q2=params['q1']
        r2=params['r1']
        beta2=params['p1']
        K2=params['K1']
        C02=data[0]

        alpha2=np.float(params['a1'].value)
        q2=np.float(params['q1'].value)
        beta2=np.float(params['p1'].value)

        c_tc = ( ( q2 )**( -1 ) * ( alpha2 * beta2 + q2 ) )**( -1 * ( alpha2 )**( -1 ) )
        C_tc = K2*c_tc

        #c_ts = np.real( ( 6 )**( -1 * ( alpha2 )**( -1 ) ) * ( ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( -1 ) * ( 2 * ( ( alpha2 )**( 3 ) * beta2 * ( -1 + 7 * beta2 ) + ( 3 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 4 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 2 ) * beta2 * ( -3 + ( 7 * q2 + beta2 * ( -7 + 18 * q2 ) ) ) ) ) ) + ( 2 * ( 2 )**( 1/3 ) * ( alpha2 )**( 2 ) * beta2 * ( -3 * ( 3 + ( 3 * alpha2 + -7 * q2 ) ) * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 3 ) * ( 13 + ( 31 * ( alpha2 )**( 2 ) + 2 * alpha2 * ( -22 + 63 * q2 ) ) ) + ( alpha2 * ( beta2 )**( 2 ) * ( -14 + ( -14 * ( alpha2 )**( 3 ) + ( 26 * q2 + ( ( alpha2 )**( 2 ) * ( -7 + 8 * q2 ) + alpha2 * ( 35 + ( -289 * q2 + 378 * ( q2 )**( 2 ) ) ) ) ) ) ) + beta2 * ( 4 + ( ( alpha2 )**( 4 ) + ( ( alpha2 )**( 3 ) * ( 6 + -14 * q2 ) + ( -14 * q2 + ( 13 * ( q2 )**( 2 ) + ( ( alpha2 )**( 2 ) * ( -5 + ( 56 * q2 + -77 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * ( -3 + ( 70 * q2 + ( -223 * ( q2 )**( 2 ) + 189 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) ) ) ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( -1/3 ) + ( 2 )**( 2/3 ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( 1/3 ) ) ) )**( ( alpha2 )**( -1 ) ) )
        #C_ts = K2*c_ts

        c_tjm = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + -1 * alpha2 * ( beta2 + ( -4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
        C_tjm = K2*c_tjm

        c_tjp = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + alpha2 * ( -1 * beta2 + ( 4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
        C_tjp = K2*c_tjp

        alpha2=params['a1']
        q2=params['q1']
        beta2=params['p1']

        tc = f_imp(C_tc,C02,beta2,q2,alpha2,r2,K2)
        tjm = f_imp(C_tjm,C02,beta2,q2,alpha2,r2,K2)
        tjp = f_imp(C_tjp,C02,beta2,q2,alpha2,r2,K2)
        #ts = f_imp(C_ts,C02,beta2,q2,alpha2,r2,K2)
        global stage
        if x[-1]>tjm and x[-1]<=tc:
          stage = 'early intermediate'
        if x[-1]>tc and x[-1]<=tjp:
          stage = 'late intermediate'
        if x[-1]>tjp:
          stage = 'early saturation'
        #if x[-1]>ts:
          #stage = 'late saturation'

        test_vec = result.covar
        test_vec1 = [alpha2.stderr/alpha2.value, q2.stderr/q2.value, r2.stderr/r2.value, beta2.stderr/beta2.value, K2.stderr/K2.value]
        for i in range(0, len(test_vec1)):
          if test_vec1[i]>0.2:
            raise ValueError
        Method1 = 'BLM'
        return Method1, Country, Type, name2, params, x, kind1, alpha2, q2, r2, beta2, K2, C02
        
    except:
      try:
        params.add('p1', value=1, min=1.,vary=False)
        minner = Minimizer(fcn2min, params,
                      fcn_args=(kind1, x))

        
        result = minner.minimize(method= 'least_squares')

        result = minner.minimize(method= 'leastsq', params = result.params)


        R2 = 1 - result.residual.var() / np.var(kind1)
        R2_generalc = R2

        params = result.params
        alpha2=params['a1']
        q2=params['q1']
        r2=params['r1']
        beta2=params['p1']
        K2=params['K1']
        C02=data[0]

        alpha2=np.float(params['a1'].value)
        q2=np.float(params['q1'].value)
        beta2=np.float(params['p1'].value)

        c_tc = ( ( q2 )**( -1 ) * ( alpha2 * beta2 + q2 ) )**( -1 * ( alpha2 )**( -1 ) )
        C_tc = K2*c_tc

        #c_ts = np.real( ( 6 )**( -1 * ( alpha2 )**( -1 ) ) * ( ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( -1 ) * ( 2 * ( ( alpha2 )**( 3 ) * beta2 * ( -1 + 7 * beta2 ) + ( 3 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 4 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 2 ) * beta2 * ( -3 + ( 7 * q2 + beta2 * ( -7 + 18 * q2 ) ) ) ) ) ) + ( 2 * ( 2 )**( 1/3 ) * ( alpha2 )**( 2 ) * beta2 * ( -3 * ( 3 + ( 3 * alpha2 + -7 * q2 ) ) * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 3 ) * ( 13 + ( 31 * ( alpha2 )**( 2 ) + 2 * alpha2 * ( -22 + 63 * q2 ) ) ) + ( alpha2 * ( beta2 )**( 2 ) * ( -14 + ( -14 * ( alpha2 )**( 3 ) + ( 26 * q2 + ( ( alpha2 )**( 2 ) * ( -7 + 8 * q2 ) + alpha2 * ( 35 + ( -289 * q2 + 378 * ( q2 )**( 2 ) ) ) ) ) ) ) + beta2 * ( 4 + ( ( alpha2 )**( 4 ) + ( ( alpha2 )**( 3 ) * ( 6 + -14 * q2 ) + ( -14 * q2 + ( 13 * ( q2 )**( 2 ) + ( ( alpha2 )**( 2 ) * ( -5 + ( 56 * q2 + -77 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * ( -3 + ( 70 * q2 + ( -223 * ( q2 )**( 2 ) + 189 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) ) ) ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( -1/3 ) + ( 2 )**( 2/3 ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( 1/3 ) ) ) )**( ( alpha2 )**( -1 ) ) )
        #C_ts = K2*c_ts

        c_tjm = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + -1 * alpha2 * ( beta2 + ( -4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
        C_tjm = K2*c_tjm

        c_tjp = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + alpha2 * ( -1 * beta2 + ( 4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
        C_tjp = K2*c_tjp

        alpha2=params['a1']
        q2=params['q1']
        beta2=params['p1']

        tc = f_imp(C_tc,C02,beta2,q2,alpha2,r2,K2)
        tjm = f_imp(C_tjm,C02,beta2,q2,alpha2,r2,K2)
        tjp = f_imp(C_tjp,C02,beta2,q2,alpha2,r2,K2)
        #ts = f_imp(C_ts,C02,beta2,q2,alpha2,r2,K2)
        if x[-1]>tjm and x[-1]<=tc:
          stage = 'early intermediate'
        if x[-1]>tc and x[-1]<=tjp:
          stage = 'late intermediate'
        if x[-1]>tjp:
          stage = 'early saturation'
        #if x[-1]>ts:
          #stage = 'late saturation'

        test_vec = result.covar
        test_vec1 = [alpha2.stderr/alpha2.value, q2.stderr/q2.value, r2.stderr/r2.value, K2.stderr/K2.value]
        for i in range(0, len(test_vec1)):
          if test_vec1[i]>0.2:
            raise ValueError
        Method1 = 'GRM'
        return Method1, Country, Type, name2, params, x, kind1, alpha2, q2, r2, beta2, K2, C02
        
      except:
        Method1 = 'RM'
        Country, Type, name2, x, kind1, K, r, alpha, C0 = RMPlot(Country,Type,name2,time_fit)
        return Method1, Country, Type, name2, params, x, kind1, alpha, 1, r, 1, K, C0
      

def BLM_plot(Country, Type, name2, params, x, kind1, alpha2, q2, r2, beta2, K2, C02, Forecast, interv_time, interv_param, togg1, togg2, x_axis, y_axis, toggdetails1):
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    name = Country
    data = kind1
    t0 = x

    args2 = (alpha2, q2, r2,beta2, K2)

    def f_ric1(t,y,*args2): 
        return r2*(y)**q2*((1-(y/K2)**alpha2))**beta2

    def f(y,beta,q,alpha):
        return hyp2f1(beta,(1-q)/alpha,1+(1-q)/alpha,y)


    def f_imp(C,C0,beta,q,alpha,r,K):
        return (C**(1-q)*f((C/K)**alpha,beta,q,alpha) - (C0)**(1-q)*f((C0/K)**alpha,beta,q,alpha))/(r*(1-q))

    def timefit(C_curve):
        return (f_imp(C_curve*K2,C02,beta2,q2,alpha2,r2,K2) - x[-1])**2
  
    def timefit1(C_curve):
        return (f_imp(C_curve*K2,C02,beta2,q2,alpha2,r2,K2) - (x[-1]+Forecast))**2
  
    res = minimize_scalar(timefit, bounds=(0, 1), method='bounded')
    C_plot = res.x

    res1 = minimize_scalar(timefit1, bounds=(0, 1), method='bounded')
    C_plot1 = res1.x

    const = 0.9999 - C_plot
    t_forecast1 = const*Forecast/100
    c_forecast = np.arange(C02,C_plot*K2+1+Forecast,0.1)

    alpha3 = alpha2
    q3 = q2
    beta3 = beta2

    alpha2 = np.float(alpha2.value)
    q2 = np.float(q2.value)
    beta2 = np.float(beta2.value)

    c_tc = ( ( q2 )**( -1 ) * ( alpha2 * beta2 + q2 ) )**( -1 * ( alpha2 )**( -1 ) )
    C_tc = K2*c_tc

    #c_ts = ( 6 )**( -1 * ( alpha2 )**( -1 ) ) * ( ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( -1 ) * ( 2 * ( ( alpha2 )**( 3 ) * beta2 * ( -1 + 7 * beta2 ) + ( 3 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 4 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 2 ) * beta2 * ( -3 + ( 7 * q2 + beta2 * ( -7 + 18 * q2 ) ) ) ) ) ) + ( 2 * ( 2 )**( 1/3 ) * ( alpha2 )**( 2 ) * beta2 * ( -3 * ( 3 + ( 3 * alpha2 + -7 * q2 ) ) * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 3 ) * ( 13 + ( 31 * ( alpha2 )**( 2 ) + 2 * alpha2 * ( -22 + 63 * q2 ) ) ) + ( alpha2 * ( beta2 )**( 2 ) * ( -14 + ( -14 * ( alpha2 )**( 3 ) + ( 26 * q2 + ( ( alpha2 )**( 2 ) * ( -7 + 8 * q2 ) + alpha2 * ( 35 + ( -289 * q2 + 378 * ( q2 )**( 2 ) ) ) ) ) ) ) + beta2 * ( 4 + ( ( alpha2 )**( 4 ) + ( ( alpha2 )**( 3 ) * ( 6 + -14 * q2 ) + ( -14 * q2 + ( 13 * ( q2 )**( 2 ) + ( ( alpha2 )**( 2 ) * ( -5 + ( 56 * q2 + -77 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * ( -3 + ( 70 * q2 + ( -223 * ( q2 )**( 2 ) + 189 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) ) ) ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( -1/3 ) + ( 2 )**( 2/3 ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( 1/3 ) ) ) )**( ( alpha2 )**( -1 ) )
    #C_ts = K2*np.real(c_ts)

    c_tjm = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + -1 * alpha2 * ( beta2 + ( -4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
    C_tjm = K2*c_tjm

    c_tjp = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + alpha2 * ( -1 * beta2 + ( 4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
    C_tjp = K2*c_tjp

    alpha2 = alpha3
    q2 = q3
    beta2 = beta3

    tc = f_imp(C_tc,C02,beta2,q2,alpha2,r2,K2)
    tjm = f_imp(C_tjm,C02,beta2,q2,alpha2,r2,K2)
    tjp = f_imp(C_tjp,C02,beta2,q2,alpha2,r2,K2)
    #ts = f_imp(C_ts,C02,beta2,q2,alpha2,r2,K2)

    global date_first
    if Type == 'Deaths':
      date_first_peak1 = date_first + timedelta(days=int(tc))
    if Type == 'Cases':
      date_first_peak1 = date_first + timedelta(days=int(tc))
      

    lang = html_widget_lang.value

    date_last = date_first + timedelta(days=len(kind1))

    if lang == 'Por':
      date_first1 = date_first.strftime('%d/%m/%y')
      date_first_peak1 = date_first_peak1.strftime('%d/%m/%y')
      date_last = date_last.strftime('%d/%m/%y')
    if lang == 'Eng':
      date_first1 = date_first.strftime('%m/%d/%y')
      date_first_peak1 = date_first_peak1.strftime('%m/%d/%y')
      date_last = date_last.strftime('%m/%d/%y')

    if Type=='Deaths':
      tipo = 'Mortes'
      tipo1 = 'morte'
      tipo2 = 'a primeira morte'
    if Type=='Cases':
      tipo = 'Casos'
      tipo1 = 'caso'
      tipo2 = 'o primeiro caso'
    

    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(x,data, 'ro',alpha=0.5, label = str(subs[lang][Type])+' '+str(name))
    ax.plot(f_imp(c_forecast,C02,beta2,q2,alpha2,r2,K2),c_forecast, '-k', label = methods_name1[lang][3])
    ax.plot(x[0],data[0], 'wo', alpha=0, label=first_death[lang][Type]+str(date_first1))
    if Forecast>0:
      ax.plot(x[0],data[0], 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+'+'+str(int(C_plot1*K2-C_plot*K2))+' '+subs[lang][Type])
      ax.plot(x[0],data[0], 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+str(int(C_plot1*K2))+' '+subs[lang][Type])
    ax.plot(x[0],data[0], 'ko', alpha=0, label=estagio[lang])
    global stage
    if stage == 'early intermediate':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][1], color='darkorange')
    if stage == 'late intermediate':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][2], color='yellow')
    if stage == 'early saturation':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      ax.plot(np.ones(100)*tjp,np.linspace(0,C_tjp,100), '-', alpha=1, color='green')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][3], color='green')
    #if stage == 'late saturation':
      #ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      #ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      #ax.plot(np.ones(100)*tjp,np.linspace(0,C_tjp,100), '-', alpha=1, color='green')
      #ax.plot(np.ones(100)*ts,np.linspace(0,C_ts,100), '-', alpha=1, color='royalblue')
      #ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][4], color='royalblue')
    ax.plot(x[0],data[0], 'o', alpha=0, label=copyright_msg)
    ax.set_xlabel(label_x[lang][Type])
    ax.set_ylabel(label_y[lang][Type])
    plt.xscale(dic[x_axis])
    plt.yscale(dic[y_axis])
    if x_axis=='Linear' and y_axis=='Linear':
      legend = ax.legend(loc='best', fontsize='small')
      line = legend.get_lines()
      text = legend.get_texts()
      text[len(text)-2].set_color(line[len(text)-2].get_color())
    if x_axis=='Logarithmic':
      ax.set_xlim(left=0.8)
    if y_axis=='Logarithmic':
      ax.set_ylim(bottom=0.8)
    plt.title(name, y=1.04)
    plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
    plt.tight_layout()
    plt.savefig('fig.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig.eps', format='eps')
    plt.show()

    if toggdetails1 == True:
      variables = []
      for key, value in params.valuesdict().items():
        variables.append(key)

      ti_names = []
      ti_values = []
      ti_vec = [tjm, tc, tjp]

      stages_dic = {'early intermediate':1, 'late intermediate':2, 'early saturation':3}

      for i in range(stages_dic[stage]):
        ti_names.append('t'+str(i+1))
        ti_values.append(ti_vec[i])
        
      for i in range(len(ti_names), 5):
        ti_names.append('')
        ti_values.append('')


      values = []
      stderr = []

      for i in range(5):
        values.append(params[variables[i]].value)
        stderr.append(params[variables[i]].stderr)

      if lang == 'Eng':
        d = {'Parameter':variables, 'Value':values, 'StdErr':stderr, 'ti':ti_names, 'ti_values':ti_values}
      else:
        d = {'Parâmetro':variables, 'Valor':values, 'Erro':stderr, 'ti':ti_names, 'valor_ti':ti_values}

      df_table = pd.DataFrame({ key:pd.Series(value) for key, value in d.items() })
      df_table.fillna('')
      pd.set_option('display.colheader_justify', 'center')
      print(df_table)
      fig, ax = plt.subplots(figsize=(8,4))
      fig.patch.set_visible(False)
      ax.axis('off')
      ax.axis('tight')
      ax.table(cellText=df_table.values, colLabels=df_table.columns, loc='center', cellLoc='center', fontsize=12)
      fig.tight_layout()
      plt.savefig('fig2.png', format='png', dpi=200)
      ax.set_rasterized(True)
      plt.savefig('fig2.eps', format='eps')
      plt.close(fig)
    
    if togg2==True:
      daily=[]
      daily.append(data[0])
      for i in range(0,len(data)-1):
        daily.append(data[i+1]-data[i])
      for i in range(0,len(daily)):
        if daily[i]<0:
          daily[i] = 0.5*(daily[i-1]+daily[i+1])
      fig, ax = plt.subplots(figsize=(6,4))
      ax.plot(x[0:len(x)],daily,'ro-', markersize=4, alpha=0.5, label=label_y_daily[lang][Type]+' '+str(name))
      ax.plot(f_imp(c_forecast,C02,beta2,q2,alpha2,r2,K2),f_ric1(0,c_forecast,*args2), '-k', label=methods_name1[lang][3])
      ax.plot(tc,f_ric1(0,C_tc,*args2), 'o', color='yellow', markersize=8, alpha=1)
      ax.plot(tc,f_ric1(0,C_tc,*args2), 'ok', markersize=8, alpha=0, label=first_peak[lang]+str(date_first_peak1))
      #ax.plot(tc*np.ones(100),np.linspace(0,f_ric1(0,C_tc,*args2),100),'-',color='yellow',alpha=1,linewidth=2,label=r'$t_c$')
      if x_axis=='Linear' and y_axis=='Linear':
        legend = ax.legend(loc='best', fontsize='small')
        pts = ax.dataLim.get_points()
        pts[0, 1] = 0 
        ax.dataLim.set_points(pts)
        ax.autoscale_view()
      if x_axis=='Logarithmic':
        ax.set_xlim(left=0.8)
      if y_axis=='Logarithmic':
        ax.set_ylim(bottom=0.8)
      ax.set_xlabel(label_x[lang][Type])
      ax.set_ylabel(label_y_daily[lang][Type])
      #ax.set_title(name+' '+methods_name[lang][3],y=1.05)
      #plt.figtext(0.51,0.9,time_dic[lang]+' UTC-3', fontsize=8,ha='center')
      plt.xscale(dic[x_axis])
      plt.yscale(dic[y_axis])
      plt.title(name, y=1.04)
      plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
      #plt.suptitle('('+data_dic[lang]+date_last+')', x=0.57, y=0.91, fontsize=8)
      plt.tight_layout()
      plt.savefig('fig1.png', format='png', dpi=200)
      ax.set_rasterized(True)
      plt.savefig('fig1.eps', format='eps')
      plt.show()

def RMPlotPlot(Country, Type, name2, params, t0, kind1, K, r, alpha, C0, Forecast, interv_time, interv_param, togg1, togg2, x_axis, y_axis, toggdetails1):

  name = Country
 
  t = np.arange(0, len(kind1)+Forecast, 1)

  def func(t,r,alpha,K):
      tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
      return K/(1+alpha*np.exp(-alpha*r*(t-tc)))**(1/alpha)
  
  alpha1 = alpha
  K1 = K
  r1 = r

  K = np.float(K.value)
  alpha = np.float(alpha.value)
  r = np.float(r.value)
  
  tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
  tc = np.float(tc)
  tjp = tc - np.log(1.5 + 0.5*alpha - 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
  tjm = tc - np.log(1.5 + 0.5*alpha + 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
  #ts = np.real(( alpha )**( -1 ) * ( r )**( -1 ) * np.log( ( 1/3 * ( 7 + 4 * alpha ) * ( np.e )**( alpha * r * tc ) + ( -1/3 * ( 2 )**( 1/3 ) * ( -31 * ( np.e )**( 2 * alpha * r * tc ) + ( -44 * alpha * ( np.e )**( 2 * alpha * r * tc ) + -13 * ( alpha )**( 2 ) * ( np.e )**( 2 * alpha * r * tc ) ) ) * ( ( 335 * ( np.e )**( 3 * alpha * r * tc ) + ( 708 * alpha * ( np.e )**( 3 * alpha * r * tc ) + ( 465 * ( alpha )**( 2 ) * ( np.e )**( 3 * alpha * r * tc ) + ( 92 * ( alpha )**( 3 ) * ( np.e )**( 3 * alpha * r * tc ) + 3 * ( 3 )**( 1/2 ) * ( ( -257 * ( np.e )**( 6 * alpha * r * tc ) + ( -1224 * alpha * ( np.e )**( 6 * alpha * r * tc ) + ( -2122 * ( alpha )**( 2 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -1712 * ( alpha )**( 3 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -681 * ( alpha )**( 4 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -136 * ( alpha )**( 5 ) * ( np.e )**( 6 * alpha * r * tc ) + -12 * ( alpha )**( 6 ) * ( np.e )**( 6 * alpha * r * tc ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) )**( -1/3 ) + 1/3 * ( 2 )**( -1/3 ) * ( ( 335 * ( np.e )**( 3 * alpha * r * tc ) + ( 708 * alpha * ( np.e )**( 3 * alpha * r * tc ) + ( 465 * ( alpha )**( 2 ) * ( np.e )**( 3 * alpha * r * tc ) + ( 92 * ( alpha )**( 3 ) * ( np.e )**( 3 * alpha * r * tc ) + 3 * ( 3 )**( 1/2 ) * ( ( -257 * ( np.e )**( 6 * alpha * r * tc ) + ( -1224 * alpha * ( np.e )**( 6 * alpha * r * tc ) + ( -2122 * ( alpha )**( 2 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -1712 * ( alpha )**( 3 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -681 * ( alpha )**( 4 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -136 * ( alpha )**( 5 ) * ( np.e )**( 6 * alpha * r * tc ) + -12 * ( alpha )**( 6 ) * ( np.e )**( 6 * alpha * r * tc ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) )**( 1/3 ) ) ) ))

  global date_first
  if Type == 'Deaths':
    date_first_peak1 = date_first + timedelta(days=int(tc))
  if Type == 'Cases':
    date_first_peak1 = date_first + timedelta(days=int(tc))
    
  lang = html_widget_lang.value

  date_last = date_first + timedelta(days=len(kind1))

  if lang == 'Por':
    date_first1 = date_first.strftime('%d/%m/%y')
    date_first_peak1 = date_first_peak1.strftime('%d/%m/%y')
    date_last = date_last.strftime('%d/%m/%y')
  if lang == 'Eng':
    date_first1 = date_first.strftime('%m/%d/%y')
    date_first_peak1 = date_first_peak1.strftime('%m/%d/%y')
    date_last = date_last.strftime('%m/%d/%y')

  alpha = alpha1
  K = K1
  r = r1
  
  def funcprime(t,r,alpha,K):
    tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
    return (K*r*alpha*np.exp(-alpha*r*(t-tc)))/(1+alpha*np.exp(-alpha*r*(t-tc)))**(1 + 1/alpha)

  if Type=='Deaths':
    tipo = 'Mortes'
    tipo1 = 'morte'
    tipo2 = 'a primeira morte'
  if Type=='Cases':
    tipo = 'Casos'
    tipo1 = 'caso'
    tipo2 = 'o primeiro caso'

  curve_forecast = func(t,r,alpha,K)
  curve = func(np.arange(0, len(kind1), 1),r,alpha,K) 
  increment = curve_forecast[-1] - curve[-1]

  fig, ax = plt.subplots(figsize=(6,4))
  ax.plot(t0,kind1,'ro', alpha=0.5, label=str(subs[lang][Type])+' '+str(name))
  ax.plot(t,func(t,r,alpha,K), '-k', label=methods_name1[lang][1])
  ax.plot(t[0],func(t[0],r,alpha,K), 'wo', alpha=0, label=first_death[lang][Type]+str(date_first1)) 
  if Forecast>0:
    ax.plot(t[0],func(t[0],r,alpha,K), 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+'+'+str(increment)+' '+subs[lang][Type])
    ax.plot(t[0],func(t[0],r,alpha,K), 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+str(curve_forecast[-1])+' '+subs[lang][Type])
  ax.plot(t[0],kind1[0], 'ko', alpha=0, label=estagio[lang])
  global stage
  if stage == 'early intermediate':
    ax.plot(np.ones(100)*tjm,np.linspace(0,func(tjm,r,alpha,K),100), '-', alpha=1, color='darkorange')
    ax.plot(t[0],kind1[0], 'o', alpha=0, label=fase[lang][1], color='darkorange')
  if stage == 'late intermediate':
    ax.plot(np.ones(100)*tjm,np.linspace(0,func(tjm,r,alpha,K),100), '-', alpha=1, color='darkorange')
    ax.plot(np.ones(100)*tc,np.linspace(0,func(tc,r,alpha,K),100), '-', alpha=1, color='yellow')
    ax.plot(t[0],kind1[0], 'o', alpha=0, label=fase[lang][2], color='yellow')
  if stage == 'early saturation':
    ax.plot(np.ones(100)*tjm,np.linspace(0,func(tjm,r,alpha,K),100), '-', alpha=1, color='darkorange')
    ax.plot(np.ones(100)*tc,np.linspace(0,func(tc,r,alpha,K),100), '-', alpha=1, color='yellow')
    ax.plot(np.ones(100)*tjp,np.linspace(0,func(tjp,r,alpha,K),100), '-', alpha=1, color='green')
    ax.plot(t[0],kind1[0], 'o', alpha=0, label=fase[lang][3], color='green')
  #if stage == 'late saturation':
    #ax.plot(np.ones(100)*tjm,np.linspace(0,func(tjm,r,alpha,K),100), '-', alpha=1, color='darkorange')
    #ax.plot(np.ones(100)*tc,np.linspace(0,func(tc,r,alpha,K),100), '-', alpha=1, color='yellow')
    #ax.plot(np.ones(100)*tjp,np.linspace(0,func(tjp,r,alpha,K),100), '-', alpha=1, color='green')
    #ax.plot(np.ones(100)*ts,np.linspace(0,func(ts,r,alpha,K),100), '-', alpha=1, color='royalblue')
    #ax.plot(t[0],kind1[0], 'o', alpha=0, label=fase[lang][4], color='royalblue')
  ax.plot(t[0],kind1[0], 'o', alpha=0, label=copyright_msg)
  ax.set_xlabel(label_x[lang][Type])
  ax.set_ylabel(label_y[lang][Type])
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True)
  plt.xscale(dic[x_axis])
  plt.yscale(dic[y_axis])
  if x_axis=='Linear' and y_axis=='Linear':
    legend = ax.legend(loc='best', fontsize='small')
    line = legend.get_lines()
    text = legend.get_texts()
    text[len(text)-2].set_color(line[len(text)-2].get_color())
  if x_axis=='Logarithmic':
    ax.set_xlim(left=0.8)
  if y_axis=='Logarithmic':
    ax.set_ylim(bottom=0.8)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  plt.title(name, y=1.04)
  plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
  plt.tight_layout()
  plt.savefig('fig.png', format='png', dpi=200)
  ax.set_rasterized(True)
  plt.savefig('fig.eps', format='eps')
  plt.show()

  if toggdetails1 == True:
    variables = []
    for key, value in params.valuesdict().items():
      variables.append(key)

    ti_names = []
    ti_values = []
    ti_vec = [tjm, tc, tjp]

    stages_dic = {'early intermediate':1, 'late intermediate':2, 'early saturation':3}

    for i in range(stages_dic[stage]):
      ti_names.append('t'+str(i+1))
      ti_values.append(ti_vec[i])
      
    for i in range(len(ti_names), 5):
      ti_names.append('')
      ti_values.append('')


    values = []
    stderr = []

    for i in range(5):
      values.append(params[variables[i]].value)
      stderr.append(params[variables[i]].stderr)

    if lang == 'Eng':
      d = {'Parameter':variables, 'Value':values, 'StdErr':stderr, 'ti':ti_names, 'ti_values':ti_values}
    else:
      d = {'Parâmetro':variables, 'Valor':values, 'Erro':stderr, 'ti':ti_names, 'valor_ti':ti_values}

    df_table = pd.DataFrame({ key:pd.Series(value) for key, value in d.items() })
    df_table.fillna('')
    pd.set_option('display.colheader_justify', 'center')
    print(df_table)
    fig, ax = plt.subplots(figsize=(8,4))
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    ax.table(cellText=df_table.values, colLabels=df_table.columns, loc='center', cellLoc='center', fontsize=12)
    fig.tight_layout()
    plt.savefig('fig2.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig2.eps', format='eps')
    plt.close(fig)

  if togg2==True:
    daily=[]
    for i in range(0,len(kind1)-1):
      daily.append(kind1[i+1]-kind1[i])
    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(t0[0:len(t0)-1],daily,'ro-', markersize=4, alpha=0.5, label=label_y_daily[lang][Type]+' '+str(name))
    ax.plot(t,funcprime(t,r,alpha,K), '-k', label=methods_name1[lang][1])
    ax.plot(tc,funcprime(tc,r,alpha,K), 'o', color='yellow', markersize=8,alpha=1)
    ax.plot(tc,funcprime(tc,r,alpha,K),'ok',alpha=0,label=first_peak[lang]+str(date_first_peak1))
    if x_axis=='Linear' and y_axis=='Linear':
      legend = ax.legend(loc='best', fontsize='small')
      pts = ax.dataLim.get_points()
      pts[0, 1] = 0 
      ax.dataLim.set_points(pts)
      ax.autoscale_view()
    if x_axis=='Logarithmic':
      ax.set_xlim(left=0.8)
    if y_axis=='Logarithmic':
      ax.set_ylim(bottom=0.8)
    ax.set_xlabel(label_x[lang][Type])
    ax.set_ylabel(label_y_daily[lang][Type])
    plt.xscale(dic[x_axis])
    plt.yscale(dic[y_axis])
    plt.title(name, y=1.04)
    plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
    plt.tight_layout()
    plt.savefig('fig1.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig1.eps', format='eps')
    plt.show()


def GRMPlotPlot(Country, Type, name2, params, x, kind1, alpha2, q2, r2, beta2, K2, C02, Forecast, interv_time, interv_param,togg1, togg2, x_axis, y_axis, toggdetails1):
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    name = Country
    data = kind1
    t0 = x

    args2 = (alpha2, q2, r2,beta2, K2)

    def f_ric1(t,y,*args2): 
        return r2*(y)**q2*((1-(y/K2)**alpha2))**beta2

    def f(y,beta,q,alpha):
        return hyp2f1(beta,(1-q)/alpha,1+(1-q)/alpha,y)


    def f_imp(C,C0,beta,q,alpha,r,K):
        return (C**(1-q)*f((C/K)**alpha,beta,q,alpha) - (C0)**(1-q)*f((C0/K)**alpha,beta,q,alpha))/(r*(1-q))


    def timefit(C_curve):
        return (f_imp(C_curve*K2,C02,beta2,q2,alpha2,r2,K2) - x[-1])**2

    def timefit1(C_curve):
        return (f_imp(C_curve*K2,C02,beta2,q2,alpha2,r2,K2) - (x[-1]+Forecast))**2
  
    res = minimize_scalar(timefit, bounds=(0, 1), method='bounded')
    C_plot = res.x

    res1 = minimize_scalar(timefit1, bounds=(0, 1), method='bounded')
    C_plot1 = res1.x

    const = 0.9999 - C_plot
    t_forecast1 = const*Forecast/100
    c_forecast = np.arange(C02,C_plot*K2+1+Forecast,0.1)

    alpha3 = alpha2
    q3 = q2
    beta3 = beta2

    alpha2 = np.float(alpha2.value)
    q2 = np.float(q2.value)
    beta2 = np.float(beta2.value)

    c_tc = ( ( q2 )**( -1 ) * ( alpha2 * beta2 + q2 ) )**( -1 * ( alpha2 )**( -1 ) )
    C_tc = K2*c_tc

    #c_ts = ( 6 )**( -1 * ( alpha2 )**( -1 ) ) * ( ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( -1 ) * ( 2 * ( ( alpha2 )**( 3 ) * beta2 * ( -1 + 7 * beta2 ) + ( 3 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 4 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 2 ) * beta2 * ( -3 + ( 7 * q2 + beta2 * ( -7 + 18 * q2 ) ) ) ) ) ) + ( 2 * ( 2 )**( 1/3 ) * ( alpha2 )**( 2 ) * beta2 * ( -3 * ( 3 + ( 3 * alpha2 + -7 * q2 ) ) * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 3 ) * ( 13 + ( 31 * ( alpha2 )**( 2 ) + 2 * alpha2 * ( -22 + 63 * q2 ) ) ) + ( alpha2 * ( beta2 )**( 2 ) * ( -14 + ( -14 * ( alpha2 )**( 3 ) + ( 26 * q2 + ( ( alpha2 )**( 2 ) * ( -7 + 8 * q2 ) + alpha2 * ( 35 + ( -289 * q2 + 378 * ( q2 )**( 2 ) ) ) ) ) ) ) + beta2 * ( 4 + ( ( alpha2 )**( 4 ) + ( ( alpha2 )**( 3 ) * ( 6 + -14 * q2 ) + ( -14 * q2 + ( 13 * ( q2 )**( 2 ) + ( ( alpha2 )**( 2 ) * ( -5 + ( 56 * q2 + -77 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * ( -3 + ( 70 * q2 + ( -223 * ( q2 )**( 2 ) + 189 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) ) ) ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( -1/3 ) + ( 2 )**( 2/3 ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( 1/3 ) ) ) )**( ( alpha2 )**( -1 ) )
    #C_ts = K2*np.real(c_ts)

    c_tjm = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + -1 * alpha2 * ( beta2 + ( -4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
    C_tjm = K2*c_tjm

    c_tjp = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + alpha2 * ( -1 * beta2 + ( 4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
    C_tjp = K2*c_tjp

    alpha2 = alpha3
    q2 = q3
    beta2 = beta3

    tc = f_imp(C_tc,C02,beta2,q2,alpha2,r2,K2)
    tjm = f_imp(C_tjm,C02,beta2,q2,alpha2,r2,K2)
    tjp = f_imp(C_tjp,C02,beta2,q2,alpha2,r2,K2)
    #ts = f_imp(C_ts,C02,beta2,q2,alpha2,r2,K2)

    global date_first
    if Type == 'Deaths':
      date_first_peak1 = date_first + timedelta(days=int(tc))
    if Type == 'Cases':
      date_first_peak1 = date_first + timedelta(days=int(tc))
      

    lang = html_widget_lang.value

    date_last = date_first + timedelta(days=len(kind1))

    if lang == 'Por':
      date_first1 = date_first.strftime('%d/%m/%y')
      date_first_peak1 = date_first_peak1.strftime('%d/%m/%y')
      date_last = date_last.strftime('%d/%m/%y')
    if lang == 'Eng':
      date_first1 = date_first.strftime('%m/%d/%y')
      date_first_peak1 = date_first_peak1.strftime('%m/%d/%y')
      date_last = date_last.strftime('%m/%d/%y')

    if Type=='Deaths':
      tipo = 'Mortes'
      tipo1 = 'morte'
      tipo2 = 'a primeira morte'
    if Type=='Cases':
      tipo = 'Casos'
      tipo1 = 'caso'
      tipo2 = 'o primeiro caso'

    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(x,data, 'ro',alpha=0.5, label = str(subs[lang][Type])+' '+str(name))
    ax.plot(f_imp(c_forecast,C02,beta2,q2,alpha2,r2,K2),c_forecast, '-k', label = methods_name1[lang][2])
    ax.plot(x[0],data[0], 'wo', alpha=0, label=first_death[lang][Type]+str(date_first1))
    if Forecast>0:
      ax.plot(x[0],data[0], 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+'+'+str(int(C_plot1*K2-C_plot*K2))+' '+subs[lang][Type])
      ax.plot(x[0],data[0], 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+str(int(C_plot1*K2))+' '+subs[lang][Type])
    ax.plot(x[0],data[0], 'ko', alpha=0, label=estagio[lang])
    global stage
    if stage == 'early intermediate':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][1], color='darkorange')
    if stage == 'late intermediate':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][2], color='yellow')
    if stage == 'early saturation':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      ax.plot(np.ones(100)*tjp,np.linspace(0,C_tjp,100), '-', alpha=1, color='green')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][3], color='green')
    #if stage == 'late saturation':
      #ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      #ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      #ax.plot(np.ones(100)*tjp,np.linspace(0,C_tjp,100), '-', alpha=1, color='green')
      #ax.plot(np.ones(100)*ts,np.linspace(0,C_ts,100), '-', alpha=1, color='royalblue')
      #ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][4], color='royalblue')
    ax.plot(x[0],data[0], 'o', alpha=0, label=copyright_msg)
    ax.set_xlabel(label_x[lang][Type])
    ax.set_ylabel(label_y[lang][Type])
    plt.xscale(dic[x_axis])
    plt.yscale(dic[y_axis])
    if x_axis=='Linear' and y_axis=='Linear':
      legend = ax.legend(loc='best', fontsize='small')
      line = legend.get_lines()
      text = legend.get_texts()
      text[len(text)-2].set_color(line[len(text)-2].get_color())
    if x_axis=='Logarithmic':
      ax.set_xlim(left=0.8)
    if y_axis=='Logarithmic':
      ax.set_ylim(bottom=0.8)
    plt.title(name, y=1.04)
    plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
    plt.tight_layout()
    plt.savefig('fig.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig.eps', format='eps')
    plt.show()

    if toggdetails1 == True:
      variables = []
      for key, value in params.valuesdict().items():
        variables.append(key)

      ti_names = []
      ti_values = []
      ti_vec = [tjm, tc, tjp]

      stages_dic = {'early intermediate':1, 'late intermediate':2, 'early saturation':3}

      for i in range(stages_dic[stage]):
        ti_names.append('t'+str(i+1))
        ti_values.append(ti_vec[i])
        
      for i in range(len(ti_names), 5):
        ti_names.append('')
        ti_values.append('')


      values = []
      stderr = []

      for i in range(5):
        values.append(params[variables[i]].value)
        stderr.append(params[variables[i]].stderr)

      if lang == 'Eng':
        d = {'Parameter':variables, 'Value':values, 'StdErr':stderr, 'ti':ti_names, 'ti_values':ti_values}
      else:
        d = {'Parâmetro':variables, 'Valor':values, 'Erro':stderr, 'ti':ti_names, 'valor_ti':ti_values}

      df_table = pd.DataFrame({ key:pd.Series(value) for key, value in d.items() })
      df_table.fillna('')
      pd.set_option('display.colheader_justify', 'center')
      print(df_table)
      fig, ax = plt.subplots(figsize=(8,4))
      fig.patch.set_visible(False)
      ax.axis('off')
      ax.axis('tight')
      ax.table(cellText=df_table.values, colLabels=df_table.columns, loc='center', cellLoc='center', fontsize=12)
      fig.tight_layout()
      plt.savefig('fig2.png', format='png', dpi=200)
      ax.set_rasterized(True)
      plt.savefig('fig2.eps', format='eps')
      plt.close(fig)
    
    if togg2==True:
      daily=[]
      daily.append(data[0])
      for i in range(0,len(data)-1):
        daily.append(data[i+1]-data[i])
      fig, ax = plt.subplots(figsize=(6,4))
      ax.plot(x[0:len(x)],daily,'ro-', markersize=4, alpha=0.5, label=label_y_daily[lang][Type]+' '+str(name))
      ax.plot(f_imp(c_forecast,C02,beta2,q2,alpha2,r2,K2),f_ric1(0,c_forecast,*args2), '-k', label=methods_name1[lang][2])
      ax.plot(tc,f_ric1(0,C_tc,*args2), 'o', color='yellow', markersize=8,alpha=1)
      ax.plot(tc,f_ric1(0,C_tc,*args2),'ok',alpha=0,label=first_peak[lang]+str(date_first_peak1))
      if x_axis=='Linear' and y_axis=='Linear':
        legend = ax.legend(loc='best', fontsize='small')
        pts = ax.dataLim.get_points()
        pts[0, 1] = 0 
        ax.dataLim.set_points(pts)
        ax.autoscale_view()
      if x_axis=='Logarithmic':
        ax.set_xlim(left=0.8)
      if y_axis=='Logarithmic':
        ax.set_ylim(bottom=0.8)
      ax.set_xlabel(label_x[lang][Type])
      ax.set_ylabel(label_y_daily[lang][Type])
      plt.xscale(dic[x_axis])
      plt.yscale(dic[y_axis])
      plt.title(name, y=1.04)
      plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
      plt.tight_layout()
      plt.savefig('fig1.png', format='png', dpi=200)
      ax.set_rasterized(True)
      plt.savefig('fig1.eps', format='eps')
      plt.show()

def automatic_fit_countries(Country, Type, name2, time_fit):

  name = Country
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
  df_recovered = pandas.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
  df_deaths = pandas.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
  df_confirmed = pandas.read_csv(url, error_bad_lines=False)
 
  df_deaths = df_deaths.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  df_confirmed = df_confirmed.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  deaths = df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
  recovered = df_recovered[(df_recovered['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
  confirmed = df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float')[0][np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_confirmed.columns.values)]
 
 
  if Type == 'Deaths':
    deaths = deaths
  if Type == 'Cases':
    deaths = confirmed

  deaths = deaths[0:time_fit]

  t0 = np.array(range(len(deaths)))

  daily1=[]
  daily1.append(deaths[0])
  for i in range(0,len(deaths)-1):
    daily1.append(deaths[i+1]-deaths[i])

  smoother = LowessSmoother(smooth_fraction=0.1, iterations=1)
  smoother.smooth(daily1)
  daily = smoother.smooth_data[0]

  peaks, peaks1 = wave_detection(daily)

  N = len(peaks1)+1

  peaks2 = peaks1

  def joy_division(N):
    if N==1:
      globals()['t'+str(1)] = t0
      globals()['deaths'+str(1)] = deaths
    for i in range(N):
      if i==N-1:
        globals()['t'+str(i+1)] = t0
        globals()['deaths'+str(i+1)] = deaths
      else:
        globals()['t'+str(i+1)] = t0[0:peaks2[i]]
        globals()['deaths'+str(i+1)] = deaths[0:peaks2[i]]
    return

  def create_params(N, params):
    params.add('r1', value=0.2, min=0, max=0.999)
    params.add('a1', value=0.2, min=0, max=1)
    params.add('q1', value=0.3, min=0, max=0.999)
    params.add('K1', value=1.1*globals()['deaths'+str(1)][-1], min=globals()['deaths'+str(1)][-1])
    params.add('p1', value=1, min=1, vary=False)
    return

  def update_params(N, params):
    params.add('p'+str(N-1), value=1, min=1)
    params.add('r'+str(N), value=0.2, min=0, max=0.999)
    params.add('a'+str(N), value=1, min=0, max=1, vary=False)
    params.add('q'+str(N), value=0.6, min=0, max=0.999)
    params.add('K'+str(N), value=1.1*globals()['deaths'+str(N)][-1], min=globals()['deaths'+str(N)][-1])
    params.add('p'+str(N), value=1, min=1, vary=False)
    params.add('rho_'+str(N-1), value=0.01, min=0.003, max=.15)
    params.add('t_'+str(N-1), value=t0[peaks2[N-2]], min=0, max=len(globals()['t'+str(N)]))
    return

  def ODE_generator(N):
    def ODE_template(y, t, params, N):
      C = y
      r_vec = []
      a_vec = []
      q_vec = []
      K_vec = []
      p_vec = []
      rho_vec = []
      t_vec = []
      for i in range(0, N):
        r_vec.append(params['r'+str(i+1)])
        a_vec.append(params['a'+str(i+1)])
        q_vec.append(params['q'+str(i+1)])
        K_vec.append(params['K'+str(i+1)])
        p_vec.append(params['p'+str(i+1)])
      for i in range(1, N):
        rho_vec.append(params['rho_'+str(i)])
        t_vec.append(params['t_'+str(i)])
      r = r_vec[0]
      K = K_vec[0]
      a = a_vec[0]
      p = p_vec[0]
      q = q_vec[0]
      if N>1:
        for i in range(1, N):
          r += 0.5*(r_vec[i]-r_vec[i-1])*(1+np.tanh(0.5*rho_vec[i-1]*(t-t_vec[i-1])))
          K += 0.5*(K_vec[i]-K_vec[i-1])*(1+np.tanh(0.5*rho_vec[i-1]*(t-t_vec[i-1])))
          a += 0.5*(a_vec[i]-a_vec[i-1])*(1+np.tanh(0.5*rho_vec[i-1]*(t-t_vec[i-1])))
          p += 0.5*(p_vec[i]-p_vec[i-1])*(1+np.tanh(0.5*rho_vec[i-1]*(t-t_vec[i-1])))
          q += 0.5*(q_vec[i]-q_vec[i-1])*(1+np.tanh(0.5*rho_vec[i-1]*(t-t_vec[i-1])))
      dCdt = r*(C**q)*(1-(C/K)**a)**p
      return dCdt
    return ODE_template

  def generate_ODEs(N):
    for i in range(N):
      globals()['ODE'+str(i+1)] = ODE_generator(i+1)
    return

  def objective_generator(N):
    def objective_template(params, time_vec, data_vec):
      ret = odeint(globals()['ODE'+str(N)], y0, time_vec, args=(params, N))
      C = ret.T
      resid = []
      pre_resid = []
      pre_resid.append(C)
      pre_resid = np.array(pre_resid)
      resid.append(pre_resid-data_vec)
      resid = np.array(resid)
      return resid      
    return objective_template

  def generate_objectives(N):
    for i in range(N):
      globals()['func'+str(i+1)] = objective_generator(i+1)
    return

  def generate_minimizers(N, params):
    globals()['minimizer'+str(N)] = Minimizer(globals()['func'+str(N)], params, fcn_args=(globals()['t'+str(N)],globals()['deaths'+str(N)]))
    return

  y0 = deaths[0]
  joy_division(N)
  fit_params = Parameters()
  create_params(1, fit_params)
  generate_ODEs(N)
  generate_objectives(N)

  try:
    for i in range(N):
      if i==0:
        generate_minimizers(i+1, fit_params)
        globals()['out'+str(2*i+1)] = globals()['minimizer'+str(i+1)].minimize(method= 'nelder')
        globals()['out'+str(2*i+2)] = globals()['minimizer'+str(i+1)].minimize(method='least_squares', params=globals()['out'+str(2*i+1)].params)
        fit_params = globals()['out'+str(2*i+2)].params
        if N>1:
          update_params(i+2, fit_params)
      else:
        generate_minimizers(i+1, fit_params)
        globals()['out'+str(2*i+1)] = globals()['minimizer'+str(i+1)].minimize(method='least_squares')
        globals()['out'+str(2*i+2)] = globals()['minimizer'+str(i+1)].minimize(method='least_squares', params=globals()['out'+str(2*i+1)].params)
        fit_params = globals()['out'+str(2*i+2)].params
        if i<N-1:
          update_params(i+2, fit_params)
  except ValueError:
    Country = 'Error'

  return Country, Type, name2, deaths, fit_params, N

def automatic_plot_countries(Country, Type, name2, kind1, params, N, Forecast, interv_time, interv_param, togg1, togg2, x_axis, y_axis, toggdetails1):

  name = Country
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
  df_recovered = pandas.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
  df_deaths = pandas.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
  df_confirmed = pandas.read_csv(url, error_bad_lines=False)
 
  df_deaths = df_deaths.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  df_confirmed = df_confirmed.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  deaths = df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
  recovered = df_recovered[(df_recovered['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
  confirmed = df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float')[0][np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_confirmed.columns.values)]

  t0 = np.array(range(len(kind1)))
  t = np.linspace(0, len(kind1)-1+Forecast, 2000)
  t_plot = np.linspace(0, len(kind1)-1, 2000)

  y0 = kind1[0]

  ret = odeint(globals()['ODE'+str(N)], y0, t, args=(params, N))
  C = ret.T[0]

  ret = odeint(globals()['ODE'+str(N)], y0, t_plot, args=(params, N))
  C1 = ret.T[0]
  
  daily_theo = globals()['ODE'+str(N)](C, t, params, N)

  #peaks_new, peaks_new1 = wave_detection(daily_theo)

  #N = len(peaks_new1)+1

  daily1=[]
  daily1.append(kind1[0])
  for i in range(0,len(kind1)-1):
    daily1.append(kind1[i+1]-kind1[i])

  tck = interpolate.splrep(t, daily_theo, s=0)
  ynew = interpolate.splev(t, tck, der=0)
  yder = interpolate.splev(t, tck, der=1)
  yder2 = interpolate.splev(t, tck, der=2)
  yder3 = interpolate.splev(t, tck, der=3)

  zero_crossings1 = np.where(np.diff(np.sign(yder)))[0]
  zero_crossings2 = np.where(np.diff(np.sign(yder2)))[0]
  zero_crossings3 = np.where(np.diff(np.sign(yder3)))[0]

  if Type == 'Deaths':
    for i in range(len(zero_crossings1[1::2])):
      locals()['date_wave'+str(i+2)] = df_deaths.columns[3+np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]+int(t[zero_crossings1[2*i+1]])]
    date_first = df_deaths.columns[3+np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]]
    date_last = df_deaths.columns[3+np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]+len(kind1)]
    for i in range(len(zero_crossings1[0::2])):
      locals()['date_peak'+str(i+1)] = df_deaths.columns[3+np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]+int(t[zero_crossings1[2*i]])]
  if Type == 'Cases':
    for i in range(len(zero_crossings1[1::2])):
      locals()['date_wave'+str(i+2)] = df_confirmed.columns[3+np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]+int(t[zero_crossings1[2*i+1]])]
    date_first = df_confirmed.columns[3+np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]]
    date_last = df_confirmed.columns[3+np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]+len(kind1)]
    for i in range(len(zero_crossings1[0::2])):
      locals()['date_peak'+str(i+1)] = df_confirmed.columns[3+np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]+int(t[zero_crossings1[2*i]])]    

  lang = html_widget_lang.value

  if lang == 'Por':
    for i in range(len(zero_crossings1[1::2])):
      locals()['date_wave'+str(i+2)] = datetime.strptime(locals()['date_wave'+str(i+2)], '%m/%d/%y').strftime('%d/%m/%y')
    date_first = datetime.strptime(date_first, '%m/%d/%y').strftime('%d/%m/%y')
    date_last = datetime.strptime(date_last, '%m/%d/%y').strftime('%d/%m/%y')
    for i in range(len(zero_crossings1[0::2])):
      locals()['date_peak'+str(i+1)] = datetime.strptime(locals()['date_peak'+str(i+1)], '%m/%d/%y').strftime('%d/%m/%y')

  t_comparison = []
  t_cross = []
  tjm_vec = []
  tc_vec = []
  tjp_vec = []
  ts_vec = []

  # Find tis from zeros of the derivatives
  # tc
  for i in range(len(zero_crossings1[0::2])):
    tc_vec.append([t[zero_crossings1[2*i]], zero_crossings1[2*i]])
  # t crossing 
  for i in range(len(zero_crossings1[1::2])):
    t_cross.append([t[zero_crossings1[2*i+1]], zero_crossings1[2*i+1]])
  # now we compute the tjm, tjp and ts based on their loaction relative to 
  # tc and t cross.
  # tjm
  for j in range(len(tc_vec)):
    for i in range(len(zero_crossings2[0::2])):
      tjm1 = []
      if j==0:
        if t[zero_crossings2[2*i]]<tc_vec[j][0]:
          tjm1.append([t[zero_crossings2[2*i]], zero_crossings2[2*i]])
      else:
        if (t[zero_crossings2[2*i]]<tc_vec[j][0]) and (t[zero_crossings2[2*i]]>t_cross[j-1][0]):
          tjm1.append([t[zero_crossings2[2*i]], zero_crossings2[2*i]])
      if len(tjm1)>0:
        tjm_vec.append(tjm1[-1])
  # tjp
  for j in range(len(tc_vec)):
    for i in range(len(zero_crossings2[1::2])):
      tjp1 = []
      if j==len(tc_vec)-1:
        if t[zero_crossings2[2*i+1]]>tc_vec[j][0]:
          tjp1.append([t[zero_crossings2[2*i+1]], zero_crossings2[2*i+1]])
      else:
        if (t[zero_crossings2[2*i+1]]>tc_vec[j][0]) and (t[zero_crossings2[2*i+1]]<t_cross[j][0]):
          tjp1.append([t[zero_crossings2[2*i+1]], zero_crossings2[2*i+1]])
      if len(tjp1)>0:
        tjp_vec.append(tjp1[-1])
  # ts
  for j in range(len(tc_vec)):
    for i in range(len(zero_crossings3[2::4])):
      ts1 = []
      if j==len(tc_vec)-1:
        if t[zero_crossings3[4*i+2]]>tc_vec[j][0]:
          ts1.append([t[zero_crossings3[4*i+2]], zero_crossings3[4*i+2]])
      else:
        if (t[zero_crossings3[4*i+2]]>tc_vec[j][0]) and (t[zero_crossings3[4*i+2]]<t_cross[j][0]):
          ts1.append([t[zero_crossings3[4*i+2]], zero_crossings3[4*i+2]])
      if len(ts1)>0:
        ts_vec.append(ts1[-1])

  N = len(t_cross)+1
  
  if N == 1:
    methods_name2 = {'Eng':'BLM fit ('+str(N)+' wave)', 'Por':'Ajuste MLB ('+str(N)+' onda)'}
  else:
    methods_name2 = {'Eng':'BLM fit ('+str(N)+' waves)', 'Por':'Ajuste MLB ('+str(N)+' ondas)'}

  wave_disc_por = [' (1a onda)', ' (2a onda)', ' (3a onda)']
  wave_disc_eng = [' (1st wave)', ' (2nd wave)', ' (3rd wave)']
  if N>3:
    for i in range(3,N):
      wave_disc_eng.append(' ('+str(i+1)+'th wave)')
      wave_disc_por.append(' ('+str(i+1)+'a onda)')
  wave_disc = {'Eng':wave_disc_eng, 'Por':wave_disc_por}

  fig, ax = plt.subplots(figsize=(8,4))
  ax.plot(t0, kind1,'ro', alpha=0.5, label=subs[lang][Type]+' '+name)
  ax.plot(t, C, 'k', alpha=1, lw=2, label=methods_name2[lang])
  ax.plot(t0, kind1, 'ro', alpha=0, label = first_death[lang][Type]+date_first)
  # plot tc
  for i in range(len(tc_vec)):
    if i < len(tc_vec)-1:
      ax.plot(np.ones(100)*t[tc_vec[i][1]], np.linspace(0,C[tc_vec[i][1]],100), '--', color='yellow')
    elif len(t_cross)>0:
      if tc_vec[-1][0]>t_cross[-1][0]:
        ax.plot(np.ones(100)*t[tc_vec[i][1]], np.linspace(0,C[tc_vec[i][1]],100), '-', color='yellow')
    #elif tc_vec[-1][0]>t_cross[-1][0]:
      #ax.plot(np.ones(100)*t[tc_vec[i][1]], np.linspace(0,C[tc_vec[i][1]],100), '-', color='yellow')
    else:
      ax.plot(np.ones(100)*t[tc_vec[i][1]], np.linspace(0,C[tc_vec[i][1]],100), '--', color='yellow')
    tc = t[tc_vec[i][1]]
  t_comparison.append(('tc', tc))
  # plot division of new wave
  for i in range(len(t_cross)):
    ax.plot(np.ones(100)*t[t_cross[i][1]], np.linspace(0,C[t_cross[i][1]],100), '-', color='black')
    ax.plot(t[t_cross[i][1]], C[t_cross[i][1]], 'ko')
    wave_date1 = {'Eng': 'Start of wave '+str(i+2)+': ', 'Por': 'Início da onda '+str(i+2)+': '}
    ax.plot(t[t_cross[i][1]], C[t_cross[i][1]], 'ko', alpha=0, label=wave_date1[lang]+locals()['date_wave'+str(i+2)])
  # plot tjm
  if len(tjm_vec)>0:
    for i in range(len(tjm_vec)):
      if i < len(tjm_vec)-1:
        ax.plot(np.ones(100)*t[tjm_vec[i][1]], np.linspace(0,C[tjm_vec[i][1]],100), '--', color='darkorange')
      elif len(t_cross)>0:
        if tjm_vec[-1][0]>t_cross[-1][0]:
          ax.plot(np.ones(100)*t[tjm_vec[i][1]], np.linspace(0,C[tjm_vec[i][1]],100), '-', color='darkorange')
      #elif tjm_vec[-1][0]>t_cross[-1][0]:
        #ax.plot(np.ones(100)*t[tjm_vec[i][1]], np.linspace(0,C[tjm_vec[i][1]],100), '-', color='darkorange')
      else:
        ax.plot(np.ones(100)*t[tjm_vec[i][1]], np.linspace(0,C[tjm_vec[i][1]],100), '--', color='darkorange')
      tjm = t[tjm_vec[i][1]]
    t_comparison.append(('tjm', tjm))
  # plot tjp
  if len(tjp_vec)>0:
    for i in range(len(tjp_vec)):
      if i < len(tjp_vec)-1:
        ax.plot(np.ones(100)*t[tjp_vec[i][1]], np.linspace(0,C[tjp_vec[i][1]],100), '--', color='green')
      elif len(t_cross)>0:
        if tjp_vec[-1][0]>t_cross[-1][0]:
          ax.plot(np.ones(100)*t[tjp_vec[i][1]], np.linspace(0,C[tjp_vec[i][1]],100), '-', color='green')
      #elif tjp_vec[-1][0]>t_cross[-1][0]:
        #ax.plot(np.ones(100)*t[tjp_vec[i][1]], np.linspace(0,C[tjp_vec[i][1]],100), '-', color='green')
      else:
        ax.plot(np.ones(100)*t[tjp_vec[i][1]], np.linspace(0,C[tjp_vec[i][1]],100), '--', color='green')
      tjp = t[tjp_vec[i][1]]
    t_comparison.append(('tjp', tjp))
  # plot ts
  #if len(ts_vec)>0:
    #for i in range(len(ts_vec)):
      #if i < len(ts_vec)-1:
        #ax.plot(np.ones(100)*t[ts_vec[i][1]], np.linspace(0,C[ts_vec[i][1]],100), '--', color='blue')
      #elif ts_vec[-1][0]>t_cross[-1][0]:
        #ax.plot(np.ones(100)*t[ts_vec[i][1]], np.linspace(0,C[ts_vec[i][1]],100), '-', color='blue')
      #else:
        #ax.plot(np.ones(100)*t[ts_vec[i][1]], np.linspace(0,C[ts_vec[i][1]],100), '--', color='blue')
      #ts = t[ts_vec[i][1]]
    #t_comparison.append(('ts', ts))
  t_final = t0[-1]
  t_comparison.append(('t_final', t_final))
  t_comparison.sort(key=lambda x: x[1])
  if len(tjm_vec)>0:
    epi_phase = [x[0] for x in t_comparison].index("t_final") - [x[0] for x in t_comparison].index("tjm")
  else:
    epi_phase = [x[0] for x in t_comparison].index("t_final") - [x[0] for x in t_comparison].index("tc") + 1
  if Forecast>0:
    ax.plot(t0[0], kind1[0], 'ko', markersize=8, alpha=0, label =forecast_message[lang][str(Forecast)]+'+'+str(int(C[-1]-C1[-1]))+' '+subs[lang][Type])
    ax.plot(t0[0], kind1[0], 'ko', markersize=8, alpha=0, label =forecast_message[lang][str(Forecast)]+str(int(C[-1]))+' '+subs[lang][Type])
  ax.plot(t0[0],kind1[0], 'o', alpha=0, label=estagio[lang])
  ax.plot(t0[0],kind1[0], 'o', alpha=0, label=fase[lang][epi_phase]+wave_disc[lang][N-1], color=colordict[epi_phase])
  ax.plot(t0[0],kind1[0], 'o', alpha=0, label=copyright_msg)
  ax.set_xlabel(label_x[lang][Type])
  ax.set_ylabel(label_y[lang][Type])
  if x_axis=='Linear' and y_axis=='Linear':
    legend = ax.legend(loc='best', fontsize='small')
    line = legend.get_lines()
    text = legend.get_texts()
    text[len(text)-2].set_color(line[len(text)-2].get_color())
  if x_axis=='Logarithmic':
    ax.set_xlim(left=0.8)
  if y_axis=='Logarithmic':
    ax.set_ylim(bottom=0.8)
  plt.xscale(dic[x_axis])
  plt.yscale(dic[y_axis])
  plt.title(name, y=1.04)
  plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
  plt.tight_layout()
  plt.savefig('fig.png', format='png', dpi=200)
  ax.set_rasterized(True)
  plt.savefig('fig.eps', format='eps')
  plt.show()

  if toggdetails1 == True:
    variables = []
    for key, value in params.valuesdict().items():
      variables.append(key)
    variables = variables[0:7*N-2]

    ti_names = []
    ti_values = []

    for i in range(N):
      ti_names.append('t1 '+wave_disc[lang][i])
      ti_names.append('t2 '+wave_disc[lang][i])
      ti_names.append('t3 '+wave_disc[lang][i])
      #ti_names.append('t4 '+wave_disc[lang][i])
      if i > len(tjm_vec)-1:
        ti_values.append('-')
      else:
        ti_values.append(int(tjm_vec[i][0]))
      if i > len(tc_vec)-1:
        ti_values.append('-')
      else:
        ti_values.append(int(tc_vec[i][0]))
      if i > len(tjp_vec)-1:
        ti_values.append('-')
      else:
        ti_values.append(int(tjp_vec[i][0]))
      #if i > len(ts_vec)-1:
        #ti_values.append('-')
      #else:
        #ti_values.append(int(ts_vec[i][0]))

    for i in range(3*N, 7*N-2):
      ti_names.append('')
      ti_values.append('')


    values = []
    stderr = []

    for i in range(7*N-2):
      values.append(params[variables[i]].value)
      stderr.append(params[variables[i]].stderr)

    if lang == 'Eng':
      d = {'Parameter':variables, 'Value':values, 'StdErr':stderr, 'ti':ti_names, 'ti_values':ti_values}
    else:
      d = {'Parâmetro':variables, 'Valor':values, 'Erro':stderr, 'ti':ti_names, 'valor_ti':ti_values}

    df_table = pd.DataFrame({ key:pd.Series(value) for key, value in d.items() })
    df_table.fillna('')
    pd.set_option('display.colheader_justify', 'center')
    print(df_table)
    fig, ax = plt.subplots(figsize=(8,4))
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    ax.table(cellText=df_table.values, colLabels=df_table.columns, loc='center', cellLoc='center', fontsize=12)
    fig.tight_layout()
    plt.savefig('fig2.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig2.eps', format='eps')
    plt.close(fig)

  if togg2 == True:
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(t0, daily1,'ro', alpha=0.5, label=label_y_daily[lang][Type]+' '+name)
    ax.plot(t, daily_theo, 'k', alpha=1, lw=2, label=methods_name2[lang])
    # plot tc
    for i in range(len(zero_crossings1[0::2])):
      peak_date1 = {'Eng': 'Peak '+str(i+1)+': ', 'Por': 'Pico '+str(i+1)+': '}
      ax.plot(t[zero_crossings1[2*i]], daily_theo[zero_crossings1[2*i]], 'o', color='yellow', alpha=1, markersize=8)
      ax.plot(t[zero_crossings1[2*i]], daily_theo[zero_crossings1[2*i]], 'kv', alpha=0, label=peak_date1[lang]+locals()['date_peak'+str(i+1)])
      tc = t[zero_crossings1[2*i]]
    # plot division of new wave
    for i in range(len(zero_crossings1[1::2])):
      ax.plot(t[zero_crossings1[2*i+1]], daily_theo[zero_crossings1[2*i+1]], 'ko', markersize=8, alpha=1)
    ax.plot(t0[0],kind1[0], 'o', alpha=0, label=copyright_msg)
    ax.set_xlabel(label_x[lang][Type])
    ax.set_ylabel(label_y_daily[lang][Type])
    if x_axis=='Linear' and y_axis=='Linear':
      legend = ax.legend(loc='best', fontsize='small')
      line = legend.get_lines()
      text = legend.get_texts()
      text[len(text)-2].set_color(line[len(text)-2].get_color())
      pts = ax.dataLim.get_points()
      pts[0, 1] = 0 
      ax.dataLim.set_points(pts)
      ax.autoscale_view()
    if x_axis=='Logarithmic':
      ax.set_xlim(left=0.8)
    if y_axis=='Logarithmic':
      ax.set_ylim(bottom=0.8)
    plt.xscale(dic[x_axis])
    plt.yscale(dic[y_axis])
    plt.title(name, y=1.04)
    plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
    plt.tight_layout()
    plt.savefig('fig1.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig1.eps', format='eps')
    plt.show()

togg_widget = widgets.Checkbox(
    value=False,
    description='Marque para mostrar/atualizar o ajuste',
    disabled=False,
    indent=False
)
togg_widget.layout.visibility = 'hidden'

togg1_widget = widgets.Checkbox(
    value=False,
    description='Check to display/hide interventions',
    disabled=False,
    indent=False
)

togg4_widget = widgets.Checkbox(
    value=False,
    description='Marque para mostrar/ocultar a curva diária',
    disabled=False,
    indent=False
)

togg_details1 = widgets.Checkbox(
    value=False,
    description='Marque para mostrar os parâmetros do ajuste',
    disabled=False,
    indent=False
)

togg_fit1 = widgets.Checkbox(
    value=False,
    description='Marque para fazer o ajuste',
    disabled=False,
    indent=False
)
togg_fit1.layout.visibility = 'hidden'

link = widgets.HTML(
    value=" ",
    placeholder=' ',
    description=' ',
)

link2 = widgets.HTML(
    value=" ",
    placeholder=' ',
    description=' ',
)

link_pdf = widgets.HTML(
    value=" ",
    placeholder=' ',
    description=' ',
)

button_fit1 = widgets.Button(description="Fazer ajuste")
def on_button_fit1_clicked(b):
  togg_fit1.value = True
button_fit1.on_click(on_button_fit1_clicked)

html_display_widget_loading_fit = widgets.HTML(value=' ') 

html_display_error = widgets.HTML(value=' ') 

time_widget = widgets.IntSlider(
    value=80,
    min=0,
    max=100,
    step=1,
    description='Tempo fit',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.0f'
    
)

def Select_method(Type, Country, toggfit1, time_fit):
  a = False
  global Meth
  lang = html_widget_lang.value
  html_display_error.value = ' '
  name = Country
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
  df_recovered = pd.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
  df_deaths = pd.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
  df_confirmed = pd.read_csv(url, error_bad_lines=False)
  url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv'
  df_population = pd.read_csv(url, error_bad_lines=False)
 
  df_deaths = df_deaths.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  df_confirmed = df_confirmed.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  df_recovered = df_recovered.groupby(['Country/Region']).aggregate(np.sum).reset_index()
  deaths = df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
  recovered = df_recovered[(df_recovered['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float')[0][np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_deaths.columns.values)]
  confirmed = df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float')[0][np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df_confirmed.columns.values)]
 
      
  if Type == 'Deaths':
    kind1 = deaths
    name2 = 'death'
  if Type == 'Cases':
    kind1 = confirmed
    name2 = 'case'
 
  x = np.arange(0,len(kind1),1)

  if time_widget.max == max(x):
    a = True
  elif time_widget.max != max(x):
    time_widget.max = max(x)
    time_widget.value = max(x)-1
    time_widget.value = max(x)
    time_fit = time_widget.value
    #a = False
    
  #if time_widget.max != max(x):
    #time_widget.max = max(x)
    #time_widget.value = max(x)
    #time_fit = time_widget.value
  #else:
    #print('target')
    #a = True

  if Type == 'Deaths':
    date_last = df_deaths.columns[3+np.where(df_deaths[(df_deaths['Country/Region'] == name)][df_deaths.columns.values[3:len(df_deaths.columns.values)]].to_numpy(dtype='float') > 0)[1][0] + time_fit]
  if Type == 'Cases':
    date_last = df_confirmed.columns[3+np.where(df_confirmed[(df_confirmed['Country/Region'] == name)][df_confirmed.columns.values[3:len(df_confirmed.columns.values)]].to_numpy(dtype='float') > 0)[1][0]+ time_fit]

  if lang == 'Por':
    date_last = datetime.strptime(date_last, '%m/%d/%y').strftime('%d/%m/%y')
  
  if time_widget.max == max(x) and a == True:
    kind1 = kind1[0:time_fit]
    x = np.arange(0,len(kind1),1)
    plt.figure(figsize=(6,4))
    plt.plot(x,kind1,'ro', alpha=0.5, label=subs[lang][Type]+' '+str(name))
    plt.plot(x[0],kind1[0], 'ro', alpha=0, label=date_last_message[lang]+date_last)
    plt.xlabel(label_x[lang][Type])
    plt.ylabel(label_y[lang][Type])
    plt.legend()
    plt.show()
  a = False

  togg4_widget.value = False
  togg_details1.value = False

  daily1=[]
  daily1.append(kind1[0])
  for i in range(0,len(kind1)-1):
    daily1.append(kind1[i+1]-kind1[i])

  smoother = LowessSmoother(smooth_fraction=0.1, iterations=1)
  smoother.smooth(daily1)
  daily = smoother.smooth_data[0]

  peaks, peaks1 = wave_detection(daily)

  N = len(peaks1)+1

  if N>1:
    Method = 'BLM-2W'
  else:
    Country, Type, name2, t0, kind1, K, r, alpha, C0 = RMPlot(Country,Type,name2,time_fit)
    tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
    tjp = tc - np.log(1.5 + 0.5*alpha - 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
    tjm = tc - np.log(1.5 + 0.5*alpha + 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
    global stage
    if max(x)>tjm:
      Method = 'BLM'
    if max(x)<tjm:
      Method = 'q-exponential'
      Meth = 'q-exponential'
      stage = 'q-exponential'

  if toggfit1 == True:
    html_display_widget_loading_fit.value=html_loading_message[lang]
    if Method == 'q-exponential':
      togg_widget.value = False
      togg1_widget.value = False
      togg4_widget.value = False
      togg_details1.value = False
      link.value = ' '
      link2.value = ' '
      link_pdf.value = ' '
      global vec
      vec = qexpPlot(Country,Type,name2,time_fit)
      togg_fit1.value = False
      html_display_widget_loading_fit.value=' '
      togg_widget.value = True
    if Method == 'BLM':
      togg_widget.value = False
      togg1_widget.value = False
      togg4_widget.value = False
      togg_details1.value = False
      link.value = ' '
      link2.value = ' '
      link_pdf.value = ' '
      global vec6
      vec6 = BLM_fit(Country,Type,name2,time_fit)
      Meth = vec6[0]
      togg_fit1.value = False
      html_display_widget_loading_fit.value=' '
      togg_widget.value = True
    if Method == 'BLM-2W':
      togg_widget.value = False
      togg1_widget.value = False
      togg4_widget.value = False
      togg_details1.value = False
      link.value = ' '
      link2.value = ' '
      link_pdf.value = ' '
      global vec_2wave_country
      vec_2wave_country = automatic_fit_countries(Country,Type,name2,time_fit)
      Meth = 'BLM-2W'
      togg_fit1.value = False
      html_display_widget_loading_fit.value=' '
      togg_widget.value = True

country1_widget = Dropdown(description='País', options=['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Angola', 'Antigua and Barbuda', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burma', 'Burundi', 'Cabo Verde', 'Cambodia', 'Cameroon', 'Canada', 'Central African Republic', 'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia', 'Denmark', 'Diamond Princess', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Fiji', 'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Grenada', 'Guatemala', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Holy See', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kazakhstan', 'Kenya', 'Korea, South', 'Kosovo', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'MS Zaandam', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Mauritania', 'Mauritius', 'Mexico', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Morocco', 'Mozambique', 'Namibia', 'Nepal', 'Netherlands', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'North Macedonia', 'Norway', 'Oman', 'Pakistan', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Poland', 'Portugal', 'Qatar', 'Romania', 'Russia', 'Rwanda', 'Saint Kitts and Nevis', 'Saint Lucia', 'Saint Vincent and the Grenadines', 'San Marino', 'Sao Tome and Principe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Slovakia', 'Slovenia', 'Somalia', 'South Africa', 'South Sudan', 'Spain', 'Sri Lanka', 'Sudan', 'Suriname', 'Sweden', 'Switzerland', 'Syria', 'Taiwan*', 'Tajikistan', 'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Trinidad and Tobago', 'Tunisia', 'Turkey', 'US', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'Uruguay', 'Uzbekistan', 'Venezuela', 'Vietnam', 'West Bank and Gaza', 'Western Sahara', 'Yemen', 'Zambia', 'Zimbabwe'])
d1_widget = Dropdown(description='Dados', options = [('Óbitos','Deaths'),('Casos','Cases')])

interact(Select_method, Type=d1_widget, Country=country1_widget, toggfit1 = togg_fit1, time_fit = time_widget);
display(button_fit1)
display(html_display_widget_loading_fit)
display(html_display_error)

t_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=28,
    step=7,
    description='Tempo máx.',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f'
    
)

def PlotMethod(togg, togg2, toggdetails1, Forecast, x_axis, y_axis):
  lang = html_widget_lang.value
  if togg2==False:
    link2.value = ' '
  togg1 = False
  interv_param = 0
  interv_time=10
  if togg == True:
    if Meth == 'q-exponential':
      Country,Type,name2, params,t0, kind1, q, r = vec
      qexpPlotPlot(Country,Type,name2, params,t0, kind1, q, r, Forecast, togg2, x_axis, y_axis, toggdetails1)
    if Meth == 'RM':
      Method1, Country, Type, name2, params, t0, kind1, alpha, q2, r, beta2, K, C0 = vec6
      RMPlotPlot(Country, Type, name2, params, t0, kind1, K, r, alpha, C0, Forecast, interv_time, interv_param, togg1, togg2, x_axis, y_axis, toggdetails1)
    if Meth == 'GRM':
      Method1, Country, Type, name2, params, t0, kind1, alpha2, q2, r2, beta2, K2, C02 = vec6
      GRMPlotPlot(Country, Type, name2, params, t0, kind1, alpha2, q2, r2, beta2, K2, C02, Forecast, interv_time, interv_param, togg1, togg2, x_axis, y_axis, toggdetails1)
    if Meth == 'BLM':
      Method1, Country, Type, name2, params, t0, kind1, alpha2, q2, r2, beta2, K2, C02 = vec6
      BLM_plot(Country, Type, name2, params, t0, kind1, alpha2, q2, r2, beta2, K2, C02, Forecast, interv_time, interv_param, togg1, togg2, x_axis, y_axis, toggdetails1)
    if Meth == 'BLM-2W':
      Country, Type, name2, kind1, params, N = vec_2wave_country
      if Country != 'Error':
        automatic_plot_countries(Country, Type, name2, kind1, params, N, Forecast, interv_time, interv_param, togg1, togg2, x_axis, y_axis, toggdetails1)
      else:
        html_display_error.value = error_message[lang]

interv_widget = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.01,
    description='Intensity',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.0f'
    
)

x_axis_widget = Dropdown(options = [('Linear','Linear'),('Logarítmico','Logarithmic')], description='Eixo-x')
y_axis_widget = Dropdown(options = [('Linear','Linear'),('Logarítmico','Logarithmic')], description='Eixo-y')

interact(PlotMethod, togg = togg_widget, togg2 = togg4_widget, toggdetails1 = togg_details1, Forecast=t_widget, x_axis=x_axis_widget, y_axis=y_axis_widget);

format_widget = Dropdown(options = ['png','eps'], description='Formato')

def format_selection(fmt):
  global FMT
  FMT = fmt

button = widgets.Button(description="Gerar figura")
button_pdf = widgets.Button(description="Generate Report")

items = [button, format_widget]
display(widgets.HBox(items))

items_link = [link, link2, link_pdf]
display(widgets.HBox(items_link))

def create_download_link(file, title):
    with open(file, "rb") as image_file:
      b64 = base64.b64encode(image_file.read()) 
    payload = b64.decode()
    if FMT == 'eps':
      filename = 'plot.eps'
      html = '<a download="{filename}" href="data:image/eps;base64,{payload}" target="">{title}</a>'
    if FMT == 'png':
      filename = 'plot.png'
      html = '<a download="{filename}" href="data:image/png;base64,{payload}" target="">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

def create_pdf_download_link(file, title = "Download Report"):
    with open(file, "rb") as image_file:
      b64 = base64.b64encode(image_file.read()) 
    payload = b64.decode()
    filename = 'report.pdf'
    html = '<a download="{filename}" href="data:application/pdf;base64,{payload}" target="">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

lang = html_widget_lang.value
def on_button_clicked(b):
  format_selection(format_widget.value)
  lang = html_widget_lang.value
  if lang=='Eng':
    link.value = create_download_link('fig.'+str(FMT),"Download Accumulated Curve")
  if lang=='Por':
    link.value = create_download_link('fig.'+str(FMT),"Baixar Curva Acumulada")
  with suppress(FileNotFoundError):
    if lang=='Eng':
      link2.value = create_download_link('fig1.'+str(FMT),"Download Daily Curve")
    if lang=='Por':
      link2.value = create_download_link('fig1.'+str(FMT),"Baixar Curva Diária")
  with suppress(FileNotFoundError):
    if lang=='Eng':
      link_pdf.value = create_download_link('fig2.'+str(FMT),"Download Fit Parameters")
    if lang=='Por':
      link_pdf.value = create_download_link('fig2.'+str(FMT),"Baixar Parâmetros do Ajuste")
def on_pdf_button_clicked(b):
  link_pdf.value = create_pdf_download_link('report.pdf')
button.on_click(on_button_clicked)
button_pdf.on_click(on_pdf_button_clicked)



interactive(children=(Dropdown(description='Dados', options=(('Óbitos', 'Deaths'), ('Casos', 'Cases')), value=…

Button(description='Fazer ajuste', style=ButtonStyle())

HTML(value=' ')

HTML(value=' ')

interactive(children=(Checkbox(value=False, description='Marque para mostrar/atualizar o ajuste', indent=False…

In [21]:
#nbi:hide_in
html_display_widget_mid = widgets.HTML(
    value = '<hr><p>Estados e cidades do Brasil e dos EUA.</p><hr>')
display(html_display_widget_mid)

HTML(value='<hr><p>Estados e cidades do Brasil e dos EUA.</p><hr>')

In [22]:
#nbi:hide_in
from ipywidgets import interact, Dropdown
from contextlib import suppress

global R2_general

togg2_widget = widgets.Checkbox(
    value=False,
    description='Marque para mostrar/atualizar o ajuste',
    disabled=False,
    indent=False
)
togg2_widget.layout.visibility = 'hidden'

togg3_widget = widgets.Checkbox(
    value=False,
    description='Check to display/hide interventions',
    disabled=False,
    indent=False
)

togg5_widget = widgets.Checkbox(
    value=False,
    description='Marque para mostrar/ocultar a curva diária',
    disabled=False,
    indent=False
)

togg_details2 = widgets.Checkbox(
    value=False,
    description='Marque para mostrar os parâmetros do ajuste',
    disabled=False,
    indent=False
)

togg_fit2 = widgets.Checkbox(
    value=False,
    description='Marque para fazer o ajuste',
    disabled=False,
    indent=False
)
togg_fit2.layout.visibility = 'hidden'

button_fit2 = widgets.Button(description="Fazer ajuste")
def on_button_fit2_clicked(b):
  togg_fit2.value = True
button_fit2.on_click(on_button_fit2_clicked)

html_display_widget_loading_fit2 = widgets.HTML(value=' ') 

html_display_error2 = widgets.HTML(value=' ') 

def RMPlot1(name,Type,name1):

  deaths = data

  deaths = deaths[0:len(deaths)-1-index]

  t_forecast1 = 1 + 0/100
  t = np.linspace(0, int(t_forecast1*len(deaths)), 2000)
  t0 = np.array(range(len(deaths)))

  def expfit(x,C0,r):
      return C0*np.exp(r*x)

  t_exp = np.array(range(len(deaths[0:20])))
  ydata = deaths[0:20]
  popt, pcov = curve_fit(expfit, t_exp, ydata)
  C0 = popt[0]

  def fcn2min(params, t, data):
      """Generalized Richard's Model"""
      K = params['K']
      alpha = params['alpha']
      r = params['r']
      tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
      model = K/(1+alpha*np.exp(-alpha*r*(t-tc)))**(1/alpha)
      return model - data

  params = Parameters()
  params.add('K', value=np.amax(deaths), min=0)
  params.add('alpha', value=.1, min=0, max=1)
  params.add('r', value=0.2, min=0, max=1)

  minner = Minimizer(fcn2min, params, fcn_args=(t0,deaths))

  out1 = minner.minimize(method= 'least_squares')

  out2 = minner.minimize(method='leastsq', params=out1.params)

  params = out2.params
  K = params['K']
  alpha = params['alpha']
  r = params['r']

  K = np.float(params['K'].value)
  alpha = np.float(params['alpha'].value)
  r = np.float(params['r'].value)

  tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
  tc = np.float(tc)
  tjp = tc - np.log(1.5 + 0.5*alpha - 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
  tjm = tc - np.log(1.5 + 0.5*alpha + 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
  #ts = np.real(( alpha )**( -1 ) * ( r )**( -1 ) * np.log( ( 1/3 * ( 7 + 4 * alpha ) * ( np.e )**( alpha * r * tc ) + ( -1/3 * ( 2 )**( 1/3 ) * ( -31 * ( np.e )**( 2 * alpha * r * tc ) + ( -44 * alpha * ( np.e )**( 2 * alpha * r * tc ) + -13 * ( alpha )**( 2 ) * ( np.e )**( 2 * alpha * r * tc ) ) ) * ( ( 335 * ( np.e )**( 3 * alpha * r * tc ) + ( 708 * alpha * ( np.e )**( 3 * alpha * r * tc ) + ( 465 * ( alpha )**( 2 ) * ( np.e )**( 3 * alpha * r * tc ) + ( 92 * ( alpha )**( 3 ) * ( np.e )**( 3 * alpha * r * tc ) + 3 * ( 3 )**( 1/2 ) * ( ( -257 * ( np.e )**( 6 * alpha * r * tc ) + ( -1224 * alpha * ( np.e )**( 6 * alpha * r * tc ) + ( -2122 * ( alpha )**( 2 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -1712 * ( alpha )**( 3 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -681 * ( alpha )**( 4 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -136 * ( alpha )**( 5 ) * ( np.e )**( 6 * alpha * r * tc ) + -12 * ( alpha )**( 6 ) * ( np.e )**( 6 * alpha * r * tc ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) )**( -1/3 ) + 1/3 * ( 2 )**( -1/3 ) * ( ( 335 * ( np.e )**( 3 * alpha * r * tc ) + ( 708 * alpha * ( np.e )**( 3 * alpha * r * tc ) + ( 465 * ( alpha )**( 2 ) * ( np.e )**( 3 * alpha * r * tc ) + ( 92 * ( alpha )**( 3 ) * ( np.e )**( 3 * alpha * r * tc ) + 3 * ( 3 )**( 1/2 ) * ( ( -257 * ( np.e )**( 6 * alpha * r * tc ) + ( -1224 * alpha * ( np.e )**( 6 * alpha * r * tc ) + ( -2122 * ( alpha )**( 2 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -1712 * ( alpha )**( 3 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -681 * ( alpha )**( 4 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -136 * ( alpha )**( 5 ) * ( np.e )**( 6 * alpha * r * tc ) + -12 * ( alpha )**( 6 ) * ( np.e )**( 6 * alpha * r * tc ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) )**( 1/3 ) ) ) ))
  
  K = params['K']
  alpha = params['alpha']
  r = params['r']

  global stage
  if t[-1]>tjm and t[-1]<=tc:
    stage = 'early intermediate'
  if t[-1]>tc and t[-1]<=tjp:
    stage = 'late intermediate'
  if t[-1]>tjp:
    stage = 'early saturation'
  #if t[-1]>ts:
    #stage = 'late saturation'

  def func(t,r,alpha,K):
      tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
      return K/(1+alpha*np.exp(-alpha*r*(t-tc)))**(1/alpha)

  R2 = 1 - out2.residual.var() / np.var(deaths)
  global R2_general
  R2_general = R2

  return name, Type, name1, t0, deaths, K, r, alpha, C0

def qexpPlot1(name,Type,name1):

  deaths = data
  deaths = deaths[0:len(deaths)-1-index]
  t_forecast1 = 1 + 0/100
  t = np.linspace(0, int(t_forecast1*len(deaths)), 2000)
  t0 = np.array(range(len(deaths)))

  def fcn2min(params, t, data):
      """Generalized Richard's Model"""
      r = params['r']
      q = params['q']
      A = data[0]**(1-q)
      model = (A+(1-q)*r*t)**(1/(1-q))
      return model - data

  params = Parameters()
  params.add('r', value=0.1, min=0)
  params.add('q', value=0.2, min=0, max=1)

  minner = Minimizer(fcn2min, params, fcn_args=(t0,deaths))

  out1 = minner.minimize(method= 'nelder')

  out2 = minner.minimize(method='leastsq', params=out1.params)

  params = out2.params
  r = params['r']
  q = params['q']

  def func(t,q,r):
      A = deaths[0]**(1-q)
      return (A+(1-q)*r*t)**(1/(1-q))

  A = deaths[0]**(1-q)

  td = (-A + (2*deaths[len(deaths)-1])**(1-q))/((1-q)*r) - len(deaths)-1

  R2 = 1 - out2.residual.var() / np.var(deaths)
  global R2_general
  R2_general = R2

  return name, Type, name1, params, t0, deaths, q, r

def qexpPlotPlot1(name,Type,name1, params,t0, kind1, q, r, Forecast, togg2, x_axis1, y_axis1, toggdetails2):

  t_forecast = 1 + Forecast/100
 
  t = np.linspace(0, len(kind1)-1+Forecast, 2000)

  def func(t,q,r):
      A = kind1[0]**(1-q)
      return (A+(1-q)*r*t)**(1/(1-q))

  def funcprime(t,q,r):
      A = kind1[0]**(1-q)
      return r*(A+(1-q)*r*t)**(q/(1-q))
 
  A = kind1[0]**(1-q)
 
  td = (-A + (2*kind1[len(kind1)-1])**(1-q))/((1-q)*r) - len(kind1)-1
  t_7 = func(max(t0)+7,q,r)

  lang = html_widget_lang.value

  if lang == 'Por':
    data_primeira_morte = datetime.strptime(data_segunda_onda, '%Y-%m-%d').strftime('%d/%m/%y')
    date_last1 = data_primeira_morte + timedelta(days=len(kind1))
  if lang == 'Eng':
    data_primeira_morte = datetime.strptime(data_segunda_onda, '%Y-%m-%d').strftime('%m/%d/%y')
    date_last1 = data_primeira_morte + timedelta(days=len(kind1))

  if Type=='Deaths':
    tipo = 'Mortes'
    tipo1 = 'morte'
    tipo2 = 'a primeira morte'
  if Type=='Cases':
    tipo = 'Casos'
    tipo1 = 'caso'
    tipo2 = 'o primeiro caso'

  fig, ax = plt.subplots(figsize=(6,4))
  ax.plot(t0,kind1,'ro', alpha=0.5, label=str(subs[lang][Type])+' '+str(name))
  ax.plot(t,func(t,q,r),'-k', label=methods_name1[lang][0])
  ax.plot(t[0],kind1[0], 'wo', alpha=0, label=r'$r= %.3f \pm %.3f$' %(r.value, r.stderr))
  ax.plot(t[0],kind1[0], 'wo', alpha=0, label=r'$q= %.3f \pm %.3f$' %(q.value, q.stderr))
  ax.plot(t[0],kind1[0], 'wo', alpha=0, label=r'$t_d= %.1f$' %(td))
  ax.plot(t[0],kind1[0], 'wo', alpha=0, label=first_death[lang][Type]+str(data_primeira_morte))
  ax.plot(t[0],kind1[0], 'ko', alpha=0, label=estagio[lang])
  ax.plot(t[0],kind1[0], 'o', alpha=0, label=fase[lang][0], color='red')
  ax.plot(t[0],kind1[0], 'o', alpha=0, label=copyright_msg)
  ax.set_xlabel(label_x[lang][Type])
  ax.set_ylabel(label_y[lang][Type])
  plt.xscale(dic1[x_axis1])
  plt.yscale(dic1[y_axis1])
  if x_axis1=='Linear' and y_axis1=='Linear':
    legend = ax.legend(loc='best', fontsize='small')
    line = legend.get_lines()
    text = legend.get_texts()
    text[-1].set_color(line[-1].get_color())
  if x_axis1=='Logarithmic':
    ax.set_xlim(left=0.8)
  if y_axis1=='Logarithmic':
    ax.set_ylim(bottom=0.8)
  plt.title(name, y=1.04)
  plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last1+')', fontsize=8, ha='center')
  plt.tight_layout()
  plt.savefig('fig.png', format='png', dpi=200)
  ax.set_rasterized(True)
  plt.savefig('fig.eps', format='eps')
  plt.show()

  if toggdetails2 == True:
    variables = []
    for key, value in params.valuesdict().items():
      variables.append(key)

    ti_names = []
    ti_values = []

    ti_names.append('td')
    ti_values.append(td)
      
    for i in range(len(ti_names), 2):
      ti_names.append('')
      ti_values.append('')


    values = []
    stderr = []

    for i in range(2):
      values.append(params[variables[i]].value)
      stderr.append(params[variables[i]].stderr)

    if lang == 'Eng':
      d = {'Parameter':variables, 'Value':values, 'StdErr':stderr, 'ti':ti_names, 'ti_values':ti_values}
    else:
      d = {'Parâmetro':variables, 'Valor':values, 'Erro':stderr, 'ti':ti_names, 'valor_ti':ti_values}

    df_table = pd.DataFrame({ key:pd.Series(value) for key, value in d.items() })
    df_table.fillna('')
    pd.set_option('display.colheader_justify', 'center')
    print(df_table)
    fig, ax = plt.subplots(figsize=(8,4))
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    ax.table(cellText=df_table.values, colLabels=df_table.columns, loc='center', cellLoc='center', fontsize=12)
    fig.tight_layout()
    plt.savefig('fig2.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig2.eps', format='eps')
    plt.close(fig)
  
  if togg2==True:
    daily=[]
    for i in range(0,len(kind1)-1):
      daily.append(kind1[i+1]-kind1[i])
    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(t0[0:len(t0)-1],daily,'ro-', markersize=4, alpha=0.5, label=label_y_daily[lang][Type]+' '+str(name))
    ax.plot(t,funcprime(t,q,r), '-k', label=methods_name1[lang][0])
    if x_axis1=='Linear' and y_axis1=='Linear':
      legend = ax.legend(loc='best', fontsize='small')
      pts = ax.dataLim.get_points()
      pts[0, 1] = 0 
      ax.dataLim.set_points(pts)
      ax.autoscale_view()
    if x_axis1=='Logarithmic':
      ax.set_xlim(left=0.8)
    if y_axis1=='Logarithmic':
      ax.set_ylim(bottom=0.8)
    ax.set_xlabel(label_x[lang][Type])
    ax.set_ylabel(label_y_daily[lang][Type])
    ax.set_title(name+' '+methods_name[lang][0],y=1.05)
    plt.figtext(0.51,0.9,time_dic[lang]+' UTC-3', fontsize=8,ha='center')
    plt.xscale(dic1[x_axis1])
    plt.yscale(dic1[y_axis1])
    plt.title(name, y=1.04)
    plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last1+')', fontsize=8, ha='center')
    plt.tight_layout()
    plt.savefig('fig1.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig1.eps', format='eps')
    plt.show()

def BLM_fit1(name,Type,name1):
    kind1 = data

    kind1 = kind1[0:len(kind1)-1-index]

    x = np.arange(0,len(kind1),1)

    daily = []

    daily.append(kind1[0])

    for j in range(0,len(kind1)-1):
        delta = kind1[j+1] - kind1[j]
        daily.append(delta)

    def f(y,beta,q,alpha):
        return hyp2f1(beta,(1-q)/alpha,1+(1-q)/alpha,y)


    def f_imp(C,C0,beta,q,alpha,r,K):
        return (C**(1-q)*f((C/K)**alpha,beta,q,alpha) - (C0)**(1-q)*f((C0/K)**alpha,beta,q,alpha))/(r*(1-q))



    def fcn2min(params, x, data):
      """hyper-generalized logistic Model (HGLM)."""
      K = params['K']
      alpha = params['alpha']
      q = params['q']
      r = params['r']
      C0=kind1[0]
      beta=params['beta']
      model = f_imp(x,C0,beta,q,alpha,r,K)
      return model - data

    params = Parameters()
    params.add('K', value=1.1*max(kind1),min=max(kind1))
    params.add('alpha', value=0.6, min=0, vary = True)
    params.add('q', value=0.9, min=0.,max=.99999, vary=True)
    params.add('r', value=0.5, min=0.,vary=True)

    try:
        params.add('beta', value=1, min=1.,vary=True)
        minner = Minimizer(fcn2min, params,
                      fcn_args=(kind1, x))

        result = minner.minimize(method= 'least_squares')

        result = minner.minimize(method= 'leastsq', params = result.params)



        R2 = 1 - result.residual.var() / np.var(kind1)
        global R2_general
        R2_general = R2

        params = result.params
        alpha2=params['alpha']
        q2=params['q']
        r2=params['r']
        beta2=params['beta']
        K2=params['K']
        C02=kind1[0]

        alpha2=np.float(params['alpha'].value)
        q2=np.float(params['q'].value)
        beta2=np.float(params['beta'].value)
        r2=np.float(params['r'].value)
        K2=np.float(params['K'].value)

        c_tc = ( ( q2 )**( -1 ) * ( alpha2 * beta2 + q2 ) )**( -1 * ( alpha2 )**( -1 ) )
        C_tc = K2*c_tc

        #c_ts = np.real( ( 6 )**( -1 * ( alpha2 )**( -1 ) ) * ( ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( -1 ) * ( 2 * ( ( alpha2 )**( 3 ) * beta2 * ( -1 + 7 * beta2 ) + ( 3 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 4 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 2 ) * beta2 * ( -3 + ( 7 * q2 + beta2 * ( -7 + 18 * q2 ) ) ) ) ) ) + ( 2 * ( 2 )**( 1/3 ) * ( alpha2 )**( 2 ) * beta2 * ( -3 * ( 3 + ( 3 * alpha2 + -7 * q2 ) ) * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 3 ) * ( 13 + ( 31 * ( alpha2 )**( 2 ) + 2 * alpha2 * ( -22 + 63 * q2 ) ) ) + ( alpha2 * ( beta2 )**( 2 ) * ( -14 + ( -14 * ( alpha2 )**( 3 ) + ( 26 * q2 + ( ( alpha2 )**( 2 ) * ( -7 + 8 * q2 ) + alpha2 * ( 35 + ( -289 * q2 + 378 * ( q2 )**( 2 ) ) ) ) ) ) ) + beta2 * ( 4 + ( ( alpha2 )**( 4 ) + ( ( alpha2 )**( 3 ) * ( 6 + -14 * q2 ) + ( -14 * q2 + ( 13 * ( q2 )**( 2 ) + ( ( alpha2 )**( 2 ) * ( -5 + ( 56 * q2 + -77 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * ( -3 + ( 70 * q2 + ( -223 * ( q2 )**( 2 ) + 189 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) ) ) ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( -1/3 ) + ( 2 )**( 2/3 ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( 1/3 ) ) ) )**( ( alpha2 )**( -1 ) ) )
        #C_ts = K2*c_ts

        c_tjm = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + -1 * alpha2 * ( beta2 + ( -4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
        C_tjm = K2*c_tjm

        c_tjp = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + alpha2 * ( -1 * beta2 + ( 4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
        C_tjp = K2*c_tjp

        tc = f_imp(C_tc,C02,beta2,q2,alpha2,r2,K2)
        tjm = f_imp(C_tjm,C02,beta2,q2,alpha2,r2,K2)
        tjp = f_imp(C_tjp,C02,beta2,q2,alpha2,r2,K2)
        #ts = f_imp(C_ts,C02,beta2,q2,alpha2,r2,K2)
        global stage
        if x[-1]>tjm and x[-1]<=tc:
          stage = 'early intermediate'
        if x[-1]>tc and x[-1]<=tjp:
          stage = 'late intermediate'
        if x[-1]>tjp:
          stage = 'early saturation'
        #if x[-1]>ts:
          #stage = 'late saturation'

        alpha2=params['alpha']
        q2=params['q']
        r2=params['r']
        beta2=params['beta']
        K2=params['K']

        test_vec = result.covar
        test_vec1 = [alpha2.stderr/alpha2.value, q2.stderr/q2.value, r2.stderr/r2.value, beta2.stderr/beta2.value, K2.stderr/K2.value]
        for i in range(0, len(test_vec1)):
          if test_vec1[i]>0.2:
            raise ValueError
        Method2 = 'BLM'
        return Method2, name, Type, name1, params, x, kind1, alpha2, q2, r2, beta2, K2, C02

    except:
      try:
        params.add('beta', value=1, min=1.,vary=False)
        minner = Minimizer(fcn2min, params,
                      fcn_args=(kind1, x))

        result = minner.minimize(method= 'least_squares')

        result = minner.minimize(method= 'leastsq', params = result.params)



        R2 = 1 - result.residual.var() / np.var(kind1)
        R2_general = R2

        params = result.params
        alpha2=params['alpha']
        q2=params['q']
        r2=params['r']
        beta2=params['beta']
        K2=params['K']
        C02=kind1[0]

        alpha2=np.float(params['alpha'].value)
        q2=np.float(params['q'].value)
        beta2=np.float(params['beta'].value)
        r2=np.float(params['r'].value)
        K2=np.float(params['K'].value)

        c_tc = ( ( q2 )**( -1 ) * ( alpha2 * beta2 + q2 ) )**( -1 * ( alpha2 )**( -1 ) )
        C_tc = K2*c_tc

        #c_ts = np.real( ( 6 )**( -1 * ( alpha2 )**( -1 ) ) * ( ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( -1 ) * ( 2 * ( ( alpha2 )**( 3 ) * beta2 * ( -1 + 7 * beta2 ) + ( 3 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 4 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 2 ) * beta2 * ( -3 + ( 7 * q2 + beta2 * ( -7 + 18 * q2 ) ) ) ) ) ) + ( 2 * ( 2 )**( 1/3 ) * ( alpha2 )**( 2 ) * beta2 * ( -3 * ( 3 + ( 3 * alpha2 + -7 * q2 ) ) * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 3 ) * ( 13 + ( 31 * ( alpha2 )**( 2 ) + 2 * alpha2 * ( -22 + 63 * q2 ) ) ) + ( alpha2 * ( beta2 )**( 2 ) * ( -14 + ( -14 * ( alpha2 )**( 3 ) + ( 26 * q2 + ( ( alpha2 )**( 2 ) * ( -7 + 8 * q2 ) + alpha2 * ( 35 + ( -289 * q2 + 378 * ( q2 )**( 2 ) ) ) ) ) ) ) + beta2 * ( 4 + ( ( alpha2 )**( 4 ) + ( ( alpha2 )**( 3 ) * ( 6 + -14 * q2 ) + ( -14 * q2 + ( 13 * ( q2 )**( 2 ) + ( ( alpha2 )**( 2 ) * ( -5 + ( 56 * q2 + -77 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * ( -3 + ( 70 * q2 + ( -223 * ( q2 )**( 2 ) + 189 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) ) ) ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( -1/3 ) + ( 2 )**( 2/3 ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( 1/3 ) ) ) )**( ( alpha2 )**( -1 ) ) )
        #C_ts = K2*c_ts

        c_tjm = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + -1 * alpha2 * ( beta2 + ( -4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
        C_tjm = K2*c_tjm

        c_tjp = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + alpha2 * ( -1 * beta2 + ( 4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
        C_tjp = K2*c_tjp

        tc = f_imp(C_tc,C02,beta2,q2,alpha2,r2,K2)
        tjm = f_imp(C_tjm,C02,beta2,q2,alpha2,r2,K2)
        tjp = f_imp(C_tjp,C02,beta2,q2,alpha2,r2,K2)
        #ts = f_imp(C_ts,C02,beta2,q2,alpha2,r2,K2)
        if x[-1]>tjm and x[-1]<=tc:
          stage = 'early intermediate'
        if x[-1]>tc and x[-1]<=tjp:
          stage = 'late intermediate'
        if x[-1]>tjp:
          stage = 'early saturation'
        #if x[-1]>ts:
          #stage = 'late saturation'
        
        alpha2=params['alpha']
        q2=params['q']
        r2=params['r']
        beta2=params['beta']
        K2=params['K']

        test_vec = result.covar
        test_vec1 = [alpha2.stderr/alpha2.value, q2.stderr/q2.value, r2.stderr/r2.value, K2.stderr/K2.value]
        for i in range(0, len(test_vec1)):
          if test_vec1[i]>0.2:
            raise ValueError
        Method2 = 'GRM'
        return Method2, name, Type, name1, params, x, kind1, alpha2, q2, r2, beta2, K2, C02

      except:
        Method2 = 'RM'
        name, Type, name1, t0, kind1, K, r, alpha, C0 = RMPlot1(name,Type,name1)
        return Method2, name, Type, name1, params, t0, kind1, alpha, 1, r, 1, K, C0

def BLM_plot1(name, Type, name1, params, x, kind1, alpha2, q2, r2, beta2, K2, C02, Forecast, interv_time, interv_param, togg1, togg2, x_axis1, y_axis1, toggdetails2):
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    data = kind1
    t0 = x
    name2 = name1

    args2 = (alpha2, q2, r2,beta2, K2)

    def f_ric1(t,y,*args2): 
        return r2*(y)**q2*((1-(y/K2)**alpha2))**beta2

    def f(y,beta,q,alpha):
        return hyp2f1(beta,(1-q)/alpha,1+(1-q)/alpha,y)


    def f_imp(C,C0,beta,q,alpha,r,K):
        return (C**(1-q)*f((C/K)**alpha,beta,q,alpha) - (C0)**(1-q)*f((C0/K)**alpha,beta,q,alpha))/(r*(1-q))

    def timefit(C_curve):
        return (f_imp(C_curve*K2,C02,beta2,q2,alpha2,r2,K2) - x[-1])**2

    def timefit1(C_curve):
        return (f_imp(C_curve*K2,C02,beta2,q2,alpha2,r2,K2) - (x[-1]+Forecast))**2
  
    res = minimize_scalar(timefit, bounds=(0, 1), method='bounded')
    C_plot = res.x

    res1 = minimize_scalar(timefit1, bounds=(0, 1), method='bounded')
    C_plot1 = res1.x

    const = 0.9999 - C_plot
    t_forecast1 = const*Forecast/100
    c_forecast = np.arange(C02,C_plot*K2+1+Forecast,0.1)

    alpha3 = alpha2
    q3 = q2
    beta3 = beta2

    alpha2 = np.float(alpha2.value)
    q2 = np.float(q2.value)
    beta2 = np.float(beta2.value)

    c_tc = ( ( q2 )**( -1 ) * ( alpha2 * beta2 + q2 ) )**( -1 * ( alpha2 )**( -1 ) )
    C_tc = K2*c_tc

    #c_ts = ( 6 )**( -1 * ( alpha2 )**( -1 ) ) * ( ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( -1 ) * ( 2 * ( ( alpha2 )**( 3 ) * beta2 * ( -1 + 7 * beta2 ) + ( 3 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 4 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 2 ) * beta2 * ( -3 + ( 7 * q2 + beta2 * ( -7 + 18 * q2 ) ) ) ) ) ) + ( 2 * ( 2 )**( 1/3 ) * ( alpha2 )**( 2 ) * beta2 * ( -3 * ( 3 + ( 3 * alpha2 + -7 * q2 ) ) * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 3 ) * ( 13 + ( 31 * ( alpha2 )**( 2 ) + 2 * alpha2 * ( -22 + 63 * q2 ) ) ) + ( alpha2 * ( beta2 )**( 2 ) * ( -14 + ( -14 * ( alpha2 )**( 3 ) + ( 26 * q2 + ( ( alpha2 )**( 2 ) * ( -7 + 8 * q2 ) + alpha2 * ( 35 + ( -289 * q2 + 378 * ( q2 )**( 2 ) ) ) ) ) ) ) + beta2 * ( 4 + ( ( alpha2 )**( 4 ) + ( ( alpha2 )**( 3 ) * ( 6 + -14 * q2 ) + ( -14 * q2 + ( 13 * ( q2 )**( 2 ) + ( ( alpha2 )**( 2 ) * ( -5 + ( 56 * q2 + -77 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * ( -3 + ( 70 * q2 + ( -223 * ( q2 )**( 2 ) + 189 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) ) ) ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( -1/3 ) + ( 2 )**( 2/3 ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( 1/3 ) ) ) )**( ( alpha2 )**( -1 ) )
    #C_ts = K2*np.real(c_ts)

    c_tjm = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + -1 * alpha2 * ( beta2 + ( -4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
    C_tjm = K2*c_tjm

    c_tjp = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + alpha2 * ( -1 * beta2 + ( 4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
    C_tjp = K2*c_tjp

    alpha2 = alpha3
    q2 = q3
    beta2 = beta3

    tc = f_imp(C_tc,C02,beta2,q2,alpha2,r2,K2)
    tjm = f_imp(C_tjm,C02,beta2,q2,alpha2,r2,K2)
    tjp = f_imp(C_tjp,C02,beta2,q2,alpha2,r2,K2)
    #ts = f_imp(C_ts,C02,beta2,q2,alpha2,r2,K2) 

    data_segunda_onda1 = datetime.strptime(data_segunda_onda, '%Y-%m-%d')
    data_primeiro_pico = data_segunda_onda1 + timedelta(days=int(tc))
    date_last1 = data_segunda_onda1 + timedelta(days=len(kind1))

    lang = html_widget_lang.value

    if lang == 'Por':
      data_primeira_morte = datetime.strptime(data_segunda_onda, '%Y-%m-%d').strftime('%d/%m/%y')
      data_primeiro_pico = data_primeiro_pico.strftime('%d/%m/%y')
      date_last1 = date_last1.strftime('%d/%m/%y')
    if lang == 'Eng':
      data_primeira_morte = datetime.strptime(data_segunda_onda, '%Y-%m-%d').strftime('%m/%d/%y')
      data_primeiro_pico = data_primeiro_pico.strftime('%m/%d/%y')
      date_last1 = date_last1.strftime('%m/%d/%y')

    if Type=='Deaths':
      tipo = 'Mortes'
      tipo1 = 'morte'
      tipo2 = 'a primeira morte'
    if Type=='Cases':
      tipo = 'Casos'
      tipo1 = 'caso'
      tipo2 = 'o primeiro caso'

    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(x,data, 'ro',alpha=0.5, label = str(subs[lang][Type])+' '+str(name))
    ax.plot(f_imp(c_forecast,C02,beta2,q2,alpha2,r2,K2),c_forecast, '-k', label = methods_name1[lang][3])
    ax.plot(x[0],data[0], 'wo', alpha=0, label=first_death[lang][Type]+str(data_primeira_morte))
    if Forecast>0:
      ax.plot(x[0],data[0], 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+'+'+str(int(C_plot1*K2-C_plot*K2))+' '+subs[lang][Type])
      ax.plot(x[0],data[0], 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+str(int(C_plot1*K2))+' '+subs[lang][Type])
    ax.plot(x[0],data[0], 'ko', alpha=0, label=estagio[lang])
    global stage
    if stage == 'early intermediate':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][1], color='darkorange')
    if stage == 'late intermediate':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][2], color='yellow')
    if stage == 'early saturation':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      ax.plot(np.ones(100)*tjp,np.linspace(0,C_tjp,100), '-', alpha=1, color='green')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][3], color='green')
    #if stage == 'late saturation':
      #ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      #ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      #ax.plot(np.ones(100)*tjp,np.linspace(0,C_tjp,100), '-', alpha=1, color='green')
      #ax.plot(np.ones(100)*ts,np.linspace(0,C_ts,100), '-', alpha=1, color='royalblue')
      #ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][4], color='royalblue')
    ax.plot(x[0],data[0], 'o', alpha=0, label=copyright_msg)
    ax.set_xlabel(label_x[lang][Type])
    ax.set_ylabel(label_y[lang][Type])
    if x_axis1=='Linear' and y_axis1=='Linear':
      legend = ax.legend(loc='best', fontsize='small')
      line = legend.get_lines()
      text = legend.get_texts()
      text[len(text)-2].set_color(line[len(text)-2].get_color())
    if x_axis1=='Logarithmic':
      ax.set_xlim(left=0.8)
    if y_axis1=='Logarithmic':
      ax.set_ylim(bottom=0.8)
    plt.xscale(dic1[x_axis1])
    plt.yscale(dic1[y_axis1])
    plt.title(name, y=1.04)
    plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last1+')', fontsize=8, ha='center')
    plt.tight_layout()
    plt.savefig('fig.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig.eps', format='eps')
    plt.show()

    if toggdetails2 == True:
      variables = []
      for key, value in params.valuesdict().items():
        variables.append(key)

      ti_names = []
      ti_values = []
      ti_vec = [tjm, tc, tjp]

      stages_dic = {'early intermediate':1, 'late intermediate':2, 'early saturation':3}

      for i in range(stages_dic[stage]):
        ti_names.append('t'+str(i+1))
        ti_values.append(ti_vec[i])
        
      for i in range(len(ti_names), 5):
        ti_names.append('')
        ti_values.append('')


      values = []
      stderr = []

      for i in range(5):
        values.append(params[variables[i]].value)
        stderr.append(params[variables[i]].stderr)

      if lang == 'Eng':
        d = {'Parameter':variables, 'Value':values, 'StdErr':stderr, 'ti':ti_names, 'ti_values':ti_values}
      else:
        d = {'Parâmetro':variables, 'Valor':values, 'Erro':stderr, 'ti':ti_names, 'valor_ti':ti_values}

      df_table = pd.DataFrame({ key:pd.Series(value) for key, value in d.items() })
      df_table.fillna('')
      pd.set_option('display.colheader_justify', 'center')
      print(df_table)
      fig, ax = plt.subplots(figsize=(8,4))
      fig.patch.set_visible(False)
      ax.axis('off')
      ax.axis('tight')
      ax.table(cellText=df_table.values, colLabels=df_table.columns, loc='center', cellLoc='center', fontsize=12)
      fig.tight_layout()
      plt.savefig('fig2.png', format='png', dpi=200)
      ax.set_rasterized(True)
      plt.savefig('fig2.eps', format='eps')
      plt.close(fig)
    
    if togg2==True:
      daily=[]
      daily.append(data[0])
      for i in range(0,len(data)-1):
        daily.append(data[i+1]-data[i])
      fig, ax = plt.subplots(figsize=(6,4))
      ax.plot(x[0:len(x)],daily,'ro-', markersize=4, alpha=0.5, label=label_y_daily[lang][Type]+' '+str(name))
      ax.plot(f_imp(c_forecast,C02,beta2,q2,alpha2,r2,K2),f_ric1(0,c_forecast,*args2), '-k', label=methods_name1[lang][3])
      ax.plot(tc,f_ric1(0,C_tc,*args2), 'o', color='yellow', makersize=8, alpha=1)
      ax.plot(tc,f_ric1(0,C_tc,*args2), 'ok', makersize=8, alpha=0, label=first_peak[lang]+str(data_primeiro_pico))
      if x_axis1=='Linear' and y_axis1=='Linear':
        legend = ax.legend(loc='best', fontsize='small')
        pts = ax.dataLim.get_points()
        pts[0, 1] = 0 
        ax.dataLim.set_points(pts)
        ax.autoscale_view()
      if x_axis1=='Logarithmic':
        ax.set_xlim(left=0.8)
      if y_axis1=='Logarithmic':
        ax.set_ylim(bottom=0.8)
      ax.set_xlabel(label_x[lang][Type])
      ax.set_ylabel(label_y_daily[lang][Type])
      ax.set_title(name+' '+methods_name[lang][3],y=1.05)
      plt.figtext(0.51,0.9,time_dic[lang]+' UTC-3', fontsize=8,ha='center')
      plt.xscale(dic1[x_axis1])
      plt.yscale(dic1[y_axis1])
      plt.title(name, y=1.04)
      plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last1+')', fontsize=8, ha='center')
      plt.tight_layout()
      plt.savefig('fig1.png', format='png', dpi=200)
      ax.set_rasterized(True)
      plt.savefig('fig1.eps', format='eps')
      plt.show()

def RMPlotPlot1(name, Type, name1, params, t0, kind1, K, r, alpha, C0, Forecast, interv_time, interv_param, togg1, togg2, x_axis1, y_axis1, toggdetails2):

  t_forecast = 1 + Forecast/100
 
  t = np.arange(0, len(kind1)+Forecast, 1)

  def func(t,r,alpha,K):
      tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
      return K/(1+alpha*np.exp(-alpha*r*(t-tc)))**(1/alpha)
  
  alpha1 = alpha
  K1 = K
  r1 = r

  K = np.float(K.value)
  alpha = np.float(alpha.value)
  r = np.float(r.value)
  
  tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
  tc = np.float(tc)
  tjp = tc - np.log(1.5 + 0.5*alpha - 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
  tjm = tc - np.log(1.5 + 0.5*alpha + 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
  #ts = np.real(( alpha )**( -1 ) * ( r )**( -1 ) * np.log( ( 1/3 * ( 7 + 4 * alpha ) * ( np.e )**( alpha * r * tc ) + ( -1/3 * ( 2 )**( 1/3 ) * ( -31 * ( np.e )**( 2 * alpha * r * tc ) + ( -44 * alpha * ( np.e )**( 2 * alpha * r * tc ) + -13 * ( alpha )**( 2 ) * ( np.e )**( 2 * alpha * r * tc ) ) ) * ( ( 335 * ( np.e )**( 3 * alpha * r * tc ) + ( 708 * alpha * ( np.e )**( 3 * alpha * r * tc ) + ( 465 * ( alpha )**( 2 ) * ( np.e )**( 3 * alpha * r * tc ) + ( 92 * ( alpha )**( 3 ) * ( np.e )**( 3 * alpha * r * tc ) + 3 * ( 3 )**( 1/2 ) * ( ( -257 * ( np.e )**( 6 * alpha * r * tc ) + ( -1224 * alpha * ( np.e )**( 6 * alpha * r * tc ) + ( -2122 * ( alpha )**( 2 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -1712 * ( alpha )**( 3 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -681 * ( alpha )**( 4 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -136 * ( alpha )**( 5 ) * ( np.e )**( 6 * alpha * r * tc ) + -12 * ( alpha )**( 6 ) * ( np.e )**( 6 * alpha * r * tc ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) )**( -1/3 ) + 1/3 * ( 2 )**( -1/3 ) * ( ( 335 * ( np.e )**( 3 * alpha * r * tc ) + ( 708 * alpha * ( np.e )**( 3 * alpha * r * tc ) + ( 465 * ( alpha )**( 2 ) * ( np.e )**( 3 * alpha * r * tc ) + ( 92 * ( alpha )**( 3 ) * ( np.e )**( 3 * alpha * r * tc ) + 3 * ( 3 )**( 1/2 ) * ( ( -257 * ( np.e )**( 6 * alpha * r * tc ) + ( -1224 * alpha * ( np.e )**( 6 * alpha * r * tc ) + ( -2122 * ( alpha )**( 2 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -1712 * ( alpha )**( 3 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -681 * ( alpha )**( 4 ) * ( np.e )**( 6 * alpha * r * tc ) + ( -136 * ( alpha )**( 5 ) * ( np.e )**( 6 * alpha * r * tc ) + -12 * ( alpha )**( 6 ) * ( np.e )**( 6 * alpha * r * tc ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) )**( 1/3 ) ) ) ))

  alpha = alpha1
  K = K1
  r = r1

  def funcprime(t,r,alpha,K):
    tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
    return (K*r*alpha*np.exp(-alpha*r*(t-tc)))/(1+alpha*np.exp(-alpha*r*(t-tc)))**(1 + 1/alpha)
 
  data_segunda_onda1 = datetime.strptime(data_segunda_onda, '%Y-%m-%d')
  data_primeiro_pico = data_segunda_onda1 + timedelta(days=int(tc))
  date_last1 = data_segunda_onda1 + timedelta(days=len(kind1))

  lang = html_widget_lang.value

  if lang == 'Por':
    data_primeira_morte = datetime.strptime(data_segunda_onda, '%Y-%m-%d').strftime('%d/%m/%y')
    data_primeiro_pico = data_primeiro_pico.strftime('%d/%m/%y')
    date_last1 = date_last1.strftime('%d/%m/%y')
  if lang == 'Eng':
    data_primeira_morte = datetime.strptime(data_segunda_onda, '%Y-%m-%d').strftime('%m/%d/%y')
    data_primeiro_pico = data_primeiro_pico.strftime('%m/%d/%y')
    date_last1 = date_last1.strftime('%m/%d/%y')

  if Type=='Deaths':
    tipo = 'Mortes'
    tipo1 = 'morte'
    tipo2 = 'a primeira morte'
  if Type=='Cases':
    tipo = 'Casos'
    tipo1 = 'caso'
    tipo2 = 'o primeiro caso'

  curve_forecast = func(t,r,alpha,K)
  curve = func(np.arange(0, len(kind1), 1),r,alpha,K) 
  increment = curve_forecast[-1] - curve[-1]
  
  fig, ax = plt.subplots(figsize=(6,4))
  ax.plot(t0,kind1,'ro', alpha=0.5, label=str(subs[lang][Type])+' '+str(name))
  ax.plot(t,func(t,r,alpha,K), '-k', label=methods_name1[lang][1])
  ax.plot(t[0],func(t[0],r,alpha,K), 'ko', alpha=0, label=first_death[lang][Type]+str(data_primeira_morte)) 
  if Forecast>0:
    ax.plot(t[0],func(t[0],r,alpha,K), 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+'+'+str(increment)+' '+subs[lang][Type])
    ax.plot(t[0],func(t[0],r,alpha,K), 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+str(curve_forecast[-1])+' '+subs[lang][Type])
  ax.plot(t[0],kind1[0], 'ko', alpha=0, label=estagio[lang])
  global stage
  if stage == 'early intermediate':
    ax.plot(np.ones(100)*tjm,np.linspace(0,func(tjm,r,alpha,K),100), '-', alpha=1, color='darkorange')
    ax.plot(t[0],kind1[0], 'o', alpha=0, label=fase[lang][1], color='darkorange')
  if stage == 'late intermediate':
    ax.plot(np.ones(100)*tjm,np.linspace(0,func(tjm,r,alpha,K),100), '-', alpha=1, color='darkorange')
    ax.plot(np.ones(100)*tc,np.linspace(0,func(tc,r,alpha,K),100), '-', alpha=1, color='yellow')
    ax.plot(t[0],kind1[0], 'o', alpha=0, label=fase[lang][2], color='yellow')
  if stage == 'early saturation':
    ax.plot(np.ones(100)*tjm,np.linspace(0,func(tjm,r,alpha,K),100), '-', alpha=1, color='darkorange')
    ax.plot(np.ones(100)*tc,np.linspace(0,func(tc,r,alpha,K),100), '-', alpha=1, color='yellow')
    ax.plot(np.ones(100)*tjp,np.linspace(0,func(tjp,r,alpha,K),100), '-', alpha=1, color='green')
    ax.plot(t[0],kind1[0], 'o', alpha=0, label=fase[lang][3], color='green')
  #if stage == 'late saturation':
    #ax.plot(np.ones(100)*tjm,np.linspace(0,func(tjm,r,alpha,K),100), '-', alpha=1, color='darkorange')
    #ax.plot(np.ones(100)*tc,np.linspace(0,func(tc,r,alpha,K),100), '-', alpha=1, color='yellow')
    #ax.plot(np.ones(100)*tjp,np.linspace(0,func(tjp,r,alpha,K),100), '-', alpha=1, color='green')
    #ax.plot(np.ones(100)*ts,np.linspace(0,func(ts,r,alpha,K),100), '-', alpha=1, color='royalblue')
    #ax.plot(t[0],kind1[0], 'o', alpha=0, label=fase[lang][4], color='royalblue')
  ax.plot(t[0],kind1[0], 'o', alpha=0, label=copyright_msg)
  ax.set_xlabel(label_x[lang][Type])
  ax.set_ylabel(label_y[lang][Type])
  ax.yaxis.set_tick_params(length=0)
  ax.xaxis.set_tick_params(length=0)
  ax.grid(b=True)
  plt.xscale(dic1[x_axis1])
  plt.yscale(dic1[y_axis1])
  if x_axis1=='Linear' and y_axis1=='Linear':
    legend = ax.legend(loc='best', fontsize='small')
    line = legend.get_lines()
    text = legend.get_texts()
    text[len(text)-2].set_color(line[len(text)-2].get_color())
  if x_axis1=='Logarithmic':
    ax.set_xlim(left=0.8)
  if y_axis1=='Logarithmic':
    ax.set_ylim(bottom=0.8)
  for spine in ('top', 'right', 'bottom', 'left'):
      ax.spines[spine].set_visible(False)
  plt.title(name, y=1.04)
  plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last1+')', fontsize=8, ha='center')
  plt.tight_layout()
  plt.savefig('fig.png', format='png', dpi=200)
  ax.set_rasterized(True)
  plt.savefig('fig.eps', format='eps')
  plt.show()

  if toggdetails2 == True:
    variables = []
    for key, value in params.valuesdict().items():
      variables.append(key)

    ti_names = []
    ti_values = []
    ti_vec = [tjm, tc, tjp]

    stages_dic = {'early intermediate':1, 'late intermediate':2, 'early saturation':3}

    for i in range(stages_dic[stage]):
      ti_names.append('t'+str(i+1))
      ti_values.append(ti_vec[i])
      
    for i in range(len(ti_names), 5):
      ti_names.append('')
      ti_values.append('')


    values = []
    stderr = []

    for i in range(5):
      values.append(params[variables[i]].value)
      stderr.append(params[variables[i]].stderr)

    if lang == 'Eng':
      d = {'Parameter':variables, 'Value':values, 'StdErr':stderr, 'ti':ti_names, 'ti_values':ti_values}
    else:
      d = {'Parâmetro':variables, 'Valor':values, 'Erro':stderr, 'ti':ti_names, 'valor_ti':ti_values}

    df_table = pd.DataFrame({ key:pd.Series(value) for key, value in d.items() })
    df_table.fillna('')
    pd.set_option('display.colheader_justify', 'center')
    print(df_table)
    fig, ax = plt.subplots(figsize=(8,4))
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    ax.table(cellText=df_table.values, colLabels=df_table.columns, loc='center', cellLoc='center', fontsize=12)
    fig.tight_layout()
    plt.savefig('fig2.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig2.eps', format='eps')
    plt.close(fig)
  
  if togg2==True:
    daily=[]
    for i in range(0,len(kind1)-1):
      daily.append(kind1[i+1]-kind1[i])
    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(t0[0:len(t0)-1],daily,'ro-', markersize=4, alpha=0.5, label=label_y_daily[lang][Type]+' '+str(name))
    ax.plot(t,funcprime(t,r,alpha,K), '-k', label=methods_name1[lang][1])
    ax.plot(tc,funcprime(tc,r,alpha,K), 'o', color='yellow', markersize=8,alpha=1)
    ax.plot(tc,funcprime(tc,r,alpha,K),'ok', markersize=8,alpha=0, label=first_peak[lang]+str(data_primeiro_pico))
    if x_axis1=='Linear' and y_axis1=='Linear':
      legend = ax.legend(loc='best', fontsize='small')
      pts = ax.dataLim.get_points()
      pts[0, 1] = 0 
      ax.dataLim.set_points(pts)
      ax.autoscale_view()
    if x_axis1=='Logarithmic':
      ax.set_xlim(left=0.8)
    if y_axis1=='Logarithmic':
      ax.set_ylim(bottom=0.8)
    ax.set_xlabel(label_x[lang][Type])
    ax.set_ylabel(label_y_daily[lang][Type])
    plt.xscale(dic1[x_axis1])
    plt.yscale(dic1[y_axis1])
    plt.title(name, y=1.04)
    plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last1+')', fontsize=8, ha='center')
    plt.tight_layout()
    plt.savefig('fig1.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig1.eps', format='eps')
    plt.show()

def GRMPlotPlot1(name, Type, name1, params, x, kind1, alpha2, q2, r2, beta2, K2, C02, Forecast, interv_time, interv_param, togg1, togg2, x_axis1, y_axis1, toggdetails2):
  with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    data = kind1
    t0 = x
    name2 = name1

    args2 = (alpha2, q2, r2,beta2, K2)

    def f_ric1(t,y,*args2): 
        return r2*(y)**q2*((1-(y/K2)**alpha2))**beta2

    def f(y,beta,q,alpha):
        return hyp2f1(beta,(1-q)/alpha,1+(1-q)/alpha,y)


    def f_imp(C,C0,beta,q,alpha,r,K):
        return (C**(1-q)*f((C/K)**alpha,beta,q,alpha) - (C0)**(1-q)*f((C0/K)**alpha,beta,q,alpha))/(r*(1-q))

    def timefit(C_curve):
        return (f_imp(C_curve*K2,C02,beta2,q2,alpha2,r2,K2) - x[-1])**2

    def timefit1(C_curve):
        return (f_imp(C_curve*K2,C02,beta2,q2,alpha2,r2,K2) - (x[-1]+Forecast))**2
  
    res = minimize_scalar(timefit, bounds=(0, 1), method='bounded')
    C_plot = res.x

    res1 = minimize_scalar(timefit1, bounds=(0, 1), method='bounded')
    C_plot1 = res1.x

    const = 0.9999 - C_plot
    t_forecast1 = const*Forecast/100
    c_forecast = np.arange(C02,C_plot*K2+1+Forecast,0.1)

    alpha3 = alpha2
    q3 = q2
    beta3 = beta2

    alpha2 = np.float(alpha2.value)
    q2 = np.float(q2.value)
    beta2 = np.float(beta2.value)

    c_tc = ( ( q2 )**( -1 ) * ( alpha2 * beta2 + q2 ) )**( -1 * ( alpha2 )**( -1 ) )
    C_tc = K2*c_tc

    #c_ts = ( 6 )**( -1 * ( alpha2 )**( -1 ) ) * ( ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( -1 ) * ( 2 * ( ( alpha2 )**( 3 ) * beta2 * ( -1 + 7 * beta2 ) + ( 3 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 4 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 2 ) * beta2 * ( -3 + ( 7 * q2 + beta2 * ( -7 + 18 * q2 ) ) ) ) ) ) + ( 2 * ( 2 )**( 1/3 ) * ( alpha2 )**( 2 ) * beta2 * ( -3 * ( 3 + ( 3 * alpha2 + -7 * q2 ) ) * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 3 ) * ( 13 + ( 31 * ( alpha2 )**( 2 ) + 2 * alpha2 * ( -22 + 63 * q2 ) ) ) + ( alpha2 * ( beta2 )**( 2 ) * ( -14 + ( -14 * ( alpha2 )**( 3 ) + ( 26 * q2 + ( ( alpha2 )**( 2 ) * ( -7 + 8 * q2 ) + alpha2 * ( 35 + ( -289 * q2 + 378 * ( q2 )**( 2 ) ) ) ) ) ) ) + beta2 * ( 4 + ( ( alpha2 )**( 4 ) + ( ( alpha2 )**( 3 ) * ( 6 + -14 * q2 ) + ( -14 * q2 + ( 13 * ( q2 )**( 2 ) + ( ( alpha2 )**( 2 ) * ( -5 + ( 56 * q2 + -77 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * ( -3 + ( 70 * q2 + ( -223 * ( q2 )**( 2 ) + 189 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) ) ) ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( -1/3 ) + ( 2 )**( 2/3 ) * ( ( 2 * ( alpha2 )**( 9 ) * ( beta2 )**( 3 ) * ( -1 + ( 21 * beta2 + ( -120 * ( beta2 )**( 2 ) + 154 * ( beta2 )**( 3 ) ) ) ) + ( 3 * ( alpha2 )**( 8 ) * ( beta2 )**( 3 ) * ( -6 + ( beta2 * ( 49 + -106 * q2 ) + ( 14 * q2 + ( -49 * ( beta2 )**( 2 ) * ( -1 + 4 * q2 ) + 2 * ( beta2 )**( 3 ) * ( -91 + 279 * q2 ) ) ) ) ) + ( 3 * ( alpha2 )**( 7 ) * ( beta2 )**( 3 ) * ( -4 + ( -14 * q2 + ( 28 * ( q2 )**( 2 ) + ( ( beta2 )**( 3 ) * ( 56 + 90 * q2 ) + ( 3 * ( beta2 )**( 2 ) * ( 32 + ( -399 * q2 + 528 * ( q2 )**( 2 ) ) ) + -2 * beta2 * ( 56 + ( -451 * q2 + 602 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( 70 * ( beta2 )**( 4 ) + ( 27 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + ( 3 * ( beta2 )**( 3 ) * ( -19 + ( -189 * q2 + 360 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 90 + ( -1596 * q2 + ( 4971 * ( q2 )**( 2 ) + -4228 * ( q2 )**( 3 ) ) ) ) + 3 * ( beta2 )**( 2 ) * ( 49 + ( 216 * q2 + ( -987 * ( q2 )**( 2 ) + 756 * ( q2 )**( 3 ) ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 5 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 3 ) * ( -46 + 70 * q2 ) + ( 12 * q2 * ( 6 + ( -35 * q2 + ( 67 * ( q2 )**( 2 ) + -42 * ( q2 )**( 3 ) ) ) ) + ( ( beta2 )**( 2 ) * ( -28 + ( 394 * q2 + ( -903 * ( q2 )**( 2 ) + 540 * ( q2 )**( 3 ) ) ) ) + beta2 * ( -26 + ( 238 * q2 + ( -1170 * ( q2 )**( 2 ) + ( 2401 * ( q2 )**( 3 ) + -1656 * ( q2 )**( 4 ) ) ) ) ) ) ) ) + ( ( alpha2 )**( 3 ) * beta2 * ( -54 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 2 * ( beta2 )**( 2 ) * ( -8 + ( 42 * q2 + ( -69 * ( q2 )**( 2 ) + 35 * ( q2 )**( 3 ) ) ) ) + 9 * beta2 * q2 * ( 12 + ( -70 * q2 + ( 144 * ( q2 )**( 2 ) + ( -119 * ( q2 )**( 3 ) + 30 * ( q2 )**( 4 ) ) ) ) ) ) ) + ( 3 * ( alpha2 )**( 4 ) * ( beta2 )**( 2 ) * ( ( beta2 )**( 2 ) * ( 28 + ( -92 * q2 + 70 * ( q2 )**( 2 ) ) ) + ( beta2 * ( 12 + ( -238 * q2 + ( 845 * ( q2 )**( 2 ) + ( -1015 * ( q2 )**( 3 ) + 360 * ( q2 )**( 4 ) ) ) ) ) + -3 * q2 * ( 18 + ( -231 * q2 + ( 868 * ( q2 )**( 2 ) + ( -1295 * ( q2 )**( 3 ) + 678 * ( q2 )**( 4 ) ) ) ) ) ) ) + 3 * ( 3 )**( 1/2 ) * ( -1 * ( alpha2 )**( 6 ) * ( beta2 )**( 2 ) * ( ( 6 * ( alpha2 )**( 3 ) * ( beta2 )**( 3 ) + ( ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) * ( -7 + 18 * q2 ) + ( q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) + 2 * alpha2 * beta2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) )**( 2 ) * ( -108 * ( q2 )**( 2 ) * ( ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) )**( 2 ) + ( 4 * beta2 * q2 * ( 2 + ( -7 * q2 + 6 * ( q2 )**( 2 ) ) ) * ( 27 + ( 63 * q2 + ( -396 * ( q2 )**( 2 ) + ( 343 * ( q2 )**( 3 ) + ( -9 * ( alpha2 )**( 2 ) * ( -3 + 7 * q2 ) + -54 * alpha2 * ( 1 + ( -7 * q2 + 9 * ( q2 )**( 2 ) ) ) ) ) ) ) ) + ( ( beta2 )**( 4 ) * ( 4 + ( 25 * ( alpha2 )**( 6 ) + ( 4 * alpha2 * ( -29 + 56 * q2 ) + ( 2 * ( alpha2 )**( 5 ) * ( -88 + 217 * q2 ) + ( ( alpha2 )**( 2 ) * ( 425 + ( -2114 * q2 + 2473 * ( q2 )**( 2 ) ) ) + ( ( alpha2 )**( 4 ) * ( 482 + ( -2534 * q2 + 3193 * ( q2 )**( 2 ) ) ) + ( alpha2 )**( 3 ) * ( -644 + ( 3990 * q2 + ( -9554 * ( q2 )**( 2 ) + 8232 * ( q2 )**( 3 ) ) ) ) ) ) ) ) ) ) + ( -2 * ( beta2 )**( 3 ) * ( 7 * ( alpha2 )**( 6 ) + ( ( alpha2 )**( 5 ) * ( -42 + 103 * q2 ) + ( -4 * ( 7 + ( -29 * q2 + 28 * ( q2 )**( 2 ) ) ) + ( 7 * ( alpha2 )**( 4 ) * ( 12 + ( -61 * q2 + 77 * ( q2 )**( 2 ) ) ) + ( alpha2 * ( 84 + ( -1142 * q2 + ( 3171 * ( q2 )**( 2 ) + -2473 * ( q2 )**( 3 ) ) ) ) + ( -1 * ( alpha2 )**( 3 ) * ( 42 + ( 365 * q2 + ( -1533 * ( q2 )**( 2 ) + 1249 * ( q2 )**( 3 ) ) ) ) + ( alpha2 )**( 2 ) * ( -63 + ( 1715 * q2 + ( -9667 * ( q2 )**( 2 ) + ( 19108 * ( q2 )**( 3 ) + -12348 * ( q2 )**( 4 ) ) ) ) ) ) ) ) ) ) ) + ( beta2 )**( 2 ) * ( -28 + ( ( alpha2 )**( 6 ) + ( -336 * q2 + ( 2284 * ( q2 )**( 2 ) + ( -4228 * ( q2 )**( 3 ) + ( 2473 * ( q2 )**( 4 ) + ( -2 * ( alpha2 )**( 4 ) * ( 15 + ( -84 * q2 + 103 * ( q2 )**( 2 ) ) ) + ( -18 * ( alpha2 )**( 3 ) * ( -6 + ( 70 * q2 + ( -201 * ( q2 )**( 2 ) + 168 * ( q2 )**( 3 ) ) ) ) + ( ( alpha2 )**( 2 ) * ( -159 + ( 1680 * q2 + ( -7154 * ( q2 )**( 2 ) + ( 13076 * ( q2 )**( 3 ) + -8471 * ( q2 )**( 4 ) ) ) ) ) + 2 * alpha2 * ( 54 + ( -126 * q2 + ( -3213 * ( q2 )**( 2 ) + ( 15344 * ( q2 )**( 3 ) + ( -23885 * ( q2 )**( 4 ) + 12348 * ( q2 )**( 5 ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) ) )**( 1/2 ) ) ) ) ) ) ) ) )**( 1/3 ) ) ) )**( ( alpha2 )**( -1 ) )
    #C_ts = K2*np.real(c_ts)

    c_tjm = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + -1 * alpha2 * ( beta2 + ( -4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
    C_tjm = K2*c_tjm

    c_tjp = ( ( ( 4 * ( alpha2 )**( 2 ) * ( beta2 )**( 2 ) + ( 2 * q2 * ( -1 + 2 * q2 ) + 2 * alpha2 * beta2 * ( -1 + 4 * q2 ) ) ) )**( -1 ) * ( ( alpha2 )**( 2 ) * beta2 + ( 2 * q2 * ( -1 + 2 * q2 ) + alpha2 * ( -1 * beta2 + ( 4 * beta2 * q2 + ( beta2 )**( 1/2 ) * ( ( 4 * q2 * ( -1 + 2 * q2 ) + beta2 * ( 1 + ( -2 * alpha2 + ( ( alpha2 )**( 2 ) + 8 * alpha2 * q2 ) ) ) ) )**( 1/2 ) ) ) ) ) )**( ( alpha2 )**( -1 ) )
    C_tjp = K2*c_tjp

    alpha2 = alpha3
    q2 = q3
    beta2 = beta3

    tc = f_imp(C_tc,C02,beta2,q2,alpha2,r2,K2)
    tjm = f_imp(C_tjm,C02,beta2,q2,alpha2,r2,K2)
    tjp = f_imp(C_tjp,C02,beta2,q2,alpha2,r2,K2)
    #ts = f_imp(C_ts,C02,beta2,q2,alpha2,r2,K2) 

    data_segunda_onda1 = datetime.strptime(data_segunda_onda, '%Y-%m-%d')
    data_primeiro_pico = data_segunda_onda1 + timedelta(days=int(tc))
    date_last1 = data_segunda_onda1 + timedelta(days=len(kind1))

    lang = html_widget_lang.value

    if lang == 'Por':
      data_primeira_morte = datetime.strptime(data_segunda_onda, '%Y-%m-%d').strftime('%d/%m/%y')
      data_primeiro_pico = data_primeiro_pico.strftime('%d/%m/%y')
      date_last1 = date_last1.strftime('%d/%m/%y')
    if lang == 'Eng':
      data_primeira_morte = datetime.strptime(data_segunda_onda, '%Y-%m-%d').strftime('%m/%d/%y')
      data_primeiro_pico = data_primeiro_pico.strftime('%m/%d/%y')
      date_last1 = date_last1.strftime('%m/%d/%y')

    if Type=='Deaths':
      tipo = 'Mortes'
      tipo1 = 'morte'
      tipo2 = 'a primeira morte'
    if Type=='Cases':
      tipo = 'Casos'
      tipo1 = 'caso'
      tipo2 = 'o primeiro caso'

    fig, ax = plt.subplots(figsize=(6,4))
    ax.plot(x,data, 'ro',alpha=0.5, label = str(subs[lang][Type])+' '+str(name))
    ax.plot(f_imp(c_forecast,C02,beta2,q2,alpha2,r2,K2),c_forecast, '-k', label = methods_name1[lang][2])
    ax.plot(x[0],data[0], 'wo', alpha=0, label=first_death[lang][Type]+str(data_primeira_morte))
    if Forecast>0:
      ax.plot(x[0],data[0], 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+'+'+str(int(C_plot1*K2-C_plot*K2))+' '+subs[lang][Type])
      ax.plot(x[0],data[0], 'wo', alpha=0, label=forecast_message[lang][str(Forecast)]+str(int(C_plot1*K2))+' '+subs[lang][Type])
    ax.plot(x[0],data[0], 'ko', alpha=0, label=estagio[lang])
    global stage
    if stage == 'early intermediate':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][1], color='darkorange')
    if stage == 'late intermediate':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][2], color='yellow')
    if stage == 'early saturation':
      ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      ax.plot(np.ones(100)*tjp,np.linspace(0,C_tjp,100), '-', alpha=1, color='green')
      ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][3], color='green')
    #if stage == 'late saturation':
      #ax.plot(np.ones(100)*tjm,np.linspace(0,C_tjm,100), '-', alpha=1, color='darkorange')
      #ax.plot(np.ones(100)*tc,np.linspace(0,C_tc,100), '-', alpha=1, color='yellow')
      #ax.plot(np.ones(100)*tjp,np.linspace(0,C_tjp,100), '-', alpha=1, color='green')
      #ax.plot(np.ones(100)*ts,np.linspace(0,C_ts,100), '-', alpha=1, color='royalblue')
      #ax.plot(x[0],data[0], 'o', alpha=0, label=fase[lang][4], color='royalblue')
    ax.plot(x[0],data[0], 'o', alpha=0, label=copyright_msg)
    ax.set_xlabel(label_x[lang][Type])
    ax.set_ylabel(label_y[lang][Type])
    if x_axis1=='Linear' and y_axis1=='Linear':
      legend = ax.legend(loc='best', fontsize='small')
      line = legend.get_lines()
      text = legend.get_texts()
      text[len(text)-2].set_color(line[len(text)-2].get_color())
    if x_axis1=='Logarithmic':
      ax.set_xlim(left=0.8)
    if y_axis1=='Logarithmic':
      ax.set_ylim(bottom=0.8)
    plt.xscale(dic1[x_axis1])
    plt.yscale(dic1[y_axis1])
    plt.title(name, y=1.04)
    plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last1+')', fontsize=8, ha='center')
    plt.tight_layout()
    plt.savefig('fig.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig.eps', format='eps')
    plt.show()

    if toggdetails2 == True:
      variables = []
      for key, value in params.valuesdict().items():
        variables.append(key)

      ti_names = []
      ti_values = []
      ti_vec = [tjm, tc, tjp]

      stages_dic = {'early intermediate':1, 'late intermediate':2, 'early saturation':3}

      for i in range(stages_dic[stage]):
        ti_names.append('t'+str(i+1))
        ti_values.append(ti_vec[i])
        
      for i in range(len(ti_names), 5):
        ti_names.append('')
        ti_values.append('')


      values = []
      stderr = []

      for i in range(5):
        values.append(params[variables[i]].value)
        stderr.append(params[variables[i]].stderr)

      if lang == 'Eng':
        d = {'Parameter':variables, 'Value':values, 'StdErr':stderr, 'ti':ti_names, 'ti_values':ti_values}
      else:
        d = {'Parâmetro':variables, 'Valor':values, 'Erro':stderr, 'ti':ti_names, 'valor_ti':ti_values}

      df_table = pd.DataFrame({ key:pd.Series(value) for key, value in d.items() })
      df_table.fillna('')
      pd.set_option('display.colheader_justify', 'center')
      print(df_table)
      fig, ax = plt.subplots(figsize=(8,4))
      fig.patch.set_visible(False)
      ax.axis('off')
      ax.axis('tight')
      ax.table(cellText=df_table.values, colLabels=df_table.columns, loc='center', cellLoc='center', fontsize=12)
      fig.tight_layout()
      plt.savefig('fig2.png', format='png', dpi=200)
      ax.set_rasterized(True)
      plt.savefig('fig2.eps', format='eps')
      plt.close(fig)
    
    if togg2==True:
      daily=[]
      daily.append(data[0])
      for i in range(0,len(data)-1):
        daily.append(data[i+1]-data[i])
      fig, ax = plt.subplots(figsize=(6,4))
      ax.plot(x[0:len(x)],daily,'ro-', markersize=4, alpha=0.5, label=label_y_daily[lang][Type]+' '+str(name))
      ax.plot(f_imp(c_forecast,C02,beta2,q2,alpha2,r2,K2),f_ric1(0,c_forecast,*args2), '-k', label=methods_name1[lang][2])
      ax.plot(tc,f_ric1(0,C_tc,*args2), 'o', color='yellow', markersize=8,alpha=1)
      ax.plot(tc,f_ric1(0,C_tc,*args2),'ok',markersize=8,alpha=0, label=first_peak[lang]+str(data_primeiro_pico))
      if x_axis1=='Linear' and y_axis1=='Linear':
        legend = ax.legend(loc='best', fontsize='small')
        pts = ax.dataLim.get_points()
        pts[0, 1] = 0 
        ax.dataLim.set_points(pts)
        ax.autoscale_view()
      if x_axis1=='Logarithmic':
        ax.set_xlim(left=0.8)
      if y_axis1=='Logarithmic':
        ax.set_ylim(bottom=0.8)
      ax.set_xlabel(label_x[lang][Type])
      ax.set_ylabel(label_y_daily[lang][Type])
      plt.xscale(dic1[x_axis1])
      plt.yscale(dic1[y_axis1])
      plt.title(name, y=1.04)
      plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last1+')', fontsize=8, ha='center')
      plt.tight_layout()
      plt.savefig('fig1.png', format='png', dpi=200)
      ax.set_rasterized(True)
      plt.savefig('fig1.eps', format='eps')
      plt.show()

def automatic_fit_cities(name,Type,name1, time_fit1):

  deaths = np.array(data)

  t0 = np.array(range(len(deaths)))

  # Cálculo da curva diária de casos/óbitos
  daily1=[]
  daily1.append(deaths[0])
  for i in range(0,len(deaths)-1):
    daily1.append(deaths[i+1]-deaths[i])

  # Suavização da curva diária
  smoother = LowessSmoother(smooth_fraction=0.1, iterations=1)
  smoother.smooth(daily1)
  daily = smoother.smooth_data[0]

  peaks, peaks1 = wave_detection(daily)

  N = len(peaks1)+1

  peaks2 = peaks1

  def joy_division(N):
    if N==1:
      globals()['t'+str(1)] = t0
      globals()['deaths'+str(1)] = deaths
    for i in range(N):
      if i==N-1:
        globals()['t'+str(i+1)] = t0
        globals()['deaths'+str(i+1)] = deaths
      else:
        globals()['t'+str(i+1)] = t0[0:peaks2[i]]
        globals()['deaths'+str(i+1)] = deaths[0:peaks2[i]]
    return

  def create_params(N, params):
    params.add('r1', value=0.2, min=0, max=0.999)
    params.add('a1', value=0.2, min=0, max=1)
    params.add('q1', value=0.3, min=0, max=0.999)
    params.add('K1', value=1.1*globals()['deaths'+str(1)][-1], min=globals()['deaths'+str(1)][-1])
    params.add('p1', value=1, min=1, vary=False)
    return

  def update_params(N, params):
    params.add('p'+str(N-1), value=1, min=1)
    params.add('r'+str(N), value=0.2, min=0, max=0.999)
    params.add('a'+str(N), value=1, min=0, max=1, vary=False)
    params.add('q'+str(N), value=0.6, min=0, max=0.999)
    params.add('K'+str(N), value=1.1*globals()['deaths'+str(N)][-1], min=globals()['deaths'+str(N)][-1])
    params.add('p'+str(N), value=1, min=1, vary=False)
    params.add('rho_'+str(N-1), value=0.01, min=0.003, max=.15)
    params.add('t_'+str(N-1), value=t0[peaks2[N-2]], min=0, max=len(globals()['t'+str(N)]))
    return

  def ODE_generator(N):
    def ODE_template(y, t, params, N):
      C = y
      r_vec = []
      a_vec = []
      q_vec = []
      K_vec = []
      p_vec = []
      rho_vec = []
      t_vec = []
      for i in range(0, N):
        r_vec.append(params['r'+str(i+1)])
        a_vec.append(params['a'+str(i+1)])
        q_vec.append(params['q'+str(i+1)])
        K_vec.append(params['K'+str(i+1)])
        p_vec.append(params['p'+str(i+1)])
      for i in range(1, N):
        rho_vec.append(params['rho_'+str(i)])
        t_vec.append(params['t_'+str(i)])
      r = r_vec[0]
      K = K_vec[0]
      a = a_vec[0]
      p = p_vec[0]
      q = q_vec[0]
      if N>1:
        for i in range(1, N):
          r += 0.5*(r_vec[i]-r_vec[i-1])*(1+np.tanh(0.5*rho_vec[i-1]*(t-t_vec[i-1])))
          K += 0.5*(K_vec[i]-K_vec[i-1])*(1+np.tanh(0.5*rho_vec[i-1]*(t-t_vec[i-1])))
          a += 0.5*(a_vec[i]-a_vec[i-1])*(1+np.tanh(0.5*rho_vec[i-1]*(t-t_vec[i-1])))
          p += 0.5*(p_vec[i]-p_vec[i-1])*(1+np.tanh(0.5*rho_vec[i-1]*(t-t_vec[i-1])))
          q += 0.5*(q_vec[i]-q_vec[i-1])*(1+np.tanh(0.5*rho_vec[i-1]*(t-t_vec[i-1])))
      dCdt = r*(C**q)*(1-(C/K)**a)**p
      return dCdt
    return ODE_template

  def generate_ODEs(N):
    for i in range(N):
      globals()['ODE'+str(i+1)] = ODE_generator(i+1)
    return

  def objective_generator(N):
    def objective_template(params, time_vec, data_vec):
      ret = odeint(globals()['ODE'+str(N)], y0, time_vec, args=(params, N))
      C = ret.T
      resid = []
      pre_resid = []
      pre_resid.append(C)
      pre_resid = np.array(pre_resid)
      resid.append(pre_resid-data_vec)
      resid = np.array(resid)
      return resid      
    return objective_template

  def generate_objectives(N):
    for i in range(N):
      globals()['func'+str(i+1)] = objective_generator(i+1)
    return

  def generate_minimizers(N, params):
    globals()['minimizer'+str(N)] = Minimizer(globals()['func'+str(N)], params, fcn_args=(globals()['t'+str(N)],globals()['deaths'+str(N)]))
    return

  y0 = deaths[0]
  joy_division(N)
  fit_params = Parameters()
  create_params(1, fit_params)
  generate_ODEs(N)
  generate_objectives(N)

  try:
    for i in range(N):
      if i==0:
        generate_minimizers(i+1, fit_params)
        globals()['out'+str(2*i+1)] = globals()['minimizer'+str(i+1)].minimize(method= 'nelder')
        globals()['out'+str(2*i+2)] = globals()['minimizer'+str(i+1)].minimize(method='least_squares', params=globals()['out'+str(2*i+1)].params)
        fit_params = globals()['out'+str(2*i+2)].params
        if N>1:
          update_params(i+2, fit_params)
      else:
        generate_minimizers(i+1, fit_params)
        globals()['out'+str(2*i+1)] = globals()['minimizer'+str(i+1)].minimize(method='least_squares')
        globals()['out'+str(2*i+2)] = globals()['minimizer'+str(i+1)].minimize(method='least_squares', params=globals()['out'+str(2*i+1)].params)
        fit_params = globals()['out'+str(2*i+2)].params
        if i<N-1:
          update_params(i+2, fit_params)
  except ValueError:
    name = 'Error'

  return name, Type, name1, deaths, fit_params, N

def automatic_plot_cities(name, Type, name1, kind1, params, N, Forecast, interv_time, interv_param, togg1, togg2, x_axis, y_axis, toggdetails2):

  t0 = np.array(range(len(kind1)))
  t = np.linspace(0, len(kind1)-1+Forecast, 2000)
  t_plot = np.linspace(0, len(kind1)-1, 2000)

  y0 = kind1[0]

  ret = odeint(globals()['ODE'+str(N)], y0, t, args=(params, N))
  C = ret.T[0]

  ret = odeint(globals()['ODE'+str(N)], y0, t_plot, args=(params, N))
  C1 = ret.T[0]
  
  daily_theo = globals()['ODE'+str(N)](C, t, params, N)

  #peaks_new, peaks_new1 = wave_detection(daily_theo)

  #N = len(peaks_new1)+1

  # Cálculo da curva diária de casos/óbitos
  daily1=[]
  daily1.append(kind1[0])
  for i in range(0,len(kind1)-1):
    daily1.append(kind1[i+1]-kind1[i])

  tck = interpolate.splrep(t, daily_theo, s=0)
  ynew = interpolate.splev(t, tck, der=0)
  yder = interpolate.splev(t, tck, der=1)
  yder2 = interpolate.splev(t, tck, der=2)
  yder3 = interpolate.splev(t, tck, der=3)

  zero_crossings1 = np.where(np.diff(np.sign(yder)))[0]
  zero_crossings2 = np.where(np.diff(np.sign(yder2)))[0]
  zero_crossings3 = np.where(np.diff(np.sign(yder3)))[0]

  date_first = datetime.strptime(data_segunda_onda, '%Y-%m-%d')
  date_last = date_first + timedelta(days=len(kind1))
  for i in range(len(zero_crossings1[1::2])):
    locals()['date_wave'+str(i+2)] = date_first + timedelta(days=int(t[zero_crossings1[2*i+1]]))
  for i in range(len(zero_crossings1[0::2])):
    locals()['date_peak'+str(i+1)] = date_first + timedelta(days=int(t[zero_crossings1[2*i]]))

  lang = html_widget_lang.value

  if lang == 'Por':
    date_first = date_first.strftime('%d/%m/%y')
    date_last = date_last.strftime('%d/%m/%y')
    for i in range(len(zero_crossings1[1::2])):
      locals()['date_wave'+str(i+2)] = locals()['date_wave'+str(i+2)].strftime('%d/%m/%y')
    for i in range(len(zero_crossings1[0::2])):
      locals()['date_peak'+str(i+1)] = locals()['date_peak'+str(i+1)].strftime('%d/%m/%y')
  if lang == 'Eng':
    date_first = date_first.strftime('%m/%d/%y')
    date_last = date_last.strftime('%m/%d/%y')
    for i in range(len(zero_crossings1[1::2])):
      locals()['date_wave'+str(i+2)] = locals()['date_wave'+str(i+2)].strftime('%m/%d/%y')
    for i in range(len(zero_crossings1[0::2])):
      locals()['date_peak'+str(i+1)] = locals()['date_peak'+str(i+1)].strftime('%m/%d/%y')

  t_comparison = []
  t_cross = []
  tjm_vec = []
  tc_vec = []
  tjp_vec = []
  ts_vec = []

  # Find tis from zeros of the derivatives
  # tc
  for i in range(len(zero_crossings1[0::2])):
    tc_vec.append([t[zero_crossings1[2*i]], zero_crossings1[2*i]])
  # t crossing 
  for i in range(len(zero_crossings1[1::2])):
    t_cross.append([t[zero_crossings1[2*i+1]], zero_crossings1[2*i+1]])
  # now we compute the tjm, tjp and ts based on their loaction relative to 
  # tc and t cross.
  # tjm
  for j in range(len(tc_vec)):
    for i in range(len(zero_crossings2[0::2])):
      tjm1 = []
      if j==0:
        if t[zero_crossings2[2*i]]<tc_vec[j][0]:
          tjm1.append([t[zero_crossings2[2*i]], zero_crossings2[2*i]])
      else:
        if (t[zero_crossings2[2*i]]<tc_vec[j][0]) and (t[zero_crossings2[2*i]]>t_cross[j-1][0]):
          tjm1.append([t[zero_crossings2[2*i]], zero_crossings2[2*i]])
      if len(tjm1)>0:
        tjm_vec.append(tjm1[-1])
  # tjp
  for j in range(len(tc_vec)):
    for i in range(len(zero_crossings2[1::2])):
      tjp1 = []
      if j==len(tc_vec)-1:
        if t[zero_crossings2[2*i+1]]>tc_vec[j][0]:
          tjp1.append([t[zero_crossings2[2*i+1]], zero_crossings2[2*i+1]])
      else:
        if (t[zero_crossings2[2*i+1]]>tc_vec[j][0]) and (t[zero_crossings2[2*i+1]]<t_cross[j][0]):
          tjp1.append([t[zero_crossings2[2*i+1]], zero_crossings2[2*i+1]])
      if len(tjp1)>0:
        tjp_vec.append(tjp1[-1])
  # ts
  for j in range(len(tc_vec)):
    for i in range(len(zero_crossings3[2::4])):
      ts1 = []
      if j==len(tc_vec)-1:
        if t[zero_crossings3[4*i+2]]>tc_vec[j][0]:
          ts1.append([t[zero_crossings3[4*i+2]], zero_crossings3[4*i+2]])
      else:
        if (t[zero_crossings3[4*i+2]]>tc_vec[j][0]) and (t[zero_crossings3[4*i+2]]<t_cross[j][0]):
          ts1.append([t[zero_crossings3[4*i+2]], zero_crossings3[4*i+2]])
      if len(ts1)>0:
        ts_vec.append(ts1[-1])

  N = len(t_cross)+1

  if N == 1:
    methods_name2 = {'Eng':'BLM fit ('+str(N)+' wave)', 'Por':'Ajuste MLB ('+str(N)+' onda)'}
  else:
    methods_name2 = {'Eng':'BLM fit ('+str(N)+' waves)', 'Por':'Ajuste MLB ('+str(N)+' ondas)'}

  wave_disc_por = [' (1a onda)', ' (2a onda)', ' (3a onda)']
  wave_disc_eng = [' (1st wave)', ' (2nd wave)', ' (3rd wave)']
  if N>3:
    for i in range(3,N):
      wave_disc_eng.append(' ('+str(i+1)+'th wave)')
      wave_disc_por.append(' ('+str(i+1)+'a onda)')
  wave_disc = {'Eng':wave_disc_eng, 'Por':wave_disc_por}

  fig, ax = plt.subplots(figsize=(8,4))
  ax.plot(t0, kind1,'ro', alpha=0.5, label=subs[lang][Type]+' '+name)
  ax.plot(t, C, 'k', alpha=1, lw=2, label=methods_name2[lang])
  ax.plot(t0, kind1, 'ro', alpha=0, label = first_death[lang][Type]+date_first)
  # plot tc
  for i in range(len(tc_vec)):
    if i < len(tc_vec)-1:
      ax.plot(np.ones(100)*t[tc_vec[i][1]], np.linspace(0,C[tc_vec[i][1]],100), '--', color='yellow')
    elif len(t_cross)>0:
      if tc_vec[-1][0]>t_cross[-1][0]:
        ax.plot(np.ones(100)*t[tc_vec[i][1]], np.linspace(0,C[tc_vec[i][1]],100), '-', color='yellow')
    #elif tc_vec[-1][0]>t_cross[-1][0]:
      #ax.plot(np.ones(100)*t[tc_vec[i][1]], np.linspace(0,C[tc_vec[i][1]],100), '-', color='yellow')
    else:
      ax.plot(np.ones(100)*t[tc_vec[i][1]], np.linspace(0,C[tc_vec[i][1]],100), '--', color='yellow')
    tc = t[tc_vec[i][1]]
  t_comparison.append(('tc', tc))
  # plot division of new wave
  for i in range(len(t_cross)):
    ax.plot(np.ones(100)*t[t_cross[i][1]], np.linspace(0,C[t_cross[i][1]],100), '-', color='black')
    ax.plot(t[t_cross[i][1]], C[t_cross[i][1]], 'ko')
    wave_date1 = {'Eng': 'Start of wave '+str(i+2)+': ', 'Por': 'Início da onda '+str(i+2)+': '}
    ax.plot(t[t_cross[i][1]], C[t_cross[i][1]], 'ko', alpha=0, label=wave_date1[lang]+locals()['date_wave'+str(i+2)])
  # plot tjm
  if len(tjm_vec)>0:
    for i in range(len(tjm_vec)):
      if i < len(tjm_vec)-1:
        ax.plot(np.ones(100)*t[tjm_vec[i][1]], np.linspace(0,C[tjm_vec[i][1]],100), '--', color='darkorange')
      elif len(t_cross)>0:
        if tjm_vec[-1][0]>t_cross[-1][0]:
          ax.plot(np.ones(100)*t[tjm_vec[i][1]], np.linspace(0,C[tjm_vec[i][1]],100), '-', color='darkorange')
      #elif tjm_vec[-1][0]>t_cross[-1][0]:
        #ax.plot(np.ones(100)*t[tjm_vec[i][1]], np.linspace(0,C[tjm_vec[i][1]],100), '-', color='darkorange')
      else:
        ax.plot(np.ones(100)*t[tjm_vec[i][1]], np.linspace(0,C[tjm_vec[i][1]],100), '--', color='darkorange')
      tjm = t[tjm_vec[i][1]]
    t_comparison.append(('tjm', tjm))
  # plot tjp
  if len(tjp_vec)>0:
    for i in range(len(tjp_vec)):
      if i < len(tjp_vec)-1:
        ax.plot(np.ones(100)*t[tjp_vec[i][1]], np.linspace(0,C[tjp_vec[i][1]],100), '--', color='green')
      elif len(t_cross)>0:
        if tjp_vec[-1][0]>t_cross[-1][0]:
          ax.plot(np.ones(100)*t[tjp_vec[i][1]], np.linspace(0,C[tjp_vec[i][1]],100), '-', color='green')
      #elif tjp_vec[-1][0]>t_cross[-1][0]:
        #ax.plot(np.ones(100)*t[tjp_vec[i][1]], np.linspace(0,C[tjp_vec[i][1]],100), '-', color='green')
      else:
        ax.plot(np.ones(100)*t[tjp_vec[i][1]], np.linspace(0,C[tjp_vec[i][1]],100), '--', color='green')
      tjp = t[tjp_vec[i][1]]
    t_comparison.append(('tjp', tjp))
  # plot ts
  #if len(ts_vec)>0:
    #for i in range(len(ts_vec)):
      #if i < len(ts_vec)-1:
        #ax.plot(np.ones(100)*t[ts_vec[i][1]], np.linspace(0,C[ts_vec[i][1]],100), '--', color='blue')
      #elif ts_vec[-1][0]>t_cross[-1][0]:
        #ax.plot(np.ones(100)*t[ts_vec[i][1]], np.linspace(0,C[ts_vec[i][1]],100), '-', color='blue')
      #else:
        #ax.plot(np.ones(100)*t[ts_vec[i][1]], np.linspace(0,C[ts_vec[i][1]],100), '--', color='blue')
      #ts = t[ts_vec[i][1]]
    #t_comparison.append(('ts', ts))
  t_final = t0[-1]
  t_comparison.append(('t_final', t_final))
  t_comparison.sort(key=lambda x: x[1])
  if len(tjm_vec)>0:
    epi_phase = [x[0] for x in t_comparison].index("t_final") - [x[0] for x in t_comparison].index("tjm")
  else:
    epi_phase = [x[0] for x in t_comparison].index("t_final") - [x[0] for x in t_comparison].index("tc") + 1
  if Forecast>0:
    ax.plot(t0[0], kind1[0], 'ko', markersize=8, alpha=0, label =forecast_message[lang][str(Forecast)]+'+'+str(int(C[-1]-C1[-1]))+' '+subs[lang][Type])
    ax.plot(t0[0], kind1[0], 'ko', markersize=8, alpha=0, label =forecast_message[lang][str(Forecast)]+str(int(C[-1]))+' '+subs[lang][Type])
  ax.plot(t0[0],kind1[0], 'o', alpha=0, label=estagio[lang])
  ax.plot(t0[0],kind1[0], 'o', alpha=0, label=fase[lang][epi_phase]+wave_disc[lang][N-1], color=colordict[epi_phase])
  ax.plot(t0[0],kind1[0], 'o', alpha=0, label=copyright_msg)
  ax.set_xlabel(label_x[lang][Type])
  ax.set_ylabel(label_y[lang][Type])
  if x_axis=='Linear' and y_axis=='Linear':
    legend = ax.legend(loc='best', fontsize='small')
    line = legend.get_lines()
    text = legend.get_texts()
    text[len(text)-2].set_color(line[len(text)-2].get_color())
  if x_axis=='Logarithmic':
    ax.set_xlim(left=0.8)
  if y_axis=='Logarithmic':
    ax.set_ylim(bottom=0.8)
  plt.xscale(dic[x_axis])
  plt.yscale(dic[y_axis])
  plt.title(name, y=1.04)
  plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
  plt.tight_layout()
  plt.savefig('fig.png', format='png', dpi=200)
  ax.set_rasterized(True)
  plt.savefig('fig.eps', format='eps')
  plt.show()

  if toggdetails2 == True:
    variables = []
    for key, value in params.valuesdict().items():
      variables.append(key)
    variables = variables[0:7*N-2]

    ti_names = []
    ti_values = []

    for i in range(N):
      ti_names.append('t1 '+wave_disc[lang][i])
      ti_names.append('t2 '+wave_disc[lang][i])
      ti_names.append('t3 '+wave_disc[lang][i])
      #ti_names.append('t4 '+wave_disc[lang][i])
      if i > len(tjm_vec)-1:
        ti_values.append('-')
      else:
        ti_values.append(int(tjm_vec[i][0]))
      if i > len(tc_vec)-1:
        ti_values.append('-')
      else:
        ti_values.append(int(tc_vec[i][0]))
      if i > len(tjp_vec)-1:
        ti_values.append('-')
      else:
        ti_values.append(int(tjp_vec[i][0]))
      #if i > len(ts_vec)-1:
        #ti_values.append('-')
      #else:
        #ti_values.append(int(ts_vec[i][0]))

    for i in range(3*N, 7*N-2):
      ti_names.append('')
      ti_values.append('')


    values = []
    stderr = []

    for i in range(7*N-2):
      values.append(params[variables[i]].value)
      stderr.append(params[variables[i]].stderr)

    if lang == 'Eng':
      d = {'Parameter':variables, 'Value':values, 'StdErr':stderr, 'ti':ti_names, 'ti_values':ti_values}
    else:
      d = {'Parâmetro':variables, 'Valor':values, 'Erro':stderr, 'ti':ti_names, 'valor_ti':ti_values}

    df_table = pd.DataFrame({ key:pd.Series(value) for key, value in d.items() })
    df_table.fillna('')
    pd.set_option('display.colheader_justify', 'center')
    print(df_table)
    fig, ax = plt.subplots(figsize=(8,4))
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')
    ax.table(cellText=df_table.values, colLabels=df_table.columns, loc='center', cellLoc='center', fontsize=12)
    fig.tight_layout()
    plt.savefig('fig2.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig2.eps', format='eps')
    plt.close(fig)

  if togg2 == True:
    fig, ax = plt.subplots(figsize=(8,4))
    ax.plot(t0, daily1,'ro', alpha=0.5, label=label_y_daily[lang][Type]+' '+name)
    ax.plot(t, daily_theo, 'k', alpha=1, lw=2, label=methods_name2[lang])
    # plot tc
    for i in range(len(zero_crossings1[0::2])):
      peak_date1 = {'Eng': 'Peak '+str(i+1)+': ', 'Por': 'Pico '+str(i+1)+': '}
      ax.plot(t[zero_crossings1[2*i]], daily_theo[zero_crossings1[2*i]], 'o', color='yellow', markersize=8, alpha=1)
      ax.plot(t[zero_crossings1[2*i]], daily_theo[zero_crossings1[2*i]], 'kv', alpha=0, label=peak_date1[lang]+locals()['date_peak'+str(i+1)])
      tc = t[zero_crossings1[2*i]]
    # plot division of new wave
    for i in range(len(zero_crossings1[1::2])):
      ax.plot(t[zero_crossings1[2*i+1]], daily_theo[zero_crossings1[2*i+1]], 'ko', markersize=8, alpha=1)
    ax.plot(t0[0],kind1[0], 'o', alpha=0, label=copyright_msg)
    ax.set_xlabel(label_x[lang][Type])
    ax.set_ylabel(label_y_daily[lang][Type])
    if x_axis=='Linear' and y_axis=='Linear':
      legend = ax.legend(loc='best', fontsize='small')
      line = legend.get_lines()
      text = legend.get_texts()
      text[len(text)-2].set_color(line[len(text)-2].get_color())
      pts = ax.dataLim.get_points()
      pts[0, 1] = 0 
      ax.dataLim.set_points(pts)
      ax.autoscale_view()
    if x_axis=='Logarithmic':
      ax.set_xlim(left=0.8)
    if y_axis=='Logarithmic':
      ax.set_ylim(bottom=0.8)
    plt.xscale(dic[x_axis])
    plt.yscale(dic[y_axis])
    plt.title(name, y=1.04)
    plt.figtext(0.55, 0.88, '('+data_dic[lang]+date_last+')', fontsize=8, ha='center')
    plt.tight_layout()
    plt.savefig('fig1.png', format='png', dpi=200)
    ax.set_rasterized(True)
    plt.savefig('fig1.eps', format='eps')
    plt.show()

country2_widget = Dropdown(description='País', options=[('Brasil','Brazil'), ('EUA','US')])
x_widget = Dropdown(description='Região',options = [('Estados','States'),('Cidades','Cities')])
y_widget = Dropdown(description='Estado')
z_widget = Dropdown(description='Cidade', options=[' '])
m_widget = Dropdown(options = [' ','q-exponential','RM', 'GRM', 'BLM'])
d_widget = Dropdown(description='Dados', options = [('Óbitos','Deaths'),('Casos','Cases')])

t1_widget = widgets.IntSlider(
    value=0,
    min=0,
    max=28,
    step=7,
    description='Tempo máx.',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=True,
    readout_format='.0f'
)

global kind
global data_segunda_onda

def update1(State,kind):
  global S1
  S1 = State
  global Estado
  Estado = df.loc[df['state']==S1]
  global data
  data = Estado[kind].loc[(Estado[kind]!=0)]
  global data_segunda_onda
  data_segunda_onda1 = Estado['date'].loc[(Estado[kind]!=0)]
  data_segunda_onda1.index = [i for i in range(0,len(data_segunda_onda1))]
  if len(data_segunda_onda1)!=0:
    data_segunda_onda = data_segunda_onda1[0]
  data.index =  [i for i in range(0,len(data))]
  return

def update2(State):
  global S2
  S2 = State
  global Estado
  Estado = df1.loc[df1['state']==S2]
  cities = Estado.groupby(['city']).aggregate(np.sum).reset_index()['city']
  z_widget.options = cities
  if z_widget.value == ' ':
    z_widget.value = cities[0]
  return

def update5(State):
  global S2
  S2 = State
  global Estado
  Estado = df1.loc[df1['state']==S2]
  return

def update3(City,kind):
  global city
  city = City
  global data
  data = Estado[Estado['city']==City][kind].loc[Estado[Estado['city']==City][kind]!=0]
  global data_segunda_onda
  data_segunda_onda1 = Estado[Estado['city']==City]['date'].loc[(Estado[Estado['city']==City][kind]!=0)]
  data_segunda_onda1.index = [i for i in range(0,len(data_segunda_onda1))]
  if len(data_segunda_onda1)!=0:
    data_segunda_onda = data_segunda_onda1[0]
  data.index =  [i for i in range(0,len(data))]
  return

def update4(City,kind):
  global city
  city = City
  global data
  data_temp = Estado[Estado['city']==City][kind].loc[Estado[Estado['city']==City][kind]!=0]
  data = pd.concat([data, data_temp])
  data.index =  [i for i in range(0,len(data))]
  return

link1 = widgets.HTML(
    value=" ",
    placeholder=' ',
    description=' ',
)

link3 = widgets.HTML(
    value=" ",
    placeholder=' ',
    description=' ',
)

link_pdf1 = widgets.HTML(
    value=" ",
    placeholder=' ',
    description=' ',
)

time_widget1 = widgets.IntSlider(
    value=100,
    min=0,
    max=100,
    step=1,
    description='Tempo fit',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.0f'   
)

def Select_method(Country2,Type,Region,State,City,time_fit1,toggfit2):
  lang = html_widget_lang.value
  b = False
  global Meth
  global data_segunda_onda
  data_segunda_onda = None
  lang = html_widget_lang.value
  html_display_error2.value=' '
  Method = ' '
  name = None
  if Country2 == 'US':
    with suppress(TypeError, ValueError, IndexError):
      global data
      url1 = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
      df3 = pd.read_csv(url1)
      df3 = df3.groupby(['Province_State']).aggregate(np.sum).reset_index()
      y_widget.options = df3['Province_State'].to_list()
      if Type == 'Deaths':
        url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'
        name1 = 'death'
      if Type == 'Cases':
        url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
        name1 = 'case'
      if Region == 'States':
        df2 = pd.read_csv(url)
        df2 = df2.groupby(['Province_State']).aggregate(np.sum).reset_index()
        data = df2[(df2['Province_State'] == State)][df2.columns.values[12:len(df2.columns.values)]].to_numpy(dtype='float')[0][np.where(df2[(df2['Province_State'] == State)][df2.columns.values[12:len(df2.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df2.columns.values)]
        name = State
        data_segunda_onda = df2.columns[12+np.where(df2[(df2['Province_State'] == State)][df2.columns.values[12:len(df2.columns.values)]].to_numpy(dtype='float')>0)[1][0]]
        data_segunda_onda = datetime.strptime(data_segunda_onda,'%m/%d/%y').strftime('%Y-%m-%d')
        z_widget.options = ' '

      if Region == 'Cities':
        df2 = pd.read_csv(url)
        z_widget.options = df2[(df2['Province_State'] == State)]['Combined_Key'].to_list()
        if z_widget.value == ' ':
          z_widget.value = z_widget.options[0]
        data = df2[(df2['Combined_Key'] == City)][df2.columns.values[12:len(df2.columns.values)]].to_numpy(dtype='float')[0][np.where(df2[(df2['Combined_Key'] == City)][df2.columns.values[12:len(df2.columns.values)]].to_numpy(dtype='float') > 0)[1][0]:len(df2.columns.values)]
        name = City
        data_segunda_onda = df2.columns[12+np.where(df2[(df2['Combined_Key'] == City)][df2.columns.values[12:len(df2.columns.values)]].to_numpy(dtype='float')>0)[1][0]]
        data_segunda_onda = datetime.strptime(data_segunda_onda,'%m/%d/%y').strftime('%Y-%m-%d')
        
  if Country2 == 'Brazil':
    y_widget.options=['AC', 'AL', 'AM', 'AP', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MG', 'MS', 'MT', 'PA', 'PB', 'PE', 'PI', 'PR', 'RJ', 'RN', 'RO', 'RR', 'RS', 'SC', 'SE', 'SP', 'TO']
    if Type == 'Deaths':
      kind = 'deaths'
      name1 = 'death'
    if Type == 'Cases':
      kind = 'totalCases'
      name1 = 'case'
    if Region == 'States':
      url = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-states.csv'
      global df
      df = pd.read_csv(url)
      update1(State,kind)
      name = State
      z_widget.options = ' '
    if Region == 'Cities':
      html_display_widget_loading_fit2.value=html_loading_data[lang]
      url2020 = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time_2020.csv.gz'
      url2021 = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time_2021.csv.gz'
      url2022 = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time_2022.csv.gz'
      url2023 = 'https://raw.githubusercontent.com/wcota/covid19br/master/cases-brazil-cities-time.csv.gz'
      url_list = [url2021, url2022, url2023]
      global df1
      dtype_dict = {'date':'object', 'state':'object', 'city':'object', 'deaths':'int32', 'totalCases':'int32'}
      dask_df1 = dd.read_csv(url2020, usecols=['date', 'state', 'city', 'deaths', 'totalCases'], dtype=dtype_dict)
      dask_df2 = dd.read_csv(url2021, usecols=['date', 'state', 'city', 'deaths', 'totalCases'], dtype=dtype_dict)
      dask_df3 = dd.read_csv(url2022, usecols=['date', 'state', 'city', 'deaths', 'totalCases'], dtype=dtype_dict)
      dask_df4 = dd.read_csv(url2023, usecols=['date', 'state', 'city', 'deaths', 'totalCases'], dtype=dtype_dict)
      df1 = dd.concat([dask_df1, dask_df2, dask_df3, dask_df4]).compute()
      update2(State)
      update3(City,kind)
      name = City

  if name != None and len(data)>0:

    if time_widget1.max == len(data)-1:
      b = True
    elif time_widget1.max != len(data)-1:
      time_widget1.max = len(data)-1
      time_widget1.value = time_widget1.max
      time_fit1 = time_widget1.value
      b = True

    #if time_widget1.max != len(data)-1:
      #time_widget1.max = len(data)-1
      #time_widget1.value = len(data)-1
      #time_fit1 = time_widget1.value
    #else:
      #b = True

    data_segunda_onda2 = datetime.strptime(data_segunda_onda, '%Y-%m-%d')
    data_segunda_onda2 = data_segunda_onda2 + timedelta(days=time_fit1)

    if lang == 'Por':
      data_segunda_onda2 = data_segunda_onda2.strftime('%d/%m/%y')
    if lang == 'Eng':
      data_segunda_onda2 = data_segunda_onda2.strftime('%m/%d/%y')


    if time_widget1.max == len(data)-1 and b == True:
      data = data[0:time_fit1]
      plt.figure(figsize=(6,4));
      plt.plot(np.array(range(len(data))),data,'ro',alpha=0.5,label=subs[lang][Type]+' '+str(name))
      plt.plot(0,data[0],'ro',alpha=0,label=date_last_message[lang]+data_segunda_onda2)
      plt.xlabel(label_x[lang][Type])
      plt.ylabel(label_y[lang][Type])
      plt.legend(loc='best')
      plt.title(name)
      plt.show()
      #b = False
      html_display_widget_loading_fit2.value=' '

    togg5_widget.value = False
    togg5_widget.disabled = False
    togg_details2.value = False
    togg_details2.disabled = False

    daily1=[]
    daily1.append(data[0])
    for i in range(0,len(data)-1):
      daily1.append(data[i+1]-data[i])

    smoother = LowessSmoother(smooth_fraction=0.1, iterations=1)
    smoother.smooth(daily1)
    daily = smoother.smooth_data[0]

    try:
      peaks, peaks1 = wave_detection(daily)

      N = len(peaks1)+1
    except:
      N=1
    finally:

      if N>1:
        Method = 'BLM-2W'
      else:
        with suppress(TypeError, ValueError, UnboundLocalError):
          name, Type, name1, t0, kind1, K, r, alpha, C0 = RMPlot1(name,Type,name1)
          tc = np.log((-1+(K/C0)**alpha)/alpha)/(alpha*r)
          tjp = tc - np.log(1.5 + 0.5*alpha - 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
          tjm = tc - np.log(1.5 + 0.5*alpha + 0.5*np.sqrt(5 + 6*alpha + alpha**2))/(r*alpha)
          if max(t0)<tjm:
            Method = 'q-exponential'
            Meth = 'q-exponential'
          if max(t0)>tjm:
            Method = 'BLM'

      if toggfit2 == True:
        html_display_widget_loading_fit2.value=html_loading_message[lang]
        if Method == 'q-exponential':
          togg2_widget.value = False
          togg3_widget.value = False
          togg5_widget.value = False
          togg2_widget.disabled = False
          togg3_widget.disabled = False
          togg5_widget.disabled = False
          togg_details2.value = False
          togg_details2.disabled = False
          link1.value = ' '
          link3.value = ' '
          link_pdf1.value = ' '
          global vec3
          vec3 = qexpPlot1(name,Type,name1)
          togg_fit2.value = False
          html_display_widget_loading_fit2.value=' '
          togg2_widget.value = True
        if Method == 'BLM':
          togg2_widget.value = False
          togg3_widget.value = False
          togg5_widget.value = False
          togg2_widget.disabled = False
          togg3_widget.disabled = False
          togg5_widget.disabled = False
          togg_details2.value = False
          togg_details2.disabled = False
          link1.value = ' '
          link3.value = ' '
          link_pdf1.value = ' '
          global vec7
          vec7 = BLM_fit1(name,Type,name1)
          Meth = vec7[0]
          togg_fit2.value = False
          html_display_widget_loading_fit2.value=' '
          togg2_widget.value = True
        if Method == 'BLM-2W':
          togg2_widget.value = False
          togg3_widget.value = False
          togg5_widget.value = False
          togg2_widget.disabled = False
          togg3_widget.disabled = False
          togg5_widget.disabled = False
          togg_details2.value = False
          togg_details2.disabled = False
          link1.value = ' '
          link3.value = ' '
          link_pdf1.value = ' '
          global vec_2wave_city
          vec_2wave_city = automatic_fit_cities(name,Type,name1,time_fit1)
          Meth = 'BLM-2W'
          togg_fit2.value = False
          html_display_widget_loading_fit2.value=' '
          togg2_widget.value = True


interact(Select_method, Country2=country2_widget, Type=d_widget, Region=x_widget, State=y_widget, City=z_widget, time_fit1=time_widget1, toggfit2 = togg_fit2);
y_widget.value = 'AC'
display(button_fit2)
display(html_display_widget_loading_fit2)
display(html_display_error2)

def PlotMethod1(togg, togg2, toggdetails2, Forecast, x_axis1, y_axis1):
  lang = html_widget_lang.value
  if togg2==False:
    link3.value = ' '
  togg1 = False
  interv_param = 0
  interv_time = 10
  if togg == True:
    if Meth == 'q-exponential':
      name,Type,name1, params,t0, kind1, q, r = vec3
      qexpPlotPlot1(name,Type,name1, params,t0, kind1, q, r, Forecast, togg2, x_axis1, y_axis1, toggdetails2)
    if Meth == 'RM':
      Method2, name, Type, name1, params, t0, kind1, alpha, q2, r, beta2, K, C0 = vec7 
      RMPlotPlot1(name, Type, name1, params, t0, kind1, K, r, alpha, C0, Forecast, interv_time, interv_param, togg1, togg2, x_axis1, y_axis1, toggdetails2)
    if Meth == 'GRM':
      Method2, name, Type, name1, params, t0, kind1, alpha2, q2, r2, beta2, K2, C02 = vec7
      GRMPlotPlot1(name, Type, name1, params, t0, kind1, alpha2, q2, r2, beta2, K2, C02, Forecast, interv_time, interv_param, togg1, togg2, x_axis1, y_axis1, toggdetails2)
    if Meth == 'BLM':
      Method2, name, Type, name1, params, t0, kind1, alpha2, q2, r2, beta2, K2, C02 = vec7
      BLM_plot1(name, Type, name1, params, t0, kind1, alpha2, q2, r2, beta2, K2, C02, Forecast, interv_time, interv_param, togg1, togg2, x_axis1, y_axis1, toggdetails2)
    if Meth == 'BLM-2W':
      name, Type, name1, kind1, params, N = vec_2wave_city
      if name != 'Error':
        automatic_plot_cities(name, Type, name1, kind1, params, N, Forecast, interv_time, interv_param, togg1, togg2, x_axis1, y_axis1, toggdetails2)
      else:
        html_display_error2.value = error_message[lang]


interv1_widget = widgets.FloatSlider(
    value=0,
    min=-1,
    max=1,
    step=0.01,
    description='Intensity',
    disabled=False,
    continuous_update=True,
    orientation='horizontal',
    readout=False,
    readout_format='.0f'
    
)

x_axis_widget1 = Dropdown(options = [('Linear','Linear'),('Logarítmico','Logarithmic')], description='Eixo-x')
y_axis_widget1 = Dropdown(options = [('Linear','Linear'),('Logarítmico','Logarithmic')], description='Eixo-y')

interact(PlotMethod1, togg = togg2_widget, togg2 = togg5_widget, toggdetails2 = togg_details2, Forecast=t1_widget, x_axis1=x_axis_widget1, y_axis1=y_axis_widget1);

format1_widget = Dropdown(options = ['png','eps'], description='Formato')

def format_selection1(fmt1):
  global FMT1
  FMT1 = fmt1

button1 = widgets.Button(description="Gerar figura")
button_pdf1 = widgets.Button(description="Generate Report")

items1 = [button1, format1_widget]
display(widgets.HBox(items1))

items_link1 = [link1, link3, link_pdf1]
display(widgets.HBox(items_link1))

def create_download_link1(file1, title):
    with open(file1, "rb") as image_file:
      b64 = base64.b64encode(image_file.read()) 
    payload = b64.decode()
    if FMT1 == 'eps':
      filename = 'plot.eps'
      html = '<a download="{filename}" href="data:image/eps;base64,{payload}" target="">{title}</a>'
    if FMT1 == 'png':
      filename = 'plot.png'
      html = '<a download="{filename}" href="data:image/png;base64,{payload}" target="">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

def create_pdf_download_link1(file, title = "Download Report"):
    with open(file, "rb") as image_file:
      b64 = base64.b64encode(image_file.read()) 
    payload = b64.decode()
    filename = 'report.pdf'
    html = '<a download="{filename}" href="data:application/pdf;base64,{payload}" target="">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return html

lang = html_widget_lang.value
def on_button_clicked1(b):
  format_selection1(format1_widget.value)
  lang = html_widget_lang.value
  if lang=='Eng':
    link1.value = create_download_link1('fig.'+str(FMT1), "Download Accumulated Curve")
  if lang=='Por':
    link1.value = create_download_link1('fig.'+str(FMT1), "Baixar Curva Acumulada")
  with suppress(FileNotFoundError):
    if lang=='Eng':
      link3.value = create_download_link1('fig1.'+str(FMT1), "Download Daily Curve")
    if lang=='Por':
      link3.value = create_download_link1('fig1.'+str(FMT1), "Baixar Curva Diária")
  with suppress(FileNotFoundError):
    if lang=='Eng':
      link_pdf1.value = create_download_link1('fig2.'+str(FMT1), "Download Fit Parameters")
    if lang=='Por':
      link_pdf1.value = create_download_link1('fig2.'+str(FMT1), "Baixar Parâmetros do Ajuste")
def on_pdf_button_clicked1(b):
  link_pdf1.value = create_pdf_download_link1('report.pdf')
button1.on_click(on_button_clicked1)
button_pdf1.on_click(on_pdf_button_clicked1)


interactive(children=(Dropdown(description='País', options=(('Brasil', 'Brazil'), ('EUA', 'US')), value='Brazi…

Button(description='Fazer ajuste', style=ButtonStyle())

HTML(value=' ')

HTML(value=' ')

interactive(children=(Checkbox(value=False, description='Marque para mostrar/atualizar o ajuste', indent=False…

In [23]:
#nbi:hide_in

html_display_widget_outro = widgets.HTML(
    value='<hr> <p>Desenvolvido por Arthur A. Brum e Gerson C. Duarte-Filho; coordenação do projeto de Giovani L. Vasconcelos. Para mais informações sobre a Rede de Pesquisa Modinterv visite nosso site em <a href="https://fisica.ufpr.br/redecovid19"> fisica.ufpr.br/redecovid19</a>.</p> <hr> <p>Os dados para países e estados/cidades dos EUA são obtidos do Repositório de Dados de COVID-19 disponiblizado pelo CSSE/Johns Hopkins University (<a href="https://systems.jhu.edu">https://systems.jhu.edu</a>). Os dados para os estados e cidades do Brasil são retirados do site <a href="https://covid19br.wcota.me/">covid19br.wcota.me</a>, mantido por Wesley Cota, da Universidade Federal de Viçosa.</p> <hr> <p>Mensages de erro: Durante a execução do aplicativo podem aparecer mensagens de erro para alguns lugares específicos. Favor reportar qualquer erro inesperado para redemodinterv@gmail.com.</p>')
display(html_display_widget_outro)
html_display_widget_license = widgets.HTML(value='<hr> <p>Copyright 2022 ModInterv</p> <p> Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: </p> <p> The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software. </p> <p> THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE. </p> <hr>' )
display(html_display_widget_license)

def on_pt_button_clicked(b):
  html_widget_lang.value = 'Por'
  html_display_widget_intro.value = '<hr><p>Este aplicativo faz ajustes matemáticos das curvas epidêmicas de países, bem como de estados e cidades do Brasil e dos EUA.</p> <p> Esta versão implementa um modelo com múltiplas ondas. Os ajustes numéricos podem demorar um tempo maior, devido ao maior número de parâmetros necessários para descrever múltiplas ondas epidêmicas. </p> <hr> <p>O aplicativo foi desenvolvido pela Rede de Pesquisa Modinterv-Covid19 formada por pesquisadores das Universidades Federais do Paraná, Pernambuco e Sergipe.</p> <hr>'
  html_display_widget_outro.value = '<hr> <p>Desenvolvido por Arthur A. Brum e Gerson C. Duarte-Filho; coordenação do projeto de Giovani L. Vasconcelos. Para mais informações sobre a Rede de Pesquisa Modinterv visite nosso site em <a href="https://fisica.ufpr.br/redecovid19">.</p> <hr> <p>Os dados para países e estados/cidades dos EUA são obtidos do Repositório de Dados de COVID-19 disponiblizado pelo CSSE/Johns Hopkins University (<a href="https://systems.jhu.edu">https://systems.jhu.edu</a>). Os dados para os estados e cidades do Brasil são retirados do site <a href="https://covid19br.wcota.me/">covid19br.wcota.me</a>, mantido por Wesley Cota, da Universidade Federal de Viçosa.</p> <hr> <p>Mensages de erro: Durante a execução do aplicativo podem aparecer mensagens de erro para alguns lugares específicos. Favor reportar qualquer erro inesperado para redemodinterv@gmail.com.</p>'
  html_display_widget_mid.value = '<hr><p>Estados e cidades do Brasil e dos EUA.</p><hr>'
  
  
  country2_widget.description = 'País'
  country1_widget.description = 'País'
  country2_widget.options = [('Brasil','Brazil'), ('EUA','US')]
  x_widget.description = 'Região'
  x_widget.options = [('Estados','States'),('Cidades','Cities')]
  y_widget.description = 'Estado'
  z_widget.description = 'Cidade'
  d_widget.description = 'Dados'
  d_widget.options = options = [('Óbitos','Deaths'),('Casos','Cases')]
  d1_widget.description = 'Dados'
  d1_widget.options = options = [('Óbitos','Deaths'),('Casos','Cases')]
  t1_widget.description = 'Tempo máx.'
  togg2_widget.description = 'Marque para mostrar/atualizar o ajuste'
  togg5_widget.description = 'Marque para mostrar/ocultar a curva diária'
  button1.description = 'Gerar figura'
  button.description = 'Gerar figura'
  format1_widget.description = 'Formato'
  format_widget.description = 'Formato'
  x_axis_widget.options = [('Linear','Linear'),('Logarítmico','Logarithmic')]
  x_axis_widget.description = 'Eixo-x'
  x_axis_widget1.options = [('Linear','Linear'),('Logarítmico','Logarithmic')]
  x_axis_widget1.description = 'Eixo-x'
  y_axis_widget.options = [('Linear','Linear'),('Logarítmico','Logarithmic')]
  y_axis_widget.description = 'Eixo-y'
  y_axis_widget1.options = [('Linear','Linear'),('Logarítmico','Logarithmic')]
  y_axis_widget1.description = 'Eixo-y'
  t_widget.description = 'Tempo máx.'
  togg_widget.description = 'Marque para mostrar/atualizar o ajuste'
  togg4_widget.description = 'Marque para mostrar/ocultar a curva diária'
  togg_details1.description = 'Marque para mostrar os parâmetros do ajuste'
  togg_details2.description = 'Marque para mostrar os parâmetros do ajuste'
  togg_fit1.description = 'Marque para fazer o ajuste'
  togg_fit2.description = 'Marque para fazer o ajuste'
  button_fit1.description = 'Fazer ajuste'
  button_fit2.description = 'Fazer ajuste'
  time_widget.description = 'Tempo fit'
  time_widget1.description = 'Tempo fit'

def on_en_button_clicked(b):
  html_widget_lang.value = 'Eng'
  html_display_widget_intro.value = '<hr><p>This app implements mathematical fits of the COVID-19 epidemic curves for countries as well as states and cities in Brazil and the US.</p> <p> This version implements a multiple-wave model. The fits may take some time to complete, owing to the larger number of parameters needed to describe multiple epidemic waves. </p> <hr> <p>This app has been set up by the MODINTERV-COVID19 Research Network at the Federal Universities of Paraná, Pernambuco, and Sergipe, Brazil.</p> <hr>'
  html_display_widget_outro.value = '<hr> Developed by Arthur A. Brum and Gerson C. Duarte-Filho; project coordination by Giovani L. Vasconcelos. For more information about our research network visit our website at <a href="https://fisica.ufpr.br/redecovid19"> fisica.ufpr.br/redecovid19</a>. <hr> The data for countries as well as for states/cities in the US are fetched from the COVID-19 Data Repository by CSSE/Johns Hopkins University (https://systems.jhu.edu), and the data for the Brazilian states and cities are from the site <a href="https://covid19br.wcota.me/">covid19br.wcota.me</a>, maintained by Wesley Cota, from the Federal University of Viçosa, Brazil. <hr> <p>Troubleshooting: There may appear an error message when trying to produce fits for some specific places. Please report any unexpected error to redemodinterv@gmail.com.</p>'
  html_display_widget_mid.value = '<hr><p>States and cities in Brazil and the US.</p><hr>'
  
  
  country2_widget.description = 'Country'
  country1_widget.description = 'Country'
  country2_widget.options = [('Brazil','Brazil'), ('US','US')]
  x_widget.description = 'Region'
  x_widget.options = [('States','States'),('Cities','Cities')]
  y_widget.description = 'State'
  z_widget.description = 'City'
  d_widget.description = 'Data'
  d_widget.options = options = [('Deaths','Deaths'),('Cases','Cases')]
  d1_widget.description = 'Data'
  d1_widget.options = options = [('Deaths','Deaths'),('Cases','Cases')]
  t1_widget.description = 'Time range'
  togg2_widget.description = 'Check to display/update the model fit'
  togg5_widget.description = 'Check to display/hide the daily curve'
  button1.description = 'Generate figure file'
  button.description = 'Generate figure file'
  format1_widget.description = 'Format'
  format_widget.description = 'Format'
  x_axis_widget.options = ['Linear','Logarithmic']
  x_axis_widget.description = 'x-Axis'
  x_axis_widget1.options = ['Linear','Logarithmic']
  x_axis_widget1.description = 'x-Axis'
  y_axis_widget.options = ['Linear','Logarithmic']
  y_axis_widget.description = 'y-Axis'
  y_axis_widget1.options = ['Linear','Logarithmic']
  y_axis_widget1.description = 'y-Axis'
  t_widget.description = 'Time range'
  togg_widget.description = 'Check to display/update the model fit'
  togg4_widget.description = 'Check to display/hide the daily curve'
  togg_details1.description = 'Check to display the parameters of the fit'
  togg_details2.description = 'Check to display the parameters of the fit'
  togg_fit1.description = 'Check to perform the fit'
  togg_fit2.description = 'Check to perform the fit'
  button_fit1.description = 'Perform fit'
  button_fit2.description = 'Perform fit'
  time_widget.description = 'Fit time'
  time_widget1.description = 'Fit time'

button_pt.on_click(on_pt_button_clicked)
button_en.on_click(on_en_button_clicked)

on_en_button_clicked(1)

HTML(value='<hr> <p>Desenvolvido por Arthur A. Brum e Gerson C. Duarte-Filho; coordenação do projeto de Giovan…

HTML(value='<hr> <p>Copyright 2022 ModInterv</p> <p> Permission is hereby granted, free of charge, to any pers…